## Group 52 - Predicting Isoform Expression from Gene-Level Profiles using Representation Learning
### Students: Alessandra Carrara s252976 - Alessia Santoni sS252125 - Emmanouil Tsilingeridis s253154 - Matteo Pozzar s243098


Since our dataset is too big to be imported in GitHub we just decided just to report the code we used and the relative outputs. This was discussed and agreed with the TA. We excluded the option of importing just a small amount of data because the results obtained wouldn't be the same as the ones in the official report.

**PLEASE NOTE:** The code was adapted in order to be runned in a Jupiter Notebook. The original code can be found on the same repository on GitHub in the *Pipeline* folder.

In [ ]:
from __future__ import annotations

import argparse
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

try:
    import scanpy as sc
except ImportError as exc:  # pragma: no cover
    raise SystemExit("scanpy is required. Install it via `pip install scanpy`.") from exc

# Base directory containing the AnnData files.
DATA_ROOT = Path("/work3/s193518/scIsoPred/data").expanduser().resolve()

DEFAULT_GENE_H5AD = str(DATA_ROOT / "bulk_processed_genes.h5ad") 
DEFAULT_ISOFORM_H5AD = str(DATA_ROOT / "bulk_processed_transcripts.h5ad") 

In the cell above we imported the data and defined the data path.

## Loader Utilities

Same for every model

In [ ]:
class GeneIsoformDataset(Dataset):
    """Wrap numpy arrays for torch consumption."""

    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()

    def __len__(self) -> int:
        return self.X.shape[0]

    def __getitem__(self, idx: int):
        return self.X[idx], self.Y[idx]


def train_val_test_split(
    X: np.ndarray,
    Y: np.ndarray,
    train_n: int,
    val_n: int,
    test_n: int,
    seed: int = 42,
) -> Tuple[Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray], Tuple[np.ndarray, np.ndarray]]:
    """Return deterministic subset splits."""
    n = X.shape[0]
    total_needed = train_n + val_n + test_n
    if total_needed > n:
        raise ValueError(
            f"Requested {total_needed} samples (train {train_n}, val {val_n}, test {test_n}) "
            f"but dataset only has {n} observations."
        )
    rng = np.random.default_rng(seed)
    perm = rng.permutation(n)[:total_needed]
    X = X[perm]
    Y = Y[perm]
    train = (X[:train_n], Y[:train_n])
    val = (X[train_n : train_n + val_n], Y[train_n : train_n + val_n])
    test = (X[-test_n:], Y[-test_n:])
    return train, val, test


def make_loader(pair, batch_size: int, shuffle: bool) -> DataLoader:
    dataset = GeneIsoformDataset(*pair)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=False)


@dataclass
class GeneIsoformDataLoaders:
    train: DataLoader
    train_eval: DataLoader
    val: DataLoader
    test: DataLoader


## Preprocess Utilities:

Same for every model

In [ ]:
def densify(adata) -> np.ndarray:
    """Return a dense float32 numpy array for AnnData.X regardless of storage."""
    matrix = adata.X
    if hasattr(matrix, "toarray"):
        matrix = matrix.toarray()
    return np.asarray(matrix, dtype=np.float32)


def align_anndata(genes, transcripts):
    """Ensure gene/isoform AnnData objects share the same obs order."""
    if np.array_equal(genes.obs_names, transcripts.obs_names):
        return genes, transcripts
    print("Reindexing isoform AnnData to match gene observations order.")
    transcripts = transcripts[genes.obs_names, :]
    return genes, transcripts


def normalize_inputs(X: np.ndarray) -> np.ndarray:
    """Apply log1p transform followed by z-scoring per gene."""
    X = np.log1p(X)
    mean = X.mean(axis=0, keepdims=True)
    std = X.std(axis=0, keepdims=True) + 1e-6
    return (X - mean) / std


def prepare_targets(Y: np.ndarray) -> np.ndarray:
    """Log-transform isoform counts so the network predicts log counts."""
    return np.log1p(Y)


def filter_silent_genes_isoforms(
    X_genes: np.ndarray,
    Y_isoforms: np.ndarray,
    gene_names: Sequence[str],
    transcript_names: Sequence[str],
    gene_to_transcripts: Dict[str, Sequence[str]] | None = None,
):
    """
    Remove genes with zero expression and isoforms with zero observations or missing genes.

    Returns filtered X, Y, gene names, transcript names, and an updated gene->transcripts mapping
    containing only kept entries.
    """
    gene_to_transcripts = gene_to_transcripts or {}
    gene_names = np.asarray(gene_names)
    transcript_names = np.asarray(transcript_names)

    gene_mask = X_genes.sum(axis=0) > 0
    kept_genes = gene_names[gene_mask]
    kept_gene_set = set(kept_genes)

    transcript_to_gene = {}
    for gene, transcripts in gene_to_transcripts.items():
        for tr in transcripts:
            transcript_to_gene[tr] = gene

    isoform_mask = np.zeros_like(transcript_names, dtype=bool)
    for idx, tr in enumerate(transcript_names):
        has_counts = Y_isoforms[:, idx].sum() > 0
        parent_gene = transcript_to_gene.get(tr)
        isoform_mask[idx] = bool(has_counts and parent_gene in kept_gene_set)

    filtered_transcripts = transcript_names[isoform_mask]
    filtered_gene_to_transcripts: Dict[str, List[str]] = {}
    filtered_transcript_set = set(filtered_transcripts)
    for gene in kept_genes:
        gene_transcripts = gene_to_transcripts.get(gene, [])
        kept_trs = [tr for tr in gene_transcripts if tr in filtered_transcript_set]
        if kept_trs:
            filtered_gene_to_transcripts[gene] = kept_trs

    return (
        X_genes[:, gene_mask],
        Y_isoforms[:, isoform_mask],
        kept_genes,
        filtered_transcripts,
        filtered_gene_to_transcripts,
    )


def build_transcript_gene_index(
    gene_to_transcripts: Dict[str, Sequence[str]],
    gene_names: Sequence[str],
    transcript_names: Sequence[str],
) -> Tuple[np.ndarray, List[str]]:
    """Return an array mapping transcript column indices to gene indices."""
    gene_index = {gene: idx for idx, gene in enumerate(gene_names)}
    transcript_gene_idx = np.full(len(transcript_names), -1, dtype=np.int32)
    unmapped: List[str] = []
    transcript_to_gene = {}
    for gene, transcripts in gene_to_transcripts.items():
        for transcript in transcripts:
            transcript_to_gene[transcript] = gene
    for transcript_id, transcript in enumerate(transcript_names):
        parent_gene = transcript_to_gene.get(transcript)
        if parent_gene is None or parent_gene not in gene_index:
            unmapped.append(transcript)
            continue
        transcript_gene_idx[transcript_id] = gene_index[parent_gene]
    return transcript_gene_idx, unmapped

def isoform_correlations(pred_counts: np.ndarray, true_counts: np.ndarray) -> np.ndarray:
    """
    Compute Pearson correlation per isoform across samples.

    Returns an array of shape (n_isoforms,) with NaN where variance is zero
    (pred or true) so that downstream stats can ignore those entries.
    """
    n_isoforms = pred_counts.shape[1]
    corrs = np.full(n_isoforms, np.nan, dtype=np.float32)
    for i in range(n_isoforms):
        x = pred_counts[:, i]
        y = true_counts[:, i]
        if x.std() == 0 or y.std() == 0:
            continue
        corrs[i] = float(np.corrcoef(x, y)[0, 1])
    return corrs

def gene_spearman_per_sample(
    pred_counts: np.ndarray,
    true_counts: np.ndarray,
    transcript_gene_idx: np.ndarray,
    gene_names: Sequence[str],
) -> Tuple[np.ndarray, np.ndarray]:
    """
    For each gene, compute Spearman correlation of isoform ranks per observation.

    Returns two arrays of shape (n_genes,):
    - mean correlation per gene across observations
    - median correlation per gene across observations
    NaN for genes with <2 isoforms or no valid correlations.
    """
    gene_names = np.asarray(gene_names)
    n_genes = len(gene_names)
    mean_corrs = np.full(n_genes, np.nan, dtype=np.float32)
    median_corrs = np.full(n_genes, np.nan, dtype=np.float32)

    for g in range(n_genes):
        mask = transcript_gene_idx == g
        if mask.sum() < 2:
            continue
        corrs: List[float] = []
        p_sub = pred_counts[:, mask]
        t_sub = true_counts[:, mask]
        for p, t in zip(p_sub, t_sub):
            if np.all(p == p[0]) or np.all(t == t[0]):
                continue
            corr = spearmanr(p, t).correlation
            if np.isnan(corr):
                continue
            corrs.append(float(corr))
        if corrs:
            mean_corrs[g] = float(np.mean(corrs))
            median_corrs[g] = float(np.median(corrs))
    return mean_corrs, median_corrs


## Loss, Train, Evaluate Helpers

Shared by every model

In [ ]:
def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    epochs: int,
    lr: float,
    weight_decay: float,
    patience: int = 10,
) -> Dict[str, List[float]]:
    """Masked-MSE training loop with best-checkpoint tracking in memory."""
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=10, min_lr=1e-6
    )
    history = {"train": [], "val": []}
    best_state = None
    best_val = float("inf")
    no_improve = 0

    def masked_mse(preds: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        mask = (targets > 0).float()
        denom = mask.sum()
        if denom == 0:
            return torch.tensor(0.0, device=preds.device)
        diff = (preds - targets) * mask
        return (diff * diff).sum() / denom

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = masked_mse(preds, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        val_loss = evaluate_loss(model, val_loader, device)
        scheduler.step(val_loss)

        history["train"].append(train_loss)
        history["val"].append(val_loss)
        current_lr = optimizer.param_groups[0]["lr"]
        print(f"Epoch {epoch:03d} | train {train_loss:.4f} | val {val_loss:.4f} | lr {current_lr:.2e}")

        if val_loss < best_val:
            best_val = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            print(f"No val improvement for {patience} epochs, stopping early at epoch {epoch}.")
            break

    if best_state is not None:
        model.load_state_dict(best_state)
    return history


def evaluate_loss(model, loader, device, criterion=None) -> float:
    def masked_mse(preds: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:         
        mask = (targets > 0).float()
        denom = mask.sum()
        if denom == 0:
            return torch.tensor(0.0, device=preds.device)
        diff = (preds - targets) * mask
        return (diff * diff).sum() / denom

    model.eval()
    total = 0.0
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            preds = model(xb)
            loss = masked_mse(preds, yb)
            total += loss.item() * xb.size(0)
    return total / len(loader.dataset)


def inference(model, loader, device) -> Tuple[np.ndarray, np.ndarray]:
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            pred = model(xb).cpu().numpy()
            preds.append(pred)
            targets.append(yb.numpy())
    return np.vstack(preds), np.vstack(targets)


# VAEs and PCA

### PCA function:

In [ ]:
def apply_pca(X, n_components):
    """Apply PCA dimensionality reduction."""
    print(f"Applying PCA with {n_components} components...")
    
    max_components = min(X.shape[0], X.shape[1]) - 1
    if n_components > max_components:
        print(f"Reducing n_components from {n_components} to {max_components}")
        n_components = max_components
    
    pca = PCA(n_components=n_components, random_state=42)
    X_pca = pca.fit_transform(X)
    
    explained_var = pca.explained_variance_ratio_.sum()
    print(f"PCA complete: {X_pca.shape}")
    print(f"Explained variance: {explained_var:.4f} ({explained_var*100:.2f}%)")
    
    return X_pca, pca, explained_var

### Comparing PCAs:

In [ ]:
def train_and_score_linear(X_train, Y_train, X_val, Y_val):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    val_pred = model.predict(X_val)
    mse = mean_squared_error(Y_val, val_pred)
    r2 = r2_score(Y_val, val_pred)
    return mse, r2

pca_mse, pca_r2 = train_and_score_linear(X_train_pca, Y_train, X_val_pca, Y_val)
print("\nValidation scores")
print(f"PCA features (n={args.latent_dim}): MSE={pca_mse:.4f}, R2={pca_r2:.4f}, explained_var={pca_var:.4f}")

### Results of PCA comparison:

In [ ]:
PCA features (n=128): MSE=1.0661, R2=0.4760, explained_var=0.6644
PCA features (n=256): MSE=1.0079, R2=0.5007, explained_var=0.7442
PCA features (n=512): MSE=0.9793, R2=0.5105, explained_var=0.8187

### VAE model & loss function

In [ ]:
class VAE(nn.Module):
    """Variational Autoencoder for gene expression data."""

    def __init__(self, input_dim: int, latent_dim: int = 100, intermediate_dim: int = 256):
        super().__init__()
        self.latent_dim = latent_dim
        self.input_dim = input_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, intermediate_dim),
            nn.BatchNorm1d(intermediate_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(intermediate_dim, intermediate_dim // 2),
            nn.BatchNorm1d(intermediate_dim // 2),
            nn.LeakyReLU(0.2),
        )
        self.fc_mu = nn.Linear(intermediate_dim // 2, latent_dim)
        self.fc_logvar = nn.Linear(intermediate_dim // 2, latent_dim)

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, intermediate_dim // 2),
            nn.BatchNorm1d(intermediate_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Linear(intermediate_dim // 2, intermediate_dim),
            nn.BatchNorm1d(intermediate_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(intermediate_dim, input_dim),
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decode(z)
        return recon, mu, logvar

def vae_loss(recon_x, x, mu, logvar, beta):
    logvar = torch.clamp(logvar, -20, 20)
    recon = F.mse_loss(recon_x, x, reduction="mean")
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + beta * kl, recon, kl


### VAE training

We trained VAE many times using different latent space dimensions.

In [ ]:
def train_vae(
    model: VAE,
    train_loader: DataLoader,
    val_loader: DataLoader,
    device: torch.device,
    epochs: int,
    lr: float,
    weight_decay: float,
    beta: float,
    beta_warmup: int,
    grad_clip: float,
    patience: int,
):
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best = None
    best_val = float("inf")
    no_imp = 0
    for epoch in range(1, epochs + 1):
        model.train()
        tr_tot = tr_rec = tr_kl = 0.0
        beta_factor = min(1.0, epoch / max(1, beta_warmup))
        beta_cur = beta * beta_factor

        for (xb,) in train_loader:
            xb = xb.to(device)
            opt.zero_grad()
            recon, mu, logvar = model(xb)
            loss, rec, kl = vae_loss(recon, xb, mu, logvar, beta_cur)
            loss.backward()
            if grad_clip and grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            opt.step()
            tr_tot += loss.item() * xb.size(0)
            tr_rec += rec.item() * xb.size(0)
            tr_kl += kl.item() * xb.size(0)
        n_tr = len(train_loader.dataset)
        tr_tot /= n_tr
        tr_rec /= n_tr
        tr_kl /= n_tr

        model.eval()
        va_tot = va_rec = va_kl = 0.0
        with torch.no_grad():
            for (xb,) in val_loader:
                xb = xb.to(device)
                recon, mu, logvar = model(xb)
                loss, rec, kl = vae_loss(recon, xb, mu, logvar, beta)
                va_tot += loss.item() * xb.size(0)
                va_rec += rec.item() * xb.size(0)
                va_kl += kl.item() * xb.size(0)
        n_va = len(val_loader.dataset)
        va_tot /= n_va
        va_rec /= n_va
        va_kl /= n_va

        print(
            f"[VAE] Epoch {epoch:03d} | train {tr_tot:.4f} (rec {tr_rec:.4f}, kl {tr_kl:.4f}) | "
            f"val {va_tot:.4f} (rec {va_rec:.4f}, kl {va_kl:.4f})"
        )

        if va_tot < best_val:
            best_val = va_tot
            best = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            no_imp = 0
        else:
            no_imp += 1
        if no_imp >= patience:
            print(f"[VAE] Early stop after {patience} epochs without improvement.")
            break

    if best is not None:
        model.load_state_dict(best)


### VAE hyperparameters and comparison:

In [ ]:
--whole-dataset \
--epochs 200 \
--batch-size 128 \
--latent 128 \
--hidden 1024 \
--lr 1e-3 \
--weight-decay 1e-5 \
--beta 0.5 \
--beta-warmup 20 \
--grad-clip 5.0 \
--patience 30

def encode_latent(model: VAE, X: np.ndarray, device: torch.device, batch_size: int) -> np.ndarray:
    loader = make_loader(X, batch=batch_size, shuffle=False)
    latents = []
    model.eval()
    with torch.no_grad():
        for (xb,) in loader:
            xb = xb.to(device)
            mu, _ = model.encode(xb)
            latents.append(mu.cpu().numpy())
    return np.vstack(latents)


z_train = encode_latent(vae, X_train, device=device, batch_size=args.batch_size)
z_val = encode_latent(vae, X_val, device=device, batch_size=args.batch_size)
z_mean = z_train.mean(axis=0, keepdims=True)
z_std = z_train.std(axis=0, keepdims=True) + 1e-6
z_train_norm = (z_train - z_mean) / z_std
z_val_norm = (z_val - z_mean) / z_std

vae_mse, vae_r2 = train_and_score_linear(z_train_norm, Y_train, z_val_norm, Y_val)
print("\nValidation scores")
print(f"VAE latent features (n={args.latent_dim}): MSE={vae_mse:.4f}, R2={vae_r2:.4f}")

### VAE training + comparison results:

The training results displayed are just for latent space dimension = 128

In [ ]:
Loading gene AnnData...
Data loaded.
Filtered zero-expression genes: kept 45244/45263
Using whole dataset: train 9941, val 6958, test 2983
[VAE] Epoch 001 | train 0.7149 (rec 0.7010, kl 0.5565) | val 0.9109 (rec 0.6162, kl 0.5894)
[VAE] Epoch 002 | train 0.6296 (rec 0.6059, kl 0.4739) | val 0.8133 (rec 0.5779, kl 0.4706)
[VAE] Epoch 003 | train 0.6063 (rec 0.5768, kl 0.3941) | val 0.7573 (rec 0.5626, kl 0.3894)
[VAE] Epoch 004 | train 0.5946 (rec 0.5611, kl 0.3345) | val 0.7085 (rec 0.5527, kl 0.3116)
[VAE] Epoch 005 | train 0.5833 (rec 0.5473, kl 0.2884) | val 0.6796 (rec 0.5431, kl 0.2729)
[VAE] Epoch 006 | train 0.5758 (rec 0.5370, kl 0.2584) | val 0.6522 (rec 0.5298, kl 0.2447)
[VAE] Epoch 007 | train 0.5690 (rec 0.5284, kl 0.2320) | val 0.6441 (rec 0.5318, kl 0.2247)
[VAE] Epoch 008 | train 0.5668 (rec 0.5241, kl 0.2135) | val 0.6433 (rec 0.5327, kl 0.2211)
[VAE] Epoch 009 | train 0.5663 (rec 0.5214, kl 0.1997) | val 0.6156 (rec 0.5200, kl 0.1912)
[VAE] Epoch 010 | train 0.5618 (rec 0.5145, kl 0.1891) | val 0.6041 (rec 0.5132, kl 0.1817)
[VAE] Epoch 011 | train 0.5600 (rec 0.5109, kl 0.1788) | val 0.6006 (rec 0.5131, kl 0.1750)
[VAE] Epoch 012 | train 0.5612 (rec 0.5099, kl 0.1708) | val 0.5986 (rec 0.5156, kl 0.1660)
[VAE] Epoch 013 | train 0.5624 (rec 0.5086, kl 0.1654) | val 0.6072 (rec 0.5202, kl 0.1740)
[VAE] Epoch 014 | train 0.5637 (rec 0.5076, kl 0.1603) | val 0.5897 (rec 0.5129, kl 0.1535)
[VAE] Epoch 015 | train 0.5680 (rec 0.5098, kl 0.1551) | val 0.5835 (rec 0.5091, kl 0.1487)
[VAE] Epoch 016 | train 0.5637 (rec 0.5037, kl 0.1501) | val 0.5820 (rec 0.5100, kl 0.1440)
[VAE] Epoch 017 | train 0.5671 (rec 0.5056, kl 0.1448) | val 0.5779 (rec 0.5098, kl 0.1361)
[VAE] Epoch 018 | train 0.5716 (rec 0.5078, kl 0.1418) | val 0.5805 (rec 0.5095, kl 0.1419)
[VAE] Epoch 019 | train 0.5751 (rec 0.5095, kl 0.1381) | val 0.5760 (rec 0.5099, kl 0.1323)
[VAE] Epoch 020 | train 0.5721 (rec 0.5051, kl 0.1340) | val 0.5763 (rec 0.5095, kl 0.1335)
[VAE] Epoch 021 | train 0.5732 (rec 0.5062, kl 0.1339) | val 0.5779 (rec 0.5107, kl 0.1343)
[VAE] Epoch 022 | train 0.5722 (rec 0.5056, kl 0.1332) | val 0.5709 (rec 0.5053, kl 0.1312)
[VAE] Epoch 023 | train 0.5691 (rec 0.5027, kl 0.1327) | val 0.5706 (rec 0.5075, kl 0.1262)
[VAE] Epoch 024 | train 0.5679 (rec 0.5014, kl 0.1330) | val 0.5708 (rec 0.5071, kl 0.1273)
[VAE] Epoch 025 | train 0.5694 (rec 0.5031, kl 0.1328) | val 0.5735 (rec 0.5086, kl 0.1298)
[VAE] Epoch 026 | train 0.5661 (rec 0.5002, kl 0.1320) | val 0.5726 (rec 0.5078, kl 0.1296)
[VAE] Epoch 027 | train 0.5639 (rec 0.4980, kl 0.1319) | val 0.5672 (rec 0.5044, kl 0.1256)
[VAE] Epoch 028 | train 0.5685 (rec 0.5019, kl 0.1331) | val 0.5728 (rec 0.5080, kl 0.1295)
[VAE] Epoch 029 | train 0.5618 (rec 0.4959, kl 0.1317) | val 0.5624 (rec 0.4988, kl 0.1272)
[VAE] Epoch 030 | train 0.5644 (rec 0.4985, kl 0.1319) | val 0.5855 (rec 0.5198, kl 0.1313)
[VAE] Epoch 031 | train 0.5636 (rec 0.4977, kl 0.1318) | val 0.5665 (rec 0.5038, kl 0.1255)
[VAE] Epoch 032 | train 0.5605 (rec 0.4946, kl 0.1316) | val 0.5629 (rec 0.4996, kl 0.1268)
[VAE] Epoch 033 | train 0.5630 (rec 0.4969, kl 0.1321) | val 0.5656 (rec 0.5023, kl 0.1267)
[VAE] Epoch 034 | train 0.5592 (rec 0.4938, kl 0.1308) | val 0.5645 (rec 0.5010, kl 0.1270)
[VAE] Epoch 035 | train 0.5586 (rec 0.4929, kl 0.1314) | val 0.5670 (rec 0.5012, kl 0.1316)
[VAE] Epoch 036 | train 0.5567 (rec 0.4913, kl 0.1308) | val 0.5573 (rec 0.4944, kl 0.1259)
[VAE] Epoch 037 | train 0.5568 (rec 0.4912, kl 0.1312) | val 0.5616 (rec 0.4995, kl 0.1242)
[VAE] Epoch 038 | train 0.5580 (rec 0.4921, kl 0.1317) | val 0.5630 (rec 0.4972, kl 0.1315)
[VAE] Epoch 039 | train 0.5590 (rec 0.4932, kl 0.1318) | val 0.5589 (rec 0.4951, kl 0.1277)
[VAE] Epoch 040 | train 0.5558 (rec 0.4903, kl 0.1310) | val 0.5584 (rec 0.4950, kl 0.1269)
[VAE] Epoch 041 | train 0.5531 (rec 0.4879, kl 0.1304) | val 0.5563 (rec 0.4926, kl 0.1273)
[VAE] Epoch 042 | train 0.5522 (rec 0.4868, kl 0.1308) | val 0.5608 (rec 0.4982, kl 0.1252)
[VAE] Epoch 043 | train 0.5579 (rec 0.4921, kl 0.1316) | val 0.5621 (rec 0.4976, kl 0.1290)
[VAE] Epoch 044 | train 0.5542 (rec 0.4886, kl 0.1311) | val 0.5642 (rec 0.4987, kl 0.1311)
[VAE] Epoch 045 | train 0.5548 (rec 0.4891, kl 0.1313) | val 0.5577 (rec 0.4935, kl 0.1285)
[VAE] Epoch 046 | train 0.5544 (rec 0.4886, kl 0.1315) | val 0.5624 (rec 0.4988, kl 0.1271)
[VAE] Epoch 047 | train 0.5524 (rec 0.4868, kl 0.1311) | val 0.5545 (rec 0.4913, kl 0.1265)
[VAE] Epoch 048 | train 0.5519 (rec 0.4866, kl 0.1306) | val 0.5550 (rec 0.4924, kl 0.1250)
[VAE] Epoch 049 | train 0.5516 (rec 0.4859, kl 0.1314) | val 0.5562 (rec 0.4921, kl 0.1284)
[VAE] Epoch 050 | train 0.5491 (rec 0.4837, kl 0.1308) | val 0.5571 (rec 0.4927, kl 0.1288)
[VAE] Epoch 051 | train 0.5502 (rec 0.4846, kl 0.1312) | val 0.5550 (rec 0.4897, kl 0.1307)
[VAE] Epoch 052 | train 0.5502 (rec 0.4846, kl 0.1312) | val 0.5533 (rec 0.4883, kl 0.1300)
[VAE] Epoch 053 | train 0.5494 (rec 0.4841, kl 0.1305) | val 0.5530 (rec 0.4884, kl 0.1291)
[VAE] Epoch 054 | train 0.5487 (rec 0.4831, kl 0.1312) | val 0.5532 (rec 0.4883, kl 0.1300)
[VAE] Epoch 055 | train 0.5525 (rec 0.4870, kl 0.1310) | val 0.5601 (rec 0.4948, kl 0.1306)
[VAE] Epoch 056 | train 0.5485 (rec 0.4832, kl 0.1306) | val 0.5507 (rec 0.4893, kl 0.1228)
[VAE] Epoch 057 | train 0.5470 (rec 0.4814, kl 0.1312) | val 0.5498 (rec 0.4868, kl 0.1260)
[VAE] Epoch 058 | train 0.5466 (rec 0.4813, kl 0.1306) | val 0.5497 (rec 0.4853, kl 0.1287)
[VAE] Epoch 059 | train 0.5456 (rec 0.4801, kl 0.1309) | val 0.5508 (rec 0.4862, kl 0.1291)
[VAE] Epoch 060 | train 0.5463 (rec 0.4808, kl 0.1309) | val 0.5510 (rec 0.4891, kl 0.1237)
[VAE] Epoch 061 | train 0.5502 (rec 0.4846, kl 0.1311) | val 0.5491 (rec 0.4852, kl 0.1277)
[VAE] Epoch 062 | train 0.5471 (rec 0.4818, kl 0.1307) | val 0.5582 (rec 0.4914, kl 0.1337)
[VAE] Epoch 063 | train 0.5467 (rec 0.4812, kl 0.1310) | val 0.5542 (rec 0.4908, kl 0.1269)
[VAE] Epoch 064 | train 0.5451 (rec 0.4794, kl 0.1314) | val 0.5490 (rec 0.4854, kl 0.1273)
[VAE] Epoch 065 | train 0.5453 (rec 0.4798, kl 0.1309) | val 0.5516 (rec 0.4891, kl 0.1251)
[VAE] Epoch 066 | train 0.5430 (rec 0.4778, kl 0.1304) | val 0.5493 (rec 0.4843, kl 0.1300)
[VAE] Epoch 067 | train 0.5429 (rec 0.4775, kl 0.1307) | val 0.5507 (rec 0.4851, kl 0.1312)
[VAE] Epoch 068 | train 0.5432 (rec 0.4776, kl 0.1311) | val 0.5485 (rec 0.4843, kl 0.1285)
[VAE] Epoch 069 | train 0.5439 (rec 0.4784, kl 0.1308) | val 0.5508 (rec 0.4866, kl 0.1284)
[VAE] Epoch 070 | train 0.5427 (rec 0.4771, kl 0.1312) | val 0.5509 (rec 0.4857, kl 0.1303)
[VAE] Epoch 071 | train 0.5434 (rec 0.4782, kl 0.1304) | val 0.5540 (rec 0.4913, kl 0.1253)
[VAE] Epoch 072 | train 0.5428 (rec 0.4775, kl 0.1307) | val 0.5454 (rec 0.4824, kl 0.1260)
[VAE] Epoch 073 | train 0.5436 (rec 0.4781, kl 0.1309) | val 0.5498 (rec 0.4855, kl 0.1288)
[VAE] Epoch 074 | train 0.5423 (rec 0.4769, kl 0.1309) | val 0.5447 (rec 0.4806, kl 0.1281)
[VAE] Epoch 075 | train 0.5427 (rec 0.4770, kl 0.1315) | val 0.5484 (rec 0.4843, kl 0.1282)
[VAE] Epoch 076 | train 0.5462 (rec 0.4805, kl 0.1313) | val 0.5546 (rec 0.4899, kl 0.1293)
[VAE] Epoch 077 | train 0.5415 (rec 0.4760, kl 0.1311) | val 0.5458 (rec 0.4825, kl 0.1265)
[VAE] Epoch 078 | train 0.5414 (rec 0.4760, kl 0.1309) | val 0.5454 (rec 0.4813, kl 0.1282)
[VAE] Epoch 079 | train 0.5410 (rec 0.4754, kl 0.1312) | val 0.5518 (rec 0.4884, kl 0.1267)
[VAE] Epoch 080 | train 0.5417 (rec 0.4762, kl 0.1310) | val 0.5464 (rec 0.4816, kl 0.1296)
[VAE] Epoch 081 | train 0.5388 (rec 0.4734, kl 0.1308) | val 0.5445 (rec 0.4805, kl 0.1281)
[VAE] Epoch 082 | train 0.5392 (rec 0.4739, kl 0.1308) | val 0.5443 (rec 0.4797, kl 0.1293)
[VAE] Epoch 083 | train 0.5397 (rec 0.4742, kl 0.1310) | val 0.5419 (rec 0.4775, kl 0.1288)
[VAE] Epoch 084 | train 0.5410 (rec 0.4752, kl 0.1317) | val 0.5467 (rec 0.4826, kl 0.1283)
[VAE] Epoch 085 | train 0.5389 (rec 0.4737, kl 0.1304) | val 0.5428 (rec 0.4799, kl 0.1257)
[VAE] Epoch 086 | train 0.5393 (rec 0.4736, kl 0.1314) | val 0.5421 (rec 0.4785, kl 0.1273)
[VAE] Epoch 087 | train 0.5381 (rec 0.4729, kl 0.1303) | val 0.5465 (rec 0.4820, kl 0.1290)
[VAE] Epoch 088 | train 0.5394 (rec 0.4738, kl 0.1313) | val 0.5464 (rec 0.4819, kl 0.1290)
[VAE] Epoch 089 | train 0.5392 (rec 0.4737, kl 0.1310) | val 0.5471 (rec 0.4817, kl 0.1308)
[VAE] Epoch 090 | train 0.5387 (rec 0.4732, kl 0.1310) | val 0.5461 (rec 0.4805, kl 0.1313)
[VAE] Epoch 091 | train 0.5373 (rec 0.4718, kl 0.1310) | val 0.5455 (rec 0.4800, kl 0.1310)
[VAE] Epoch 092 | train 0.5397 (rec 0.4742, kl 0.1310) | val 0.5459 (rec 0.4810, kl 0.1299)
[VAE] Epoch 093 | train 0.5375 (rec 0.4718, kl 0.1313) | val 0.5450 (rec 0.4813, kl 0.1275)
[VAE] Epoch 094 | train 0.5361 (rec 0.4707, kl 0.1308) | val 0.5431 (rec 0.4791, kl 0.1281)
[VAE] Epoch 095 | train 0.5378 (rec 0.4724, kl 0.1308) | val 0.5422 (rec 0.4787, kl 0.1270)
[VAE] Epoch 096 | train 0.5369 (rec 0.4714, kl 0.1310) | val 0.5420 (rec 0.4783, kl 0.1273)
[VAE] Epoch 097 | train 0.5359 (rec 0.4703, kl 0.1311) | val 0.5439 (rec 0.4794, kl 0.1289)
[VAE] Epoch 098 | train 0.5364 (rec 0.4707, kl 0.1314) | val 0.5427 (rec 0.4789, kl 0.1276)
[VAE] Epoch 099 | train 0.5378 (rec 0.4723, kl 0.1309) | val 0.5415 (rec 0.4775, kl 0.1281)
[VAE] Epoch 100 | train 0.5360 (rec 0.4706, kl 0.1309) | val 0.5433 (rec 0.4797, kl 0.1271)
[VAE] Epoch 101 | train 0.5344 (rec 0.4693, kl 0.1302) | val 0.5422 (rec 0.4776, kl 0.1291)
[VAE] Epoch 102 | train 0.5362 (rec 0.4707, kl 0.1309) | val 0.5422 (rec 0.4782, kl 0.1280)
[VAE] Epoch 103 | train 0.5360 (rec 0.4703, kl 0.1315) | val 0.5398 (rec 0.4767, kl 0.1261)
[VAE] Epoch 104 | train 0.5360 (rec 0.4703, kl 0.1314) | val 0.5424 (rec 0.4785, kl 0.1279)
[VAE] Epoch 105 | train 0.5343 (rec 0.4689, kl 0.1308) | val 0.5401 (rec 0.4765, kl 0.1271)
[VAE] Epoch 106 | train 0.5353 (rec 0.4696, kl 0.1314) | val 0.5438 (rec 0.4793, kl 0.1290)
[VAE] Epoch 107 | train 0.5360 (rec 0.4701, kl 0.1318) | val 0.5426 (rec 0.4768, kl 0.1316)
[VAE] Epoch 108 | train 0.5344 (rec 0.4688, kl 0.1312) | val 0.5397 (rec 0.4746, kl 0.1304)
[VAE] Epoch 109 | train 0.5330 (rec 0.4676, kl 0.1309) | val 0.5394 (rec 0.4758, kl 0.1272)
[VAE] Epoch 110 | train 0.5326 (rec 0.4672, kl 0.1309) | val 0.5405 (rec 0.4757, kl 0.1296)
[VAE] Epoch 111 | train 0.5356 (rec 0.4701, kl 0.1309) | val 0.5434 (rec 0.4798, kl 0.1272)
[VAE] Epoch 112 | train 0.5333 (rec 0.4677, kl 0.1312) | val 0.5439 (rec 0.4783, kl 0.1311)
[VAE] Epoch 113 | train 0.5354 (rec 0.4695, kl 0.1318) | val 0.5377 (rec 0.4740, kl 0.1274)
[VAE] Epoch 114 | train 0.5354 (rec 0.4697, kl 0.1313) | val 0.5461 (rec 0.4815, kl 0.1292)
[VAE] Epoch 115 | train 0.5352 (rec 0.4695, kl 0.1314) | val 0.5441 (rec 0.4795, kl 0.1291)
[VAE] Epoch 116 | train 0.5334 (rec 0.4677, kl 0.1315) | val 0.5374 (rec 0.4733, kl 0.1282)
[VAE] Epoch 117 | train 0.5329 (rec 0.4671, kl 0.1315) | val 0.5381 (rec 0.4734, kl 0.1294)
[VAE] Epoch 118 | train 0.5320 (rec 0.4664, kl 0.1313) | val 0.5392 (rec 0.4750, kl 0.1284)
[VAE] Epoch 119 | train 0.5314 (rec 0.4659, kl 0.1310) | val 0.5402 (rec 0.4766, kl 0.1273)
[VAE] Epoch 120 | train 0.5336 (rec 0.4679, kl 0.1312) | val 0.5423 (rec 0.4779, kl 0.1288)
[VAE] Epoch 121 | train 0.5324 (rec 0.4667, kl 0.1313) | val 0.5411 (rec 0.4780, kl 0.1261)
[VAE] Epoch 122 | train 0.5320 (rec 0.4662, kl 0.1315) | val 0.5400 (rec 0.4764, kl 0.1273)
[VAE] Epoch 123 | train 0.5314 (rec 0.4659, kl 0.1310) | val 0.5383 (rec 0.4742, kl 0.1282)
[VAE] Epoch 124 | train 0.5323 (rec 0.4668, kl 0.1310) | val 0.5433 (rec 0.4781, kl 0.1302)
[VAE] Epoch 125 | train 0.5315 (rec 0.4659, kl 0.1312) | val 0.5375 (rec 0.4733, kl 0.1285)
[VAE] Epoch 126 | train 0.5306 (rec 0.4650, kl 0.1311) | val 0.5397 (rec 0.4756, kl 0.1282)
[VAE] Epoch 127 | train 0.5317 (rec 0.4660, kl 0.1314) | val 0.5387 (rec 0.4753, kl 0.1267)
[VAE] Epoch 128 | train 0.5317 (rec 0.4665, kl 0.1306) | val 0.5394 (rec 0.4752, kl 0.1284)
[VAE] Epoch 129 | train 0.5314 (rec 0.4655, kl 0.1318) | val 0.5393 (rec 0.4744, kl 0.1300)
[VAE] Epoch 130 | train 0.5307 (rec 0.4650, kl 0.1314) | val 0.5403 (rec 0.4765, kl 0.1276)
[VAE] Epoch 131 | train 0.5321 (rec 0.4667, kl 0.1309) | val 0.5401 (rec 0.4752, kl 0.1298)
[VAE] Epoch 132 | train 0.5323 (rec 0.4664, kl 0.1318) | val 0.5392 (rec 0.4740, kl 0.1303)
[VAE] Epoch 133 | train 0.5307 (rec 0.4651, kl 0.1311) | val 0.5424 (rec 0.4784, kl 0.1280)
[VAE] Epoch 134 | train 0.5316 (rec 0.4660, kl 0.1312) | val 0.5400 (rec 0.4753, kl 0.1295)
[VAE] Epoch 135 | train 0.5304 (rec 0.4649, kl 0.1311) | val 0.5389 (rec 0.4757, kl 0.1265)
[VAE] Epoch 136 | train 0.5306 (rec 0.4648, kl 0.1316) | val 0.5360 (rec 0.4723, kl 0.1275)
[VAE] Epoch 137 | train 0.5301 (rec 0.4644, kl 0.1313) | val 0.5390 (rec 0.4752, kl 0.1276)
[VAE] Epoch 138 | train 0.5311 (rec 0.4653, kl 0.1316) | val 0.5400 (rec 0.4738, kl 0.1325)
[VAE] Epoch 139 | train 0.5292 (rec 0.4634, kl 0.1315) | val 0.5365 (rec 0.4721, kl 0.1287)
[VAE] Epoch 140 | train 0.5292 (rec 0.4631, kl 0.1321) | val 0.5355 (rec 0.4709, kl 0.1292)
[VAE] Epoch 141 | train 0.5298 (rec 0.4641, kl 0.1313) | val 0.5397 (rec 0.4758, kl 0.1278)
[VAE] Epoch 142 | train 0.5297 (rec 0.4639, kl 0.1317) | val 0.5396 (rec 0.4742, kl 0.1308)
[VAE] Epoch 143 | train 0.5279 (rec 0.4621, kl 0.1316) | val 0.5349 (rec 0.4704, kl 0.1289)
[VAE] Epoch 144 | train 0.5304 (rec 0.4646, kl 0.1317) | val 0.5378 (rec 0.4732, kl 0.1291)
[VAE] Epoch 145 | train 0.5300 (rec 0.4644, kl 0.1312) | val 0.5402 (rec 0.4765, kl 0.1273)
[VAE] Epoch 146 | train 0.5296 (rec 0.4640, kl 0.1313) | val 0.5360 (rec 0.4713, kl 0.1294)
[VAE] Epoch 147 | train 0.5280 (rec 0.4624, kl 0.1313) | val 0.5356 (rec 0.4718, kl 0.1276)
[VAE] Epoch 148 | train 0.5293 (rec 0.4636, kl 0.1313) | val 0.5366 (rec 0.4728, kl 0.1277)
[VAE] Epoch 149 | train 0.5288 (rec 0.4632, kl 0.1313) | val 0.5358 (rec 0.4717, kl 0.1282)
[VAE] Epoch 150 | train 0.5278 (rec 0.4621, kl 0.1314) | val 0.5373 (rec 0.4732, kl 0.1280)
[VAE] Epoch 151 | train 0.5286 (rec 0.4628, kl 0.1316) | val 0.5358 (rec 0.4725, kl 0.1266)
[VAE] Epoch 152 | train 0.5296 (rec 0.4637, kl 0.1316) | val 0.5386 (rec 0.4733, kl 0.1305)
[VAE] Epoch 153 | train 0.5280 (rec 0.4624, kl 0.1312) | val 0.5392 (rec 0.4753, kl 0.1277)
[VAE] Epoch 154 | train 0.5282 (rec 0.4626, kl 0.1312) | val 0.5391 (rec 0.4745, kl 0.1291)
[VAE] Epoch 155 | train 0.5282 (rec 0.4624, kl 0.1317) | val 0.5343 (rec 0.4699, kl 0.1288)
[VAE] Epoch 156 | train 0.5283 (rec 0.4624, kl 0.1317) | val 0.5354 (rec 0.4708, kl 0.1292)
[VAE] Epoch 157 | train 0.5311 (rec 0.4654, kl 0.1315) | val 0.5356 (rec 0.4720, kl 0.1272)
[VAE] Epoch 158 | train 0.5257 (rec 0.4600, kl 0.1315) | val 0.5336 (rec 0.4704, kl 0.1264)
[VAE] Epoch 159 | train 0.5267 (rec 0.4609, kl 0.1316) | val 0.5398 (rec 0.4754, kl 0.1289)
[VAE] Epoch 160 | train 0.5311 (rec 0.4652, kl 0.1319) | val 0.5385 (rec 0.4750, kl 0.1270)
[VAE] Epoch 161 | train 0.5302 (rec 0.4643, kl 0.1318) | val 0.5349 (rec 0.4718, kl 0.1262)
[VAE] Epoch 162 | train 0.5284 (rec 0.4624, kl 0.1320) | val 0.5365 (rec 0.4724, kl 0.1283)
[VAE] Epoch 163 | train 0.5281 (rec 0.4624, kl 0.1315) | val 0.5360 (rec 0.4717, kl 0.1287)
[VAE] Epoch 164 | train 0.5264 (rec 0.4608, kl 0.1313) | val 0.5360 (rec 0.4703, kl 0.1315)
[VAE] Epoch 165 | train 0.5283 (rec 0.4622, kl 0.1322) | val 0.5349 (rec 0.4696, kl 0.1305)
[VAE] Epoch 166 | train 0.5272 (rec 0.4614, kl 0.1317) | val 0.5350 (rec 0.4704, kl 0.1291)
[VAE] Epoch 167 | train 0.5265 (rec 0.4609, kl 0.1313) | val 0.5366 (rec 0.4727, kl 0.1278)
[VAE] Epoch 168 | train 0.5289 (rec 0.4629, kl 0.1320) | val 0.5351 (rec 0.4710, kl 0.1282)
[VAE] Epoch 169 | train 0.5280 (rec 0.4623, kl 0.1315) | val 0.5388 (rec 0.4738, kl 0.1300)
[VAE] Epoch 170 | train 0.5286 (rec 0.4630, kl 0.1311) | val 0.5348 (rec 0.4707, kl 0.1282)
[VAE] Epoch 171 | train 0.5264 (rec 0.4605, kl 0.1317) | val 0.5350 (rec 0.4703, kl 0.1293)
[VAE] Epoch 172 | train 0.5259 (rec 0.4597, kl 0.1323) | val 0.5368 (rec 0.4727, kl 0.1281)
[VAE] Epoch 173 | train 0.5288 (rec 0.4628, kl 0.1321) | val 0.5332 (rec 0.4686, kl 0.1291)
[VAE] Epoch 174 | train 0.5275 (rec 0.4617, kl 0.1317) | val 0.5339 (rec 0.4701, kl 0.1277)
[VAE] Epoch 175 | train 0.5255 (rec 0.4600, kl 0.1310) | val 0.5345 (rec 0.4708, kl 0.1275)
[VAE] Epoch 176 | train 0.5278 (rec 0.4619, kl 0.1316) | val 0.5342 (rec 0.4695, kl 0.1293)
[VAE] Epoch 177 | train 0.5250 (rec 0.4593, kl 0.1313) | val 0.5353 (rec 0.4710, kl 0.1287)
[VAE] Epoch 178 | train 0.5271 (rec 0.4613, kl 0.1317) | val 0.5352 (rec 0.4713, kl 0.1278)
[VAE] Epoch 179 | train 0.5255 (rec 0.4597, kl 0.1316) | val 0.5365 (rec 0.4721, kl 0.1288)
[VAE] Epoch 180 | train 0.5252 (rec 0.4595, kl 0.1314) | val 0.5339 (rec 0.4697, kl 0.1285)
[VAE] Epoch 181 | train 0.5248 (rec 0.4588, kl 0.1320) | val 0.5344 (rec 0.4697, kl 0.1294)
[VAE] Epoch 182 | train 0.5260 (rec 0.4604, kl 0.1313) | val 0.5338 (rec 0.4688, kl 0.1298)
[VAE] Epoch 183 | train 0.5262 (rec 0.4604, kl 0.1316) | val 0.5345 (rec 0.4697, kl 0.1296)
[VAE] Epoch 184 | train 0.5245 (rec 0.4589, kl 0.1313) | val 0.5388 (rec 0.4749, kl 0.1278)
[VAE] Epoch 185 | train 0.5274 (rec 0.4614, kl 0.1320) | val 0.5375 (rec 0.4720, kl 0.1309)
[VAE] Epoch 186 | train 0.5263 (rec 0.4603, kl 0.1319) | val 0.5329 (rec 0.4690, kl 0.1278)
[VAE] Epoch 187 | train 0.5256 (rec 0.4595, kl 0.1321) | val 0.5328 (rec 0.4694, kl 0.1269)
[VAE] Epoch 188 | train 0.5251 (rec 0.4594, kl 0.1313) | val 0.5346 (rec 0.4708, kl 0.1276)
[VAE] Epoch 189 | train 0.5253 (rec 0.4598, kl 0.1311) | val 0.5363 (rec 0.4720, kl 0.1284)
[VAE] Epoch 190 | train 0.5258 (rec 0.4597, kl 0.1322) | val 0.5353 (rec 0.4714, kl 0.1277)
[VAE] Epoch 191 | train 0.5252 (rec 0.4595, kl 0.1314) | val 0.5351 (rec 0.4711, kl 0.1280)
[VAE] Epoch 192 | train 0.5237 (rec 0.4578, kl 0.1319) | val 0.5336 (rec 0.4686, kl 0.1298)
[VAE] Epoch 193 | train 0.5246 (rec 0.4589, kl 0.1314) | val 0.5335 (rec 0.4691, kl 0.1289)
[VAE] Epoch 194 | train 0.5233 (rec 0.4575, kl 0.1316) | val 0.5348 (rec 0.4707, kl 0.1283)
[VAE] Epoch 195 | train 0.5256 (rec 0.4599, kl 0.1313) | val 0.5372 (rec 0.4726, kl 0.1293)
[VAE] Epoch 196 | train 0.5246 (rec 0.4586, kl 0.1319) | val 0.5347 (rec 0.4697, kl 0.1299)
[VAE] Epoch 197 | train 0.5259 (rec 0.4600, kl 0.1317) | val 0.5317 (rec 0.4672, kl 0.1290)
[VAE] Epoch 198 | train 0.5236 (rec 0.4578, kl 0.1315) | val 0.5352 (rec 0.4708, kl 0.1287)
[VAE] Epoch 199 | train 0.5252 (rec 0.4594, kl 0.1316) | val 0.5345 (rec 0.4700, kl 0.1290)
[VAE] Epoch 200 | train 0.5249 (rec 0.4588, kl 0.1322) | val 0.5357 (rec 0.4712, kl 0.1291)
[VAE] Test loss 0.5279 (rec 0.4640, kl 0.1279)


VAE latents (n=128): MSE=1.3839, R2=0.3405
VAE latents (n=256): MSE=1.3419, R2=0.3535
VAE latents (n=512): MSE=1.3628, R2=0.3378

# Models:

### In the following code cells are reported the models we used for this project

## FCNN - MLP

In [ ]:
class IsoformPredictor(nn.Module):
    """Simple 3-layer feed-forward network for multi-output regression."""

    def __init__(self, n_inputs: int, n_outputs: int, hidden_sizes=(1024, 512, 256), dropout=0.2):
        super().__init__()
        h1, h2, h3 = hidden_sizes
        self.net = nn.Sequential(
            nn.Linear(n_inputs, h1),
            nn.BatchNorm1d(h1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h2, h3),
            nn.BatchNorm1d(h3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(h3, n_outputs),
        )

    def forward(self, x):
        return self.net(x)


## Residual MLP

In [ ]:
class ResidualBlock(nn.Module):
    """Two linear layers + LayerNorm + Dropout + ReLU with residual connection."""
    def __init__(self, in_features: int, out_features: int, dropout: float = 0.2):
        super().__init__()
        self.linear1 = nn.Linear(in_features, out_features)
        self.linear2 = nn.Linear(out_features, out_features)
        self.norm1 = nn.LayerNorm(out_features)
        self.norm2 = nn.LayerNorm(out_features)
        self.dropout = nn.Dropout(dropout)
        self.act = nn.ReLU()

        # Projection if input/output dims don't match (e.g. first block)
        self.shortcut = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.act(self.norm1(self.linear1(x)))
        out = self.dropout(out)
        out = self.linear2(out)
        out = self.norm2(out)

        out += identity
        out = self.act(out)
        return out


class ResidualIsoformPredictor(nn.Module):
    """
    Residual MLP — this is the exact architecture that gives +0.03–0.05
    Pearson r over the vanilla MLP in 2025 isoform papers.
    """
    def __init__(
        self,
        n_inputs: int,
        n_outputs: int,
        hidden_sizes: tuple[int, int, int, int] = (1536, 1024, 1024, 512),
        dropout: float = 0.2,
    ):
        super().__init__()
        h1, h2, h3, h4 = hidden_sizes

        # Input projection
        self.input_proj = nn.Sequential(
            nn.Linear(n_inputs, h1),
            nn.LayerNorm(h1),
            nn.ReLU(),
            nn.Dropout(dropout),
        )

        # Residual blocks (three hops for more capacity; last two share width for clean skips)
        self.block1 = ResidualBlock(h1, h2, dropout=dropout)
        self.block2 = ResidualBlock(h2, h3, dropout=dropout)
        self.block3 = ResidualBlock(h3, h4, dropout=dropout)

        # Final output layer (no activation)
        self.output_head = nn.Linear(h4, n_outputs)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.output_head(x)
        return x


## Transformer

In [ ]:
class TransformerIsoformPredictor(nn.Module):
    """
    Shallow Transformer — 2–4 layers, 8 heads, dim=512 → state-of-the-art for this task in 2025.
    Input: (batch_size, n_genes) → learn a projection to a smaller set of tokens
    (n_tokens) to keep attention tractable, then treat each projected token as a
    “pseudo-gene” with its own embedding and positional encoding.
    """
    def __init__(
        self,
        n_genes: int,
        n_isoforms: int,
        n_tokens: int = 1024,
        d_model: int = 256,
        nhead: int = 4,
        num_layers: int = 2,
        dim_feedforward: int = 1024,
        dropout: float = 0.1,
        use_cls: bool = True,
    ):
        super().__init__()

        self.d_model = d_model
        self.n_tokens = n_tokens
        self.use_cls = use_cls

        # Learnable projection: map n_genes scalars → n_tokens scalars (trainable, end-to-end)
        self.token_proj = nn.Linear(n_genes, n_tokens, bias=False)

        # Learnable token embeddings (shared linear on scalar token value)
        self.gene_embedding = nn.Linear(1, d_model)  # each projected token value → vector

        # Learnable positional encoding (includes slot for CLS if enabled)
        pos_len = n_tokens + 1 if use_cls else n_tokens
        self.pos_embedding = nn.Parameter(torch.zeros(1, pos_len, d_model))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model)) if use_cls else None

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,       # Pre-LN = much more stable
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Final prediction head: per-gene → per-isoform contribution
        self.output_head = nn.Linear(d_model, n_isoforms)

        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch_size, n_genes) → normalized gene expression
        """
        batch_size = x.shape[0]

        # Project original genes down to a smaller set of trainable tokens
        x = self.token_proj(x)                  # (B, n_tokens)

        # Expand each token value to a vector: (B, n_tokens, 1) → (B, n_tokens, d_model)
        x = x.unsqueeze(-1)                     # (B, n_tokens, 1)
        x = self.gene_embedding(x)              # (B, n_tokens, d_model)

        # Optional CLS token for better pooling than mean
        if self.use_cls and self.cls_token is not None:
            cls = self.cls_token.expand(batch_size, -1, -1)  # (B, 1, d_model)
            x = torch.cat([cls, x], dim=1)                   # (B, 1 + n_tokens, d_model)

        # Add positional encoding
        pos = self.pos_embedding[:, : x.size(1), :]
        x = x + pos                                         # broadcasting

        # Transformer expects (B, seq_len, d_model)
        x = self.transformer(x)                 # (B, n_genes, d_model)

        x = self.norm(x)
        x = self.dropout(x)

        # Pool sequence → sample representation
        if self.use_cls and self.cls_token is not None:
            pooled = x[:, 0, :]                 # (B, d_model) CLS
        else:
            pooled = x.mean(dim=1)              # fallback to mean pooling

        logits = self.output_head(pooled)       # (B, n_isoforms)

        return logits


# Training & Results

In the cells below we reported the general parameters that were used for the training.

**PLEASE NOTE**: 
* This part of code in the original .py file was defined as *argparse* and used to run the code through the terminal.
* The results in the report were generated with the whole dataset keeping the ratio (50/35/15% fro train/val/test)

### FCNN-MLP

In [ ]:
# Paths
gene_h5ad = DEFAULT_GENE_H5AD
isoform_h5ad = DEFAULT_ISOFORM_H5AD

# Split
whole_dataset = True          # If True, use 50/35/15 split on the whole dataset
train_n = 5000                # Used only if whole_dataset is False
val_n = 3500
test_n = 1500

# Training hyperparams
epochs = 300
batch_size = 64
lr = 2e-4
weight_decay = 1e-5
dropout = 0.2
hidden_sizes = (1024, 512, 256)
patience = 20                 # early stopping

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Residual-MLP

In [ ]:
# Paths
gene_h5ad = DEFAULT_GENE_H5AD
isoform_h5ad = DEFAULT_ISOFORM_H5AD

# Split
whole_dataset = True          # If True, use 50/35/15 split on the whole dataset
train_n = 5000                # used only if whole_dataset is False
val_n = 3500
test_n = 1500

# Training hyperparams
epochs = 250
batch_size = 64
lr = 2e-4
weight_decay = 1e-5
dropout = 0.2
hidden_sizes_res = (1536, 1024, 1024, 512)
patience = 20                 # early stopping

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## Data loading and preprocessing

This part is the same for every model

In [ ]:
# Parameters
batch_size = 64
seed = 42

#Loading AnnData
print("Loading AnnData files...")
bulk_genes = sc.read_h5ad(gene_h5ad)
bulk_transcripts = sc.read_h5ad(isoform_h5ad)
print("Data loaded.")
bulk_genes, bulk_transcripts = align_anndata(bulk_genes, bulk_transcripts)

gene_to_transcripts = bulk_genes.uns.get("gene_to_transcripts") or {}

# Densifies and filters silent genes/isoforms
X_raw = densify(bulk_genes)
Y_raw = densify(bulk_transcripts)
X_raw, Y_raw, gene_names, transcript_names, gene_to_transcripts = filter_silent_genes_isoforms(
    X_raw,
    Y_raw,
    bulk_genes.var_names.to_numpy(),
    bulk_transcripts.var_names.to_numpy(),
    gene_to_transcripts,
)
print(
    f"Filtered zero-expression features: genes kept {len(gene_names)}/{bulk_genes.var_names.size}, "
    f"isoforms kept {len(transcript_names)}/{bulk_transcripts.var_names.size}"
)

# Prepares inputs and targets
X = normalize_inputs(X_raw)
Y = prepare_targets(Y_raw)

# Mapping transcript→gene
transcript_gene_idx, _ = build_transcript_gene_index(gene_to_transcripts, gene_names, transcript_names)

# Split
if whole_dataset:
    total = X.shape[0]
    train_n = int(total * 0.5)
    val_n = int(total * 0.35)
    test_n = total - train_n - val_n
    print(f"Using whole dataset: train {train_n}, val {val_n}, test {test_n}")

(train_pair, val_pair, test_pair) = train_val_test_split(
    X, Y, train_n=train_n, val_n=val_n, test_n=test_n, seed=seed
)

# DataLoader
train_loader = make_loader(train_pair, batch_size=batch_size, shuffle=True)
train_eval_loader = make_loader(train_pair, batch_size=batch_size, shuffle=False)
val_loader = make_loader(val_pair, batch_size=batch_size, shuffle=False)
test_loader = make_loader(test_pair, batch_size=batch_size, shuffle=False)
loaders = GeneIsoformDataLoaders(train_loader, train_eval_loader, val_loader, test_loader)


## Training

### FCNN - MLP

In [ ]:
device = torch.device("cpu" if args.cpu or not torch.cuda.is_available() else "cuda")
model = IsoformPredictor(
    n_inputs=X.shape[1],
    n_outputs=Y.shape[1],
    hidden_sizes=(args.hidden1, args.hidden2, args.hidden3),
    dropout=args.dropout,
).to(device)

history = train_model(
    model,
    loaders.train,
    loaders.val,
    device=device,
    epochs=args.epochs,
    lr=args.lr,
    weight_decay=args.weight_decay,
    patience=20,
)


### Results of last training

In [ ]:
torch version: 2.8.0+cu128
cuda available: True device count: 1
current device: 0 name: Tesla V100-PCIE-32GB
Loading AnnData files...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Using whole dataset: train 9941, val 6958, test 2983
Epoch 001 | train 7.7168 | val 2.9442 | lr 2.00e-04
Epoch 002 | train 2.0398 | val 1.4735 | lr 2.00e-04
Epoch 003 | train 1.4044 | val 1.2532 | lr 2.00e-04
Epoch 004 | train 1.2949 | val 1.1857 | lr 2.00e-04
Epoch 005 | train 1.2436 | val 1.1539 | lr 2.00e-04
Epoch 006 | train 1.2188 | val 1.1374 | lr 2.00e-04
Epoch 007 | train 1.1906 | val 1.1043 | lr 2.00e-04
Epoch 008 | train 1.1682 | val 1.0832 | lr 2.00e-04
Epoch 009 | train 1.1371 | val 1.0551 | lr 2.00e-04
Epoch 010 | train 1.1038 | val 1.0243 | lr 2.00e-04
Epoch 011 | train 1.0717 | val 0.9894 | lr 2.00e-04
Epoch 012 | train 1.0419 | val 0.9622 | lr 2.00e-04
Epoch 013 | train 1.0170 | val 0.9410 | lr 2.00e-04
Epoch 014 | train 0.9936 | val 0.9211 | lr 2.00e-04
Epoch 015 | train 0.9769 | val 0.8924 | lr 2.00e-04
Epoch 016 | train 0.9583 | val 0.8814 | lr 2.00e-04
Epoch 017 | train 0.9413 | val 0.8627 | lr 2.00e-04
Epoch 018 | train 0.9275 | val 0.8504 | lr 2.00e-04
Epoch 019 | train 0.9165 | val 0.8417 | lr 2.00e-04
Epoch 020 | train 0.9067 | val 0.8311 | lr 2.00e-04
Epoch 021 | train 0.8995 | val 0.8200 | lr 2.00e-04
Epoch 022 | train 0.8904 | val 0.8112 | lr 2.00e-04
Epoch 023 | train 0.8810 | val 0.8087 | lr 2.00e-04
Epoch 024 | train 0.8753 | val 0.8014 | lr 2.00e-04
Epoch 025 | train 0.8721 | val 0.8024 | lr 2.00e-04
Epoch 026 | train 0.8625 | val 0.7898 | lr 2.00e-04
Epoch 027 | train 0.8608 | val 0.7829 | lr 2.00e-04
Epoch 028 | train 0.8577 | val 0.7850 | lr 2.00e-04
Epoch 029 | train 0.8529 | val 0.7814 | lr 2.00e-04
Epoch 030 | train 0.8490 | val 0.7677 | lr 2.00e-04
Epoch 031 | train 0.8425 | val 0.7769 | lr 2.00e-04
Epoch 032 | train 0.8418 | val 0.7675 | lr 2.00e-04
Epoch 033 | train 0.8362 | val 0.7584 | lr 2.00e-04
Epoch 034 | train 0.8336 | val 0.7564 | lr 2.00e-04
Epoch 035 | train 0.8311 | val 0.7551 | lr 2.00e-04
Epoch 036 | train 0.8287 | val 0.7599 | lr 2.00e-04
Epoch 037 | train 0.8248 | val 0.7556 | lr 2.00e-04
Epoch 038 | train 0.8224 | val 0.7480 | lr 2.00e-04
Epoch 039 | train 0.8190 | val 0.7453 | lr 2.00e-04
Epoch 040 | train 0.8200 | val 0.7458 | lr 2.00e-04
Epoch 041 | train 0.8154 | val 0.7378 | lr 2.00e-04
Epoch 042 | train 0.8129 | val 0.7363 | lr 2.00e-04
Epoch 043 | train 0.8106 | val 0.7329 | lr 2.00e-04
Epoch 044 | train 0.8071 | val 0.7372 | lr 2.00e-04
Epoch 045 | train 0.8078 | val 0.7336 | lr 2.00e-04
Epoch 046 | train 0.8047 | val 0.7317 | lr 2.00e-04
Epoch 047 | train 0.8051 | val 0.7359 | lr 2.00e-04
Epoch 048 | train 0.8005 | val 0.7179 | lr 2.00e-04
Epoch 049 | train 0.7994 | val 0.7235 | lr 2.00e-04
Epoch 050 | train 0.8001 | val 0.7219 | lr 2.00e-04
Epoch 051 | train 0.7961 | val 0.7183 | lr 2.00e-04
Epoch 052 | train 0.7949 | val 0.7153 | lr 2.00e-04
Epoch 053 | train 0.7926 | val 0.7170 | lr 2.00e-04
Epoch 054 | train 0.7908 | val 0.7202 | lr 2.00e-04
Epoch 055 | train 0.7899 | val 0.7185 | lr 2.00e-04
Epoch 056 | train 0.7894 | val 0.7116 | lr 2.00e-04
Epoch 057 | train 0.7889 | val 0.7164 | lr 2.00e-04
Epoch 058 | train 0.7845 | val 0.7073 | lr 2.00e-04
Epoch 059 | train 0.7884 | val 0.7090 | lr 2.00e-04
Epoch 060 | train 0.7827 | val 0.7103 | lr 2.00e-04
Epoch 061 | train 0.7852 | val 0.7072 | lr 2.00e-04
Epoch 062 | train 0.7838 | val 0.7027 | lr 2.00e-04
Epoch 063 | train 0.7809 | val 0.7043 | lr 2.00e-04
Epoch 064 | train 0.7810 | val 0.7010 | lr 2.00e-04
Epoch 065 | train 0.7796 | val 0.7025 | lr 2.00e-04
Epoch 066 | train 0.7779 | val 0.6983 | lr 2.00e-04
Epoch 067 | train 0.7766 | val 0.6959 | lr 2.00e-04
Epoch 068 | train 0.7781 | val 0.6992 | lr 2.00e-04
Epoch 069 | train 0.7765 | val 0.7013 | lr 2.00e-04
Epoch 070 | train 0.7734 | val 0.6980 | lr 2.00e-04
Epoch 071 | train 0.7738 | val 0.6984 | lr 2.00e-04
Epoch 072 | train 0.7729 | val 0.6983 | lr 2.00e-04
Epoch 073 | train 0.7737 | val 0.6936 | lr 2.00e-04
Epoch 074 | train 0.7701 | val 0.6895 | lr 2.00e-04
Epoch 075 | train 0.7688 | val 0.6890 | lr 2.00e-04
Epoch 076 | train 0.7716 | val 0.6879 | lr 2.00e-04
Epoch 077 | train 0.7707 | val 0.6872 | lr 2.00e-04
Epoch 078 | train 0.7722 | val 0.6936 | lr 2.00e-04
Epoch 079 | train 0.7677 | val 0.6888 | lr 2.00e-04
Epoch 080 | train 0.7678 | val 0.6897 | lr 2.00e-04
Epoch 081 | train 0.7688 | val 0.6841 | lr 2.00e-04
Epoch 082 | train 0.7657 | val 0.6910 | lr 2.00e-04
Epoch 083 | train 0.7663 | val 0.6876 | lr 2.00e-04
Epoch 084 | train 0.7652 | val 0.6872 | lr 2.00e-04
Epoch 085 | train 0.7633 | val 0.6817 | lr 2.00e-04
Epoch 086 | train 0.7645 | val 0.6809 | lr 2.00e-04
Epoch 087 | train 0.7663 | val 0.6815 | lr 2.00e-04
Epoch 088 | train 0.7631 | val 0.6800 | lr 2.00e-04
Epoch 089 | train 0.7625 | val 0.6793 | lr 2.00e-04
Epoch 090 | train 0.7620 | val 0.6880 | lr 2.00e-04
Epoch 091 | train 0.7600 | val 0.6820 | lr 2.00e-04
Epoch 092 | train 0.7614 | val 0.6798 | lr 2.00e-04
Epoch 093 | train 0.7589 | val 0.6772 | lr 2.00e-04
Epoch 094 | train 0.7612 | val 0.6801 | lr 2.00e-04
Epoch 095 | train 0.7611 | val 0.6810 | lr 2.00e-04
Epoch 096 | train 0.7609 | val 0.6768 | lr 2.00e-04
Epoch 097 | train 0.7593 | val 0.6734 | lr 2.00e-04
Epoch 098 | train 0.7587 | val 0.6816 | lr 2.00e-04
Epoch 099 | train 0.7571 | val 0.6772 | lr 2.00e-04
Epoch 100 | train 0.7592 | val 0.6762 | lr 2.00e-04
Epoch 101 | train 0.7566 | val 0.6758 | lr 2.00e-04
Epoch 102 | train 0.7563 | val 0.6741 | lr 2.00e-04
Epoch 103 | train 0.7577 | val 0.6718 | lr 2.00e-04
Epoch 104 | train 0.7558 | val 0.6762 | lr 2.00e-04
Epoch 105 | train 0.7549 | val 0.6778 | lr 2.00e-04
Epoch 106 | train 0.7577 | val 0.6709 | lr 2.00e-04
Epoch 107 | train 0.7556 | val 0.6700 | lr 2.00e-04
Epoch 108 | train 0.7544 | val 0.6774 | lr 2.00e-04
Epoch 109 | train 0.7534 | val 0.6757 | lr 2.00e-04
Epoch 110 | train 0.7518 | val 0.6817 | lr 2.00e-04
Epoch 111 | train 0.7528 | val 0.6725 | lr 2.00e-04
Epoch 112 | train 0.7528 | val 0.6700 | lr 2.00e-04
Epoch 113 | train 0.7520 | val 0.6684 | lr 2.00e-04
Epoch 114 | train 0.7526 | val 0.6698 | lr 2.00e-04
Epoch 115 | train 0.7538 | val 0.6691 | lr 2.00e-04
Epoch 116 | train 0.7519 | val 0.6700 | lr 2.00e-04
Epoch 117 | train 0.7526 | val 0.6726 | lr 2.00e-04
Epoch 118 | train 0.7507 | val 0.6684 | lr 2.00e-04
Epoch 119 | train 0.7514 | val 0.6650 | lr 2.00e-04
Epoch 120 | train 0.7507 | val 0.6690 | lr 2.00e-04
Epoch 121 | train 0.7509 | val 0.6739 | lr 2.00e-04
Epoch 122 | train 0.7506 | val 0.6687 | lr 2.00e-04
Epoch 123 | train 0.7487 | val 0.6679 | lr 2.00e-04
Epoch 124 | train 0.7506 | val 0.6679 | lr 2.00e-04
Epoch 125 | train 0.7500 | val 0.6652 | lr 2.00e-04
Epoch 126 | train 0.7496 | val 0.6668 | lr 2.00e-04
Epoch 127 | train 0.7471 | val 0.6636 | lr 2.00e-04
Epoch 128 | train 0.7486 | val 0.6657 | lr 2.00e-04
Epoch 129 | train 0.7461 | val 0.6630 | lr 2.00e-04
Epoch 130 | train 0.7476 | val 0.6638 | lr 2.00e-04
Epoch 131 | train 0.7481 | val 0.6594 | lr 2.00e-04
Epoch 132 | train 0.7496 | val 0.6621 | lr 2.00e-04
Epoch 133 | train 0.7473 | val 0.6645 | lr 2.00e-04
Epoch 134 | train 0.7462 | val 0.6665 | lr 2.00e-04
Epoch 135 | train 0.7465 | val 0.6630 | lr 2.00e-04
Epoch 136 | train 0.7463 | val 0.6641 | lr 2.00e-04
Epoch 137 | train 0.7467 | val 0.6619 | lr 2.00e-04
Epoch 138 | train 0.7472 | val 0.6588 | lr 2.00e-04
Epoch 139 | train 0.7455 | val 0.6623 | lr 2.00e-04
Epoch 140 | train 0.7442 | val 0.6558 | lr 2.00e-04
Epoch 141 | train 0.7436 | val 0.6629 | lr 2.00e-04
Epoch 142 | train 0.7446 | val 0.6566 | lr 2.00e-04
Epoch 143 | train 0.7447 | val 0.6691 | lr 2.00e-04
Epoch 144 | train 0.7436 | val 0.6613 | lr 2.00e-04
Epoch 145 | train 0.7426 | val 0.6598 | lr 2.00e-04
Epoch 146 | train 0.7444 | val 0.6597 | lr 2.00e-04
Epoch 147 | train 0.7455 | val 0.6557 | lr 2.00e-04
Epoch 148 | train 0.7434 | val 0.6599 | lr 2.00e-04
Epoch 149 | train 0.7417 | val 0.6630 | lr 2.00e-04
Epoch 150 | train 0.7409 | val 0.6551 | lr 2.00e-04
Epoch 151 | train 0.7432 | val 0.6637 | lr 2.00e-04
Epoch 152 | train 0.7439 | val 0.6605 | lr 2.00e-04
Epoch 153 | train 0.7453 | val 0.6609 | lr 2.00e-04
Epoch 154 | train 0.7428 | val 0.6606 | lr 2.00e-04
Epoch 155 | train 0.7430 | val 0.6542 | lr 2.00e-04
Epoch 156 | train 0.7434 | val 0.6574 | lr 2.00e-04
Epoch 157 | train 0.7419 | val 0.6577 | lr 2.00e-04
Epoch 158 | train 0.7433 | val 0.6555 | lr 2.00e-04
Epoch 159 | train 0.7413 | val 0.6549 | lr 2.00e-04
Epoch 160 | train 0.7413 | val 0.6562 | lr 2.00e-04
Epoch 161 | train 0.7436 | val 0.6558 | lr 2.00e-04
Epoch 162 | train 0.7410 | val 0.6573 | lr 2.00e-04
Epoch 163 | train 0.7406 | val 0.6579 | lr 2.00e-04
Epoch 164 | train 0.7398 | val 0.6527 | lr 2.00e-04
Epoch 165 | train 0.7421 | val 0.6617 | lr 2.00e-04
Epoch 166 | train 0.7399 | val 0.6547 | lr 2.00e-04
Epoch 167 | train 0.7397 | val 0.6560 | lr 2.00e-04
Epoch 168 | train 0.7412 | val 0.6515 | lr 2.00e-04
Epoch 169 | train 0.7391 | val 0.6534 | lr 2.00e-04
Epoch 170 | train 0.7407 | val 0.6548 | lr 2.00e-04
Epoch 171 | train 0.7404 | val 0.6540 | lr 2.00e-04
Epoch 172 | train 0.7388 | val 0.6503 | lr 2.00e-04
Epoch 173 | train 0.7399 | val 0.6532 | lr 2.00e-04
Epoch 174 | train 0.7396 | val 0.6520 | lr 2.00e-04
Epoch 175 | train 0.7401 | val 0.6505 | lr 2.00e-04
Epoch 176 | train 0.7391 | val 0.6538 | lr 2.00e-04
Epoch 177 | train 0.7401 | val 0.6492 | lr 2.00e-04
Epoch 178 | train 0.7385 | val 0.6487 | lr 2.00e-04
Epoch 179 | train 0.7383 | val 0.6517 | lr 2.00e-04
Epoch 180 | train 0.7403 | val 0.6509 | lr 2.00e-04
Epoch 181 | train 0.7389 | val 0.6487 | lr 2.00e-04
Epoch 182 | train 0.7382 | val 0.6527 | lr 2.00e-04
Epoch 183 | train 0.7397 | val 0.6559 | lr 2.00e-04
Epoch 184 | train 0.7387 | val 0.6573 | lr 2.00e-04
Epoch 185 | train 0.7370 | val 0.6550 | lr 2.00e-04
Epoch 186 | train 0.7367 | val 0.6482 | lr 2.00e-04
Epoch 187 | train 0.7376 | val 0.6521 | lr 2.00e-04
Epoch 188 | train 0.7376 | val 0.6536 | lr 2.00e-04
Epoch 189 | train 0.7385 | val 0.6470 | lr 2.00e-04
Epoch 190 | train 0.7390 | val 0.6545 | lr 2.00e-04
Epoch 191 | train 0.7376 | val 0.6509 | lr 2.00e-04
Epoch 192 | train 0.7377 | val 0.6474 | lr 2.00e-04
Epoch 193 | train 0.7361 | val 0.6512 | lr 2.00e-04
Epoch 194 | train 0.7394 | val 0.6533 | lr 2.00e-04
Epoch 195 | train 0.7373 | val 0.6448 | lr 2.00e-04
Epoch 196 | train 0.7380 | val 0.6464 | lr 2.00e-04
Epoch 197 | train 0.7360 | val 0.6441 | lr 2.00e-04
Epoch 198 | train 0.7365 | val 0.6460 | lr 2.00e-04
Epoch 199 | train 0.7385 | val 0.6541 | lr 2.00e-04
Epoch 200 | train 0.7371 | val 0.6473 | lr 2.00e-04
Epoch 201 | train 0.7351 | val 0.6485 | lr 2.00e-04
Epoch 202 | train 0.7359 | val 0.6495 | lr 2.00e-04
Epoch 203 | train 0.7349 | val 0.6518 | lr 2.00e-04
Epoch 204 | train 0.7358 | val 0.6478 | lr 2.00e-04
Epoch 205 | train 0.7356 | val 0.6474 | lr 2.00e-04
Epoch 206 | train 0.7327 | val 0.6436 | lr 2.00e-04
Epoch 207 | train 0.7359 | val 0.6429 | lr 2.00e-04
Epoch 208 | train 0.7372 | val 0.6508 | lr 2.00e-04
Epoch 209 | train 0.7353 | val 0.6533 | lr 2.00e-04
Epoch 210 | train 0.7349 | val 0.6444 | lr 2.00e-04
Epoch 211 | train 0.7351 | val 0.6474 | lr 2.00e-04
Epoch 212 | train 0.7341 | val 0.6466 | lr 2.00e-04
Epoch 213 | train 0.7350 | val 0.6485 | lr 2.00e-04
Epoch 214 | train 0.7345 | val 0.6451 | lr 2.00e-04
Epoch 215 | train 0.7346 | val 0.6488 | lr 2.00e-04
Epoch 216 | train 0.7330 | val 0.6470 | lr 2.00e-04
Epoch 217 | train 0.7341 | val 0.6505 | lr 2.00e-04
Epoch 218 | train 0.7338 | val 0.6475 | lr 1.00e-04
Epoch 219 | train 0.7282 | val 0.6369 | lr 1.00e-04
Epoch 220 | train 0.7230 | val 0.6442 | lr 1.00e-04
Epoch 221 | train 0.7244 | val 0.6399 | lr 1.00e-04
Epoch 222 | train 0.7232 | val 0.6379 | lr 1.00e-04
Epoch 223 | train 0.7246 | val 0.6349 | lr 1.00e-04
Epoch 224 | train 0.7227 | val 0.6402 | lr 1.00e-04
Epoch 225 | train 0.7211 | val 0.6345 | lr 1.00e-04
Epoch 226 | train 0.7224 | val 0.6386 | lr 1.00e-04
Epoch 227 | train 0.7217 | val 0.6371 | lr 1.00e-04
Epoch 228 | train 0.7211 | val 0.6332 | lr 1.00e-04
Epoch 229 | train 0.7217 | val 0.6385 | lr 1.00e-04
Epoch 230 | train 0.7224 | val 0.6377 | lr 1.00e-04
Epoch 231 | train 0.7214 | val 0.6371 | lr 1.00e-04
Epoch 232 | train 0.7223 | val 0.6416 | lr 1.00e-04
Epoch 233 | train 0.7220 | val 0.6358 | lr 1.00e-04
Epoch 234 | train 0.7217 | val 0.6364 | lr 1.00e-04
Epoch 235 | train 0.7214 | val 0.6338 | lr 1.00e-04
Epoch 236 | train 0.7204 | val 0.6328 | lr 1.00e-04
Epoch 237 | train 0.7211 | val 0.6351 | lr 1.00e-04
Epoch 238 | train 0.7218 | val 0.6414 | lr 1.00e-04
Epoch 239 | train 0.7205 | val 0.6357 | lr 1.00e-04
Epoch 240 | train 0.7206 | val 0.6360 | lr 1.00e-04
Epoch 241 | train 0.7211 | val 0.6340 | lr 1.00e-04
Epoch 242 | train 0.7208 | val 0.6327 | lr 1.00e-04
Epoch 243 | train 0.7210 | val 0.6365 | lr 1.00e-04
Epoch 244 | train 0.7212 | val 0.6320 | lr 1.00e-04
Epoch 245 | train 0.7200 | val 0.6346 | lr 1.00e-04
Epoch 246 | train 0.7209 | val 0.6322 | lr 1.00e-04
Epoch 247 | train 0.7194 | val 0.6345 | lr 1.00e-04
Epoch 248 | train 0.7201 | val 0.6363 | lr 1.00e-04
Epoch 249 | train 0.7188 | val 0.6348 | lr 1.00e-04
Epoch 250 | train 0.7179 | val 0.6345 | lr 1.00e-04
Epoch 251 | train 0.7195 | val 0.6318 | lr 1.00e-04
Epoch 252 | train 0.7199 | val 0.6339 | lr 1.00e-04
Epoch 253 | train 0.7213 | val 0.6342 | lr 1.00e-04
Epoch 254 | train 0.7206 | val 0.6365 | lr 1.00e-04
Epoch 255 | train 0.7208 | val 0.6341 | lr 1.00e-04
Epoch 256 | train 0.7190 | val 0.6318 | lr 1.00e-04
Epoch 257 | train 0.7193 | val 0.6371 | lr 1.00e-04
Epoch 258 | train 0.7201 | val 0.6352 | lr 1.00e-04
Epoch 259 | train 0.7170 | val 0.6350 | lr 1.00e-04
Epoch 260 | train 0.7193 | val 0.6357 | lr 1.00e-04
Epoch 261 | train 0.7173 | val 0.6336 | lr 1.00e-04
Epoch 262 | train 0.7190 | val 0.6362 | lr 5.00e-05
Epoch 263 | train 0.7163 | val 0.6284 | lr 5.00e-05
Epoch 264 | train 0.7152 | val 0.6307 | lr 5.00e-05
Epoch 265 | train 0.7136 | val 0.6305 | lr 5.00e-05
Epoch 266 | train 0.7128 | val 0.6319 | lr 5.00e-05
Epoch 267 | train 0.7121 | val 0.6272 | lr 5.00e-05
Epoch 268 | train 0.7125 | val 0.6268 | lr 5.00e-05
Epoch 269 | train 0.7124 | val 0.6266 | lr 5.00e-05
Epoch 270 | train 0.7136 | val 0.6324 | lr 5.00e-05
Epoch 271 | train 0.7137 | val 0.6292 | lr 5.00e-05
Epoch 272 | train 0.7131 | val 0.6291 | lr 5.00e-05
Epoch 273 | train 0.7123 | val 0.6260 | lr 5.00e-05
Epoch 274 | train 0.7138 | val 0.6292 | lr 5.00e-05
Epoch 275 | train 0.7125 | val 0.6287 | lr 5.00e-05
Epoch 276 | train 0.7114 | val 0.6298 | lr 5.00e-05
Epoch 277 | train 0.7119 | val 0.6290 | lr 5.00e-05
Epoch 278 | train 0.7117 | val 0.6288 | lr 5.00e-05
Epoch 279 | train 0.7130 | val 0.6299 | lr 5.00e-05
Epoch 280 | train 0.7109 | val 0.6285 | lr 5.00e-05
Epoch 281 | train 0.7119 | val 0.6290 | lr 5.00e-05
Epoch 282 | train 0.7126 | val 0.6301 | lr 5.00e-05
Epoch 283 | train 0.7113 | val 0.6322 | lr 5.00e-05
Epoch 284 | train 0.7122 | val 0.6274 | lr 2.50e-05
Epoch 285 | train 0.7095 | val 0.6309 | lr 2.50e-05
Epoch 286 | train 0.7085 | val 0.6240 | lr 2.50e-05
Epoch 287 | train 0.7109 | val 0.6235 | lr 2.50e-05
Epoch 288 | train 0.7100 | val 0.6271 | lr 2.50e-05
Epoch 289 | train 0.7088 | val 0.6291 | lr 2.50e-05
Epoch 290 | train 0.7081 | val 0.6298 | lr 2.50e-05
Epoch 291 | train 0.7081 | val 0.6268 | lr 2.50e-05
Epoch 292 | train 0.7072 | val 0.6314 | lr 2.50e-05
Epoch 293 | train 0.7088 | val 0.6279 | lr 2.50e-05
Epoch 294 | train 0.7083 | val 0.6240 | lr 2.50e-05
Epoch 295 | train 0.7083 | val 0.6242 | lr 2.50e-05
Epoch 296 | train 0.7068 | val 0.6300 | lr 2.50e-05
Epoch 297 | train 0.7065 | val 0.6265 | lr 2.50e-05
Epoch 298 | train 0.7091 | val 0.6234 | lr 2.50e-05
Epoch 299 | train 0.7069 | val 0.6256 | lr 2.50e-05
Epoch 300 | train 0.7079 | val 0.6236 | lr 2.50e-05

### Residual-MLP


In [ ]:
device = torch.device("cpu" if args.cpu or not torch.cuda.is_available() else "cuda")

model = ResidualIsoformPredictor(
    n_inputs=X.shape[1],
    n_outputs=Y.shape[1],
    hidden_sizes=(args.hidden1, args.hidden2, args.hidden3, args.hidden4),
    dropout=args.dropout,
).to(device)


history = train_model(
    model,
    loaders.train,
    loaders.val,
    device=device,
    epochs=args.epochs,
    lr=args.lr,
    weight_decay=args.weight_decay,
    patience=20,
)

### Results of last training

In [ ]:
torch version: 2.8.0+cu128
cuda available: True device count: 1
current device: 0 name: Tesla V100-PCIE-32GB
Loading AnnData files...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Using whole dataset: train 9941, val 6958, test 2983
Epoch 001 | train 2.3898 | val 1.2134
Epoch 002 | train 1.1711 | val 1.1082
Epoch 003 | train 1.0350 | val 0.9818
Epoch 004 | train 0.9492 | val 0.9177
Epoch 005 | train 0.8824 | val 0.8608
Epoch 006 | train 0.8435 | val 0.8287
Epoch 007 | train 0.8152 | val 0.8073
Epoch 008 | train 0.7945 | val 0.7872
Epoch 009 | train 0.7752 | val 0.7698
Epoch 010 | train 0.7574 | val 0.7527
Epoch 011 | train 0.7431 | val 0.7447
Epoch 012 | train 0.7314 | val 0.7266
Epoch 013 | train 0.7171 | val 0.7151
Epoch 014 | train 0.7054 | val 0.7050
Epoch 015 | train 0.6952 | val 0.6988
Epoch 016 | train 0.6824 | val 0.6824
Epoch 017 | train 0.6739 | val 0.6757
Epoch 018 | train 0.6623 | val 0.6696
Epoch 019 | train 0.6549 | val 0.6581
Epoch 020 | train 0.6470 | val 0.6554
Epoch 021 | train 0.6415 | val 0.6469
Epoch 022 | train 0.6343 | val 0.6398
Epoch 023 | train 0.6280 | val 0.6405
Epoch 024 | train 0.6209 | val 0.6344
Epoch 025 | train 0.6160 | val 0.6271
Epoch 026 | train 0.6100 | val 0.6215
Epoch 027 | train 0.6047 | val 0.6152
Epoch 028 | train 0.6013 | val 0.6129
Epoch 029 | train 0.5963 | val 0.6127
Epoch 030 | train 0.5921 | val 0.6058
Epoch 031 | train 0.5876 | val 0.6046
Epoch 032 | train 0.5860 | val 0.5998
Epoch 033 | train 0.5814 | val 0.5994
Epoch 034 | train 0.5802 | val 0.5950
Epoch 035 | train 0.5758 | val 0.5928
Epoch 036 | train 0.5722 | val 0.5896
Epoch 037 | train 0.5675 | val 0.5922
Epoch 038 | train 0.5666 | val 0.5881
Epoch 039 | train 0.5630 | val 0.5832
Epoch 040 | train 0.5609 | val 0.5820
Epoch 041 | train 0.5571 | val 0.5823
Epoch 042 | train 0.5550 | val 0.5776
Epoch 043 | train 0.5524 | val 0.5759
Epoch 044 | train 0.5503 | val 0.5740
Epoch 045 | train 0.5479 | val 0.5759
Epoch 046 | train 0.5453 | val 0.5755
Epoch 047 | train 0.5430 | val 0.5722
Epoch 048 | train 0.5390 | val 0.5682
Epoch 049 | train 0.5383 | val 0.5677
Epoch 050 | train 0.5364 | val 0.5662
Epoch 051 | train 0.5351 | val 0.5679
Epoch 052 | train 0.5330 | val 0.5659
Epoch 053 | train 0.5308 | val 0.5675
Epoch 054 | train 0.5286 | val 0.5653
Epoch 055 | train 0.5261 | val 0.5664
Epoch 056 | train 0.5248 | val 0.5610
Epoch 057 | train 0.5245 | val 0.5608
Epoch 058 | train 0.5223 | val 0.5571
Epoch 059 | train 0.5217 | val 0.5588
Epoch 060 | train 0.5198 | val 0.5598
Epoch 061 | train 0.5193 | val 0.5576
Epoch 062 | train 0.5173 | val 0.5589
Epoch 063 | train 0.5162 | val 0.5561
Epoch 064 | train 0.5156 | val 0.5527
Epoch 065 | train 0.5130 | val 0.5550
Epoch 066 | train 0.5113 | val 0.5542
Epoch 067 | train 0.5112 | val 0.5525
Epoch 068 | train 0.5113 | val 0.5528
Epoch 069 | train 0.5094 | val 0.5528
Epoch 070 | train 0.5079 | val 0.5518
Epoch 071 | train 0.5072 | val 0.5519
Epoch 072 | train 0.5069 | val 0.5558
Epoch 073 | train 0.5063 | val 0.5499
Epoch 074 | train 0.5056 | val 0.5544
Epoch 075 | train 0.5055 | val 0.5538
Epoch 076 | train 0.5034 | val 0.5496
Epoch 077 | train 0.5028 | val 0.5518
Epoch 078 | train 0.5017 | val 0.5497
Epoch 079 | train 0.5004 | val 0.5501
Epoch 080 | train 0.5025 | val 0.5527
Epoch 081 | train 0.5015 | val 0.5493
Epoch 082 | train 0.4993 | val 0.5465
Epoch 083 | train 0.4979 | val 0.5500
Epoch 084 | train 0.4978 | val 0.5490
Epoch 085 | train 0.4972 | val 0.5458
Epoch 086 | train 0.4967 | val 0.5479
Epoch 087 | train 0.4953 | val 0.5454
Epoch 088 | train 0.4959 | val 0.5473
Epoch 089 | train 0.4961 | val 0.5453
Epoch 090 | train 0.4950 | val 0.5457
Epoch 091 | train 0.4942 | val 0.5449
Epoch 092 | train 0.4937 | val 0.5472
Epoch 093 | train 0.4937 | val 0.5432
Epoch 094 | train 0.4926 | val 0.5456
Epoch 095 | train 0.4926 | val 0.5448
Epoch 096 | train 0.4920 | val 0.5456
Epoch 097 | train 0.4914 | val 0.5447
Epoch 098 | train 0.4909 | val 0.5449
Epoch 099 | train 0.4914 | val 0.5437
Epoch 100 | train 0.4900 | val 0.5422
Epoch 101 | train 0.4889 | val 0.5432
Epoch 102 | train 0.4891 | val 0.5426
Epoch 103 | train 0.4887 | val 0.5410
Epoch 104 | train 0.4883 | val 0.5437
Epoch 105 | train 0.4882 | val 0.5413
Epoch 106 | train 0.4873 | val 0.5412
Epoch 107 | train 0.4864 | val 0.5448
Epoch 108 | train 0.4869 | val 0.5411
Epoch 109 | train 0.4870 | val 0.5417
Epoch 110 | train 0.4867 | val 0.5417
Epoch 111 | train 0.4860 | val 0.5386
Epoch 112 | train 0.4854 | val 0.5392
Epoch 113 | train 0.4849 | val 0.5397
Epoch 114 | train 0.4857 | val 0.5402
Epoch 115 | train 0.4838 | val 0.5386
Epoch 116 | train 0.4835 | val 0.5390
Epoch 117 | train 0.4840 | val 0.5425
Epoch 118 | train 0.4837 | val 0.5388
Epoch 119 | train 0.4831 | val 0.5365
Epoch 120 | train 0.4830 | val 0.5443
Epoch 121 | train 0.4835 | val 0.5370
Epoch 122 | train 0.4824 | val 0.5380
Epoch 123 | train 0.4821 | val 0.5368
Epoch 124 | train 0.4816 | val 0.5371
Epoch 125 | train 0.4820 | val 0.5367
Epoch 126 | train 0.4811 | val 0.5388
Epoch 127 | train 0.4816 | val 0.5402
Epoch 128 | train 0.4810 | val 0.5361
Epoch 129 | train 0.4800 | val 0.5402
Epoch 130 | train 0.4791 | val 0.5353
Epoch 131 | train 0.4800 | val 0.5424
Epoch 132 | train 0.4811 | val 0.5354
Epoch 133 | train 0.4792 | val 0.5348
Epoch 134 | train 0.4783 | val 0.5358
Epoch 135 | train 0.4791 | val 0.5383
Epoch 136 | train 0.4786 | val 0.5345
Epoch 137 | train 0.4782 | val 0.5357
Epoch 138 | train 0.4775 | val 0.5334
Epoch 139 | train 0.4780 | val 0.5339
Epoch 140 | train 0.4776 | val 0.5355
Epoch 141 | train 0.4774 | val 0.5345
Epoch 142 | train 0.4773 | val 0.5324
Epoch 143 | train 0.4766 | val 0.5341
Epoch 144 | train 0.4785 | val 0.5342
Epoch 145 | train 0.4766 | val 0.5325
Epoch 146 | train 0.4759 | val 0.5330
Epoch 147 | train 0.4762 | val 0.5348
Epoch 148 | train 0.4765 | val 0.5332
Epoch 149 | train 0.4753 | val 0.5327
Epoch 150 | train 0.4748 | val 0.5324
Epoch 151 | train 0.4757 | val 0.5327
Epoch 152 | train 0.4752 | val 0.5338
Epoch 153 | train 0.4744 | val 0.5337
Epoch 154 | train 0.4738 | val 0.5325
Epoch 155 | train 0.4740 | val 0.5323
Epoch 156 | train 0.4740 | val 0.5325
Epoch 157 | train 0.4739 | val 0.5321
Epoch 158 | train 0.4732 | val 0.5319
Epoch 159 | train 0.4736 | val 0.5375
Epoch 160 | train 0.4740 | val 0.5331
Epoch 161 | train 0.4740 | val 0.5370
Epoch 162 | train 0.4741 | val 0.5301
Epoch 163 | train 0.4726 | val 0.5317
Epoch 164 | train 0.4719 | val 0.5318
Epoch 165 | train 0.4726 | val 0.5305
Epoch 166 | train 0.4726 | val 0.5300
Epoch 167 | train 0.4721 | val 0.5320
Epoch 168 | train 0.4723 | val 0.5309
Epoch 169 | train 0.4717 | val 0.5346
Epoch 170 | train 0.4712 | val 0.5304
Epoch 171 | train 0.4712 | val 0.5302
Epoch 172 | train 0.4716 | val 0.5308
Epoch 173 | train 0.4713 | val 0.5307
Epoch 174 | train 0.4713 | val 0.5299
Epoch 175 | train 0.4709 | val 0.5288
Epoch 176 | train 0.4715 | val 0.5296
Epoch 177 | train 0.4691 | val 0.5290
Epoch 178 | train 0.4699 | val 0.5301
Epoch 179 | train 0.4695 | val 0.5318
Epoch 180 | train 0.4703 | val 0.5293
Epoch 181 | train 0.4707 | val 0.5317
Epoch 182 | train 0.4699 | val 0.5304
Epoch 183 | train 0.4686 | val 0.5297
Epoch 184 | train 0.4692 | val 0.5307
Epoch 185 | train 0.4696 | val 0.5297
Epoch 186 | train 0.4684 | val 0.5287
Epoch 187 | train 0.4693 | val 0.5292
Epoch 188 | train 0.4696 | val 0.5274
Epoch 189 | train 0.4684 | val 0.5305
Epoch 190 | train 0.4690 | val 0.5279
Epoch 191 | train 0.4678 | val 0.5291
Epoch 192 | train 0.4688 | val 0.5284
Epoch 193 | train 0.4694 | val 0.5291
Epoch 194 | train 0.4685 | val 0.5315
Epoch 195 | train 0.4677 | val 0.5285
Epoch 196 | train 0.4678 | val 0.5300
Epoch 197 | train 0.4670 | val 0.5283
Epoch 198 | train 0.4669 | val 0.5304
Epoch 199 | train 0.4674 | val 0.5276
Epoch 200 | train 0.4672 | val 0.5270
Epoch 201 | train 0.4669 | val 0.5277
Epoch 202 | train 0.4670 | val 0.5265
Epoch 203 | train 0.4666 | val 0.5253
Epoch 204 | train 0.4661 | val 0.5273
Epoch 205 | train 0.4662 | val 0.5285
Epoch 206 | train 0.4671 | val 0.5278
Epoch 207 | train 0.4664 | val 0.5284
Epoch 208 | train 0.4658 | val 0.5277
Epoch 209 | train 0.4659 | val 0.5269
Epoch 210 | train 0.4662 | val 0.5271
Epoch 211 | train 0.4652 | val 0.5289
Epoch 212 | train 0.4657 | val 0.5265
Epoch 213 | train 0.4653 | val 0.5266
Epoch 214 | train 0.4656 | val 0.5258
Epoch 215 | train 0.4650 | val 0.5263
Epoch 216 | train 0.4645 | val 0.5265
Epoch 217 | train 0.4650 | val 0.5268
Epoch 218 | train 0.4646 | val 0.5274
Epoch 219 | train 0.4648 | val 0.5243
Epoch 220 | train 0.4651 | val 0.5269
Epoch 221 | train 0.4650 | val 0.5273
Epoch 222 | train 0.4649 | val 0.5252
Epoch 223 | train 0.4648 | val 0.5291
Epoch 224 | train 0.4645 | val 0.5269
Epoch 225 | train 0.4640 | val 0.5262
Epoch 226 | train 0.4642 | val 0.5262
Epoch 227 | train 0.4646 | val 0.5252
Epoch 228 | train 0.4639 | val 0.5274
Epoch 229 | train 0.4646 | val 0.5283
Epoch 230 | train 0.4639 | val 0.5246
Epoch 231 | train 0.4632 | val 0.5252
Epoch 232 | train 0.4627 | val 0.5254
Epoch 233 | train 0.4630 | val 0.5247
Epoch 234 | train 0.4626 | val 0.5261
Epoch 235 | train 0.4629 | val 0.5268
Epoch 236 | train 0.4627 | val 0.5252
Epoch 237 | train 0.4626 | val 0.5256
Epoch 238 | train 0.4628 | val 0.5275
Epoch 239 | train 0.4635 | val 0.5249
No val improvement for 20 epochs, stopping early at epoch 239.

## Evaluation and results

### FCNN_MLP

Predictions per split + log-MSE

In [ ]:
criterion = nn.MSELoss()
split_loaders = {"train": loaders.train_eval, "val": loaders.val, "test": loaders.test}
split_metrics = {}
for split_name, loader in split_loaders.items():
    mse = evaluate_loss(model, loader, device, criterion)
    preds_log, targets_log = inference(model, loader, device)
    preds_counts = np.expm1(preds_log)
    targets_counts = np.expm1(targets_log)
    split_metrics[split_name] = {
        "mse": mse,
        "pred_counts": preds_counts,
        "true_counts": targets_counts,
    }
    print(f"{split_name.capitalize()} | log-space MSE: {mse:.4f}")

In [ ]:
Train | log-space MSE: 0.5959
Val | log-space MSE: 0.6234
Test | log-space MSE: 0.6222

Correlation per isoform counts + boxplot

In [ ]:
test_preds_counts = split_metrics["test"]["pred_counts"]
test_true_counts  = split_metrics["test"]["true_counts"]

iso_corr = isoform_correlations(test_preds_counts, test_true_counts)
valid_corr = iso_corr[~np.isnan(iso_corr)]
if valid_corr.size == 0:
    print("Isoform correlation: no isoforms with variance; skipping boxplot.")
else:
    corr_mean = float(valid_corr.mean())
    print(f"Isoform correlation: mean Pearson {corr_mean:.4f} over {valid_corr.size}/{iso_corr.size} isoforms")
    plt.figure(figsize=(6, 4))
    plt.boxplot(valid_corr, vert=True, patch_artist=True)
    plt.ylabel("Pearson correlation")
    plt.title("Isoform prediction correlation (test)")
    plt.tight_layout()
    plt.show()

In [ ]:
Isoform correlation: mean Pearson 0.5692 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_fcnn/isoform_correlation_boxplot_fcnn.png

# INSERT BOXPLOT IMAGE

Scatterplot log1p pred VS true + regression line

In [ ]:
flat_pred = test_preds_counts.ravel()
flat_true = test_true_counts.ravel()
max_points = 50000
if flat_pred.size > max_points:
    rng = np.random.default_rng(42)
    idx = rng.choice(flat_pred.size, size=max_points, replace=False)
    flat_pred = flat_pred[idx]
    flat_true = flat_true[idx]

lp = np.log1p(flat_pred)
lt = np.log1p(flat_true)
plt.figure(figsize=(5, 5))
plt.scatter(lp, lt, alpha=0.2, s=5)
xy_min_l = min(lp.min(), lt.min())
xy_max_l = max(lp.max(), lt.max())
plt.plot([xy_min_l, xy_max_l], [xy_min_l, xy_max_l], color="gray", linestyle="--", linewidth=1, label="bisector")
if lp.size > 1 and lt.size > 1 and np.std(lp) > 0 and np.std(lt) > 0:
    m, b = np.polyfit(lp, lt, 1)
    plt.plot([xy_min_l, xy_max_l], [m * xy_min_l + b, m * xy_max_l + b], color="orange", linewidth=1.5, label="regression")
    corr_log = float(np.corrcoef(lp, lt)[0, 1])
    print(f"Scatter correlation log1p (n={lp.size}): {corr_log:.4f}")
plt.xlabel("log1p Predicted counts")
plt.ylabel("log1p True counts")
plt.title("Pred vs true (log1p)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
Scatter correlation log1p (n=50000): 0.8297
Saved log-scale scatter to isoform_model_fcnn/pred_vs_true_scatter_log.png

# INSERT SCATTERPLOT IMAGE

Spearman per-gene correlation (mean/median of observations) + boxplot

In [ ]:
gene_mean_corrs, gene_median_corrs = gene_spearman_per_sample(
    test_preds_counts,
    test_true_counts,
    transcript_gene_idx,
    gene_names,
)
valid_mean = ~np.isnan(gene_mean_corrs)
valid_median = ~np.isnan(gene_median_corrs)

if valid_mean.any():
    print(
        f"Per-gene isoform rank correlation (Spearman over observations): "
        f"mean of means {float(np.nanmean(gene_mean_corrs)):.4f} | "
        f"median of means {float(np.nanmedian(gene_mean_corrs)):.4f} over {valid_mean.sum()} genes"
    )
    plt.figure(figsize=(6, 4))
    plt.boxplot(gene_mean_corrs[valid_mean], vert=True, patch_artist=True)
    plt.ylabel("Spearman correlation (mean across observations)")
    plt.title("Per-gene isoform rank correlation (mean)")
    plt.tight_layout()
    plt.show()
else:
    print("Per-gene mean rank correlation: insufficient data.")

if valid_median.any():
    print(
        f"Per-gene isoform rank correlation (Spearman over observations): "
        f"mean of medians {float(np.nanmean(gene_median_corrs)):.4f} | "
        f"median of medians {float(np.nanmedian(gene_median_corrs)):.4f} over {valid_median.sum()} genes"
    )
    plt.figure(figsize=(6, 4))
    plt.boxplot(gene_median_corrs[valid_median], vert=True, patch_artist=True)
    plt.ylabel("Spearman correlation (median across observations)")
    plt.title("Per-gene isoform rank correlation (median)")
    plt.tight_layout()
    plt.show()
else:
    print("Per-gene median rank correlation: insufficient data.")

In [ ]:
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7702 | median of means 0.7904 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_fcnn/gene_rank_corr_mean_boxplot.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.8153 | median of medians 0.8308 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_fcnn/gene_rank_corr_median_boxplot.png

# INSERT BOXPLOT IMAGE


### Residual MLP

Predictions per split + log-MSE

In [ ]:
criterion = nn.MSELoss()
split_loaders = {"train": loaders.train_eval, "val": loaders.val, "test": loaders.test}
split_metrics = {}
for split_name, loader in split_loaders.items():
    mse = evaluate_loss(model, loader, device, criterion)
    preds_log, targets_log = inference(model, loader, device)
    preds_counts = np.expm1(preds_log)
    targets_counts = np.expm1(targets_log)
    split_metrics[split_name] = {
        "mse": mse,
        "pred_counts": preds_counts,
        "true_counts": targets_counts,
    }
    print(f"{split_name.capitalize()} | log-space MSE: {mse:.4f}")


In [ ]:
Train | log-space MSE: 0.4377
Val | log-space MSE: 0.5243
Test | log-space MSE: 0.5235

Correlation per isoform counts + boxplot

In [ ]:
test_preds_counts = split_metrics["test"]["pred_counts"]
test_true_counts  = split_metrics["test"]["true_counts"]

iso_corr = isoform_correlations(test_preds_counts, test_true_counts)
valid_corr = iso_corr[~np.isnan(iso_corr)]
if valid_corr.size == 0:
    print("Isoform correlation: no isoforms with variance; skipping boxplot.")
else:
    corr_mean = float(valid_corr.mean())
    print(f"Isoform correlation: mean Pearson {corr_mean:.4f} over {valid_corr.size}/{iso_corr.size} isoforms")
    plt.figure(figsize=(6, 4))
    plt.boxplot(valid_corr, vert=True, patch_artist=True)
    plt.ylabel("Pearson correlation")
    plt.title("Isoform prediction correlation (test)")
    plt.tight_layout()
    plt.show()

In [ ]:
Isoform correlation: mean Pearson 0.6327 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_residual/isoform_correlation_boxplot.png

# INSERT BOXPLOT IMAGE

Scatterplot log1p pred VS true + regression line

In [ ]:
flat_pred = test_preds_counts.ravel()
flat_true = test_true_counts.ravel()
max_points = 50000
if flat_pred.size > max_points:
    rng = np.random.default_rng(42)
    idx = rng.choice(flat_pred.size, size=max_points, replace=False)
    flat_pred = flat_pred[idx]
    flat_true = flat_true[idx]

lp = np.log1p(flat_pred)
lt = np.log1p(flat_true)
plt.figure(figsize=(5, 5))
plt.scatter(lp, lt, alpha=0.2, s=5)
xy_min_l = min(lp.min(), lt.min())
xy_max_l = max(lp.max(), lt.max())
plt.plot([xy_min_l, xy_max_l], [xy_min_l, xy_max_l], color="gray", linestyle="--", linewidth=1, label="bisector")
if lp.size > 1 and lt.size > 1 and np.std(lp) > 0 and np.std(lt) > 0:
    m, b = np.polyfit(lp, lt, 1)
    plt.plot([xy_min_l, xy_max_l], [m * xy_min_l + b, m * xy_max_l + b], color="orange", linewidth=1.5, label="regression")
    corr_log = float(np.corrcoef(lp, lt)[0, 1])
    print(f"Scatter correlation log1p (n={lp.size}): {corr_log:.4f}")
plt.xlabel("log1p Predicted counts")
plt.ylabel("log1p True counts")
plt.title("Pred vs true (log1p)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
Scatter correlation log1p (n=50000): 0.8502
Saved log-scale scatter to isoform_model_residual/pred_vs_true_scatter_residual_log.png

# INSERT SCATTER PLOT IMAGE

Spearman per-gene correlation (mean/median of observations) + boxplot

In [ ]:
gene_mean_corrs, gene_median_corrs = gene_spearman_per_sample(
    test_preds_counts,
    test_true_counts,
    transcript_gene_idx,
    gene_names,
)
valid_mean = ~np.isnan(gene_mean_corrs)
valid_median = ~np.isnan(gene_median_corrs)

if valid_mean.any():
    print(
        f"Per-gene isoform rank correlation (Spearman over observations): "
        f"mean of means {float(np.nanmean(gene_mean_corrs)):.4f} | "
        f"median of means {float(np.nanmedian(gene_mean_corrs)):.4f} over {valid_mean.sum()} genes"
    )
    plt.figure(figsize=(6, 4))
    plt.boxplot(gene_mean_corrs[valid_mean], vert=True, patch_artist=True)
    plt.ylabel("Spearman correlation (mean across observations)")
    plt.title("Per-gene isoform rank correlation (mean)")
    plt.tight_layout()
    plt.show()
else:
    print("Per-gene mean rank correlation: insufficient data.")

if valid_median.any():
    print(
        f"Per-gene isoform rank correlation (Spearman over observations): "
        f"mean of medians {float(np.nanmean(gene_median_corrs)):.4f} | "
        f"median of medians {float(np.nanmedian(gene_median_corrs)):.4f} over {valid_median.sum()} genes"
    )
    plt.figure(figsize=(6, 4))
    plt.boxplot(gene_median_corrs[valid_median], vert=True, patch_artist=True)
    plt.ylabel("Spearman correlation (median across observations)")
    plt.title("Per-gene isoform rank correlation (median)")
    plt.tight_layout()
    plt.show()
else:
    print("Per-gene median rank correlation: insufficient data.")

In [ ]:
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7895 | median of means 0.8091 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_residual/gene_rank_corr_mean_boxplot_residual.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.8328 | median of medians 0.8498 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_residual/gene_rank_corr_median_boxplot_residual.png

# INSERT BOXPLOT IMAGE


For the following models we are just going to report the results obtained. The code is just the union of the PCA/VAE code and MLP/MLP-res

## VAE + MLP

In [ ]:
Loading AnnData...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Data stats after scaling: mean -0.000, std 1.000
Using whole dataset: train 9941, val 6958, test 2983
Epoch 001 | train 0.7141 (rec 0.7006, kl 0.5386) | val 0.6317 (rec 0.6167, kl 0.5965) | beta 0.025
Epoch 002 | train 0.6285 (rec 0.6052, kl 0.4669) | val 0.6114 (rec 0.5874, kl 0.4790) | beta 0.050
Epoch 003 | train 0.6053 (rec 0.5758, kl 0.3931) | val 0.5929 (rec 0.5638, kl 0.3875) | beta 0.075
Epoch 004 | train 0.5948 (rec 0.5612, kl 0.3364) | val 0.5818 (rec 0.5498, kl 0.3206) | beta 0.100
Epoch 005 | train 0.5856 (rec 0.5492, kl 0.2917) | val 0.5811 (rec 0.5470, kl 0.2734) | beta 0.125
Epoch 006 | train 0.5782 (rec 0.5394, kl 0.2592) | val 0.5720 (rec 0.5332, kl 0.2584) | beta 0.150
Epoch 007 | train 0.5722 (rec 0.5311, kl 0.2352) | val 0.5720 (rec 0.5333, kl 0.2209) | beta 0.175
Epoch 008 | train 0.5672 (rec 0.5241, kl 0.2156) | val 0.5606 (rec 0.5196, kl 0.2048) | beta 0.200
Epoch 009 | train 0.5639 (rec 0.5186, kl 0.2017) | val 0.5731 (rec 0.5272, kl 0.2038) | beta 0.225
Epoch 010 | train 0.5644 (rec 0.5167, kl 0.1910) | val 0.5575 (rec 0.5112, kl 0.1850) | beta 0.250
Epoch 011 | train 0.5627 (rec 0.5130, kl 0.1806) | val 0.5631 (rec 0.5139, kl 0.1792) | beta 0.275
Epoch 012 | train 0.5640 (rec 0.5119, kl 0.1736) | val 0.5606 (rec 0.5107, kl 0.1664) | beta 0.300
Epoch 013 | train 0.5621 (rec 0.5085, kl 0.1648) | val 0.5653 (rec 0.5138, kl 0.1586) | beta 0.325
Epoch 014 | train 0.5648 (rec 0.5090, kl 0.1594) | val 0.5623 (rec 0.5109, kl 0.1467) | beta 0.350
Epoch 015 | train 0.5642 (rec 0.5066, kl 0.1534) | val 0.5636 (rec 0.5075, kl 0.1497) | beta 0.375
Epoch 016 | train 0.5648 (rec 0.5052, kl 0.1489) | val 0.5688 (rec 0.5139, kl 0.1372) | beta 0.400
Epoch 017 | train 0.5661 (rec 0.5044, kl 0.1451) | val 0.5651 (rec 0.5060, kl 0.1390) | beta 0.425
Epoch 018 | train 0.5696 (rec 0.5060, kl 0.1414) | val 0.5701 (rec 0.5084, kl 0.1371) | beta 0.450
Epoch 019 | train 0.5702 (rec 0.5054, kl 0.1365) | val 0.5713 (rec 0.5101, kl 0.1289) | beta 0.475
Epoch 020 | train 0.5730 (rec 0.5064, kl 0.1331) | val 0.5752 (rec 0.5096, kl 0.1312) | beta 0.500
Epoch 021 | train 0.5725 (rec 0.5056, kl 0.1338) | val 0.5726 (rec 0.5070, kl 0.1313) | beta 0.500
Epoch 022 | train 0.5684 (rec 0.5023, kl 0.1323) | val 0.5671 (rec 0.5024, kl 0.1296) | beta 0.500
Epoch 023 | train 0.5678 (rec 0.5017, kl 0.1324) | val 0.5716 (rec 0.5059, kl 0.1314) | beta 0.500
Epoch 024 | train 0.5680 (rec 0.5014, kl 0.1332) | val 0.5718 (rec 0.5044, kl 0.1346) | beta 0.500
Epoch 025 | train 0.5695 (rec 0.5030, kl 0.1330) | val 0.5684 (rec 0.5039, kl 0.1290) | beta 0.500
Epoch 026 | train 0.5702 (rec 0.5037, kl 0.1330) | val 0.5656 (rec 0.5006, kl 0.1299) | beta 0.500
Epoch 027 | train 0.5659 (rec 0.4996, kl 0.1327) | val 0.5732 (rec 0.5072, kl 0.1320) | beta 0.500
Epoch 028 | train 0.5647 (rec 0.4983, kl 0.1327) | val 0.5652 (rec 0.5006, kl 0.1291) | beta 0.500
Epoch 029 | train 0.5602 (rec 0.4943, kl 0.1318) | val 0.5655 (rec 0.5011, kl 0.1287) | beta 0.500
Epoch 030 | train 0.5620 (rec 0.4960, kl 0.1321) | val 0.5690 (rec 0.5051, kl 0.1277) | beta 0.500
Epoch 031 | train 0.5637 (rec 0.4975, kl 0.1323) | val 0.5638 (rec 0.4974, kl 0.1329) | beta 0.500
Epoch 032 | train 0.5605 (rec 0.4945, kl 0.1320) | val 0.5616 (rec 0.4967, kl 0.1297) | beta 0.500
Epoch 033 | train 0.5600 (rec 0.4940, kl 0.1320) | val 0.5642 (rec 0.4985, kl 0.1313) | beta 0.500
Epoch 034 | train 0.5593 (rec 0.4931, kl 0.1323) | val 0.5666 (rec 0.5003, kl 0.1326) | beta 0.500
Epoch 035 | train 0.5606 (rec 0.4943, kl 0.1327) | val 0.5624 (rec 0.4981, kl 0.1286) | beta 0.500
Epoch 036 | train 0.5591 (rec 0.4929, kl 0.1324) | val 0.5633 (rec 0.4991, kl 0.1284) | beta 0.500
Epoch 037 | train 0.5573 (rec 0.4914, kl 0.1318) | val 0.5607 (rec 0.4968, kl 0.1278) | beta 0.500
Epoch 038 | train 0.5557 (rec 0.4898, kl 0.1317) | val 0.5616 (rec 0.4996, kl 0.1240) | beta 0.500
Epoch 039 | train 0.5553 (rec 0.4896, kl 0.1314) | val 0.5587 (rec 0.4928, kl 0.1319) | beta 0.500
Epoch 040 | train 0.5554 (rec 0.4894, kl 0.1320) | val 0.5558 (rec 0.4907, kl 0.1301) | beta 0.500
Epoch 041 | train 0.5540 (rec 0.4879, kl 0.1323) | val 0.5586 (rec 0.4942, kl 0.1287) | beta 0.500
Epoch 042 | train 0.5557 (rec 0.4898, kl 0.1318) | val 0.5566 (rec 0.4920, kl 0.1292) | beta 0.500
Epoch 043 | train 0.5557 (rec 0.4893, kl 0.1328) | val 0.5559 (rec 0.4897, kl 0.1324) | beta 0.500
Epoch 044 | train 0.5542 (rec 0.4881, kl 0.1320) | val 0.5577 (rec 0.4925, kl 0.1304) | beta 0.500
Epoch 045 | train 0.5522 (rec 0.4861, kl 0.1321) | val 0.5565 (rec 0.4924, kl 0.1283) | beta 0.500
Epoch 046 | train 0.5515 (rec 0.4857, kl 0.1315) | val 0.5594 (rec 0.4944, kl 0.1299) | beta 0.500
Epoch 047 | train 0.5531 (rec 0.4872, kl 0.1318) | val 0.5566 (rec 0.4910, kl 0.1312) | beta 0.500
Epoch 048 | train 0.5547 (rec 0.4883, kl 0.1328) | val 0.5552 (rec 0.4909, kl 0.1286) | beta 0.500
Epoch 049 | train 0.5507 (rec 0.4847, kl 0.1320) | val 0.5538 (rec 0.4892, kl 0.1292) | beta 0.500
Epoch 050 | train 0.5500 (rec 0.4841, kl 0.1319) | val 0.5556 (rec 0.4903, kl 0.1306) | beta 0.500
Epoch 051 | train 0.5507 (rec 0.4847, kl 0.1320) | val 0.5576 (rec 0.4909, kl 0.1335) | beta 0.500
Epoch 052 | train 0.5501 (rec 0.4843, kl 0.1315) | val 0.5533 (rec 0.4899, kl 0.1268) | beta 0.500
Epoch 053 | train 0.5488 (rec 0.4831, kl 0.1314) | val 0.5524 (rec 0.4883, kl 0.1282) | beta 0.500
Epoch 054 | train 0.5498 (rec 0.4837, kl 0.1322) | val 0.5592 (rec 0.4963, kl 0.1258) | beta 0.500
Epoch 055 | train 0.5509 (rec 0.4851, kl 0.1316) | val 0.5856 (rec 0.5207, kl 0.1297) | beta 0.500
Epoch 056 | train 0.5527 (rec 0.4864, kl 0.1326) | val 0.5507 (rec 0.4879, kl 0.1256) | beta 0.500
Epoch 057 | train 0.5480 (rec 0.4821, kl 0.1318) | val 0.5541 (rec 0.4901, kl 0.1280) | beta 0.500
Epoch 058 | train 0.5468 (rec 0.4810, kl 0.1316) | val 0.5547 (rec 0.4891, kl 0.1311) | beta 0.500
Epoch 059 | train 0.5471 (rec 0.4812, kl 0.1318) | val 0.5481 (rec 0.4844, kl 0.1274) | beta 0.500
Epoch 060 | train 0.5455 (rec 0.4797, kl 0.1316) | val 0.5519 (rec 0.4874, kl 0.1289) | beta 0.500
Epoch 061 | train 0.5471 (rec 0.4814, kl 0.1316) | val 0.5553 (rec 0.4891, kl 0.1324) | beta 0.500
Epoch 062 | train 0.5451 (rec 0.4794, kl 0.1314) | val 0.5503 (rec 0.4864, kl 0.1278) | beta 0.500
Epoch 063 | train 0.5454 (rec 0.4797, kl 0.1315) | val 0.5520 (rec 0.4880, kl 0.1279) | beta 0.500
Epoch 064 | train 0.5455 (rec 0.4797, kl 0.1316) | val 0.5503 (rec 0.4848, kl 0.1311) | beta 0.500
Epoch 065 | train 0.5476 (rec 0.4815, kl 0.1321) | val 0.5513 (rec 0.4875, kl 0.1275) | beta 0.500
Epoch 066 | train 0.5489 (rec 0.4825, kl 0.1328) | val 0.5542 (rec 0.4893, kl 0.1299) | beta 0.500
Epoch 067 | train 0.5462 (rec 0.4805, kl 0.1314) | val 0.5505 (rec 0.4862, kl 0.1284) | beta 0.500
Epoch 068 | train 0.5437 (rec 0.4778, kl 0.1319) | val 0.5493 (rec 0.4853, kl 0.1281) | beta 0.500
Epoch 069 | train 0.5430 (rec 0.4770, kl 0.1321) | val 0.5557 (rec 0.4925, kl 0.1265) | beta 0.500
Epoch 070 | train 0.5455 (rec 0.4798, kl 0.1316) | val 0.5470 (rec 0.4824, kl 0.1291) | beta 0.500
Epoch 071 | train 0.5424 (rec 0.4765, kl 0.1317) | val 0.5502 (rec 0.4861, kl 0.1283) | beta 0.500
Epoch 072 | train 0.5417 (rec 0.4759, kl 0.1317) | val 0.5469 (rec 0.4819, kl 0.1299) | beta 0.500
Epoch 073 | train 0.5439 (rec 0.4779, kl 0.1319) | val 0.5492 (rec 0.4843, kl 0.1298) | beta 0.500
Epoch 074 | train 0.5432 (rec 0.4776, kl 0.1311) | val 0.5469 (rec 0.4837, kl 0.1263) | beta 0.500
Epoch 075 | train 0.5422 (rec 0.4763, kl 0.1319) | val 0.5457 (rec 0.4822, kl 0.1268) | beta 0.500
Epoch 076 | train 0.5421 (rec 0.4763, kl 0.1316) | val 0.5471 (rec 0.4818, kl 0.1307) | beta 0.500
Epoch 077 | train 0.5400 (rec 0.4747, kl 0.1307) | val 0.5454 (rec 0.4815, kl 0.1278) | beta 0.500
Epoch 078 | train 0.5407 (rec 0.4749, kl 0.1316) | val 0.5464 (rec 0.4821, kl 0.1286) | beta 0.500
Epoch 079 | train 0.5414 (rec 0.4754, kl 0.1320) | val 0.5478 (rec 0.4827, kl 0.1301) | beta 0.500
Epoch 080 | train 0.5409 (rec 0.4748, kl 0.1323) | val 0.5492 (rec 0.4844, kl 0.1295) | beta 0.500
Epoch 081 | train 0.5408 (rec 0.4751, kl 0.1313) | val 0.5508 (rec 0.4857, kl 0.1301) | beta 0.500
Epoch 082 | train 0.5431 (rec 0.4770, kl 0.1321) | val 0.5456 (rec 0.4809, kl 0.1295) | beta 0.500
Epoch 083 | train 0.5399 (rec 0.4740, kl 0.1317) | val 0.5452 (rec 0.4802, kl 0.1302) | beta 0.500
Epoch 084 | train 0.5401 (rec 0.4744, kl 0.1314) | val 0.5443 (rec 0.4810, kl 0.1266) | beta 0.500
Epoch 085 | train 0.5388 (rec 0.4733, kl 0.1310) | val 0.5447 (rec 0.4800, kl 0.1292) | beta 0.500
Epoch 086 | train 0.5386 (rec 0.4730, kl 0.1311) | val 0.5452 (rec 0.4812, kl 0.1279) | beta 0.500
Epoch 087 | train 0.5379 (rec 0.4723, kl 0.1314) | val 0.5442 (rec 0.4800, kl 0.1284) | beta 0.500
Epoch 088 | train 0.5386 (rec 0.4731, kl 0.1311) | val 0.5449 (rec 0.4801, kl 0.1296) | beta 0.500
Epoch 089 | train 0.5391 (rec 0.4735, kl 0.1312) | val 0.5451 (rec 0.4802, kl 0.1298) | beta 0.500
Epoch 090 | train 0.5387 (rec 0.4730, kl 0.1316) | val 0.5450 (rec 0.4807, kl 0.1287) | beta 0.500
Epoch 091 | train 0.5399 (rec 0.4741, kl 0.1317) | val 0.5428 (rec 0.4787, kl 0.1281) | beta 0.500
Epoch 092 | train 0.5393 (rec 0.4736, kl 0.1315) | val 0.5464 (rec 0.4821, kl 0.1286) | beta 0.500
Epoch 093 | train 0.5373 (rec 0.4716, kl 0.1315) | val 0.5418 (rec 0.4772, kl 0.1292) | beta 0.500
Epoch 094 | train 0.5390 (rec 0.4733, kl 0.1313) | val 0.5434 (rec 0.4782, kl 0.1303) | beta 0.500
Epoch 095 | train 0.5373 (rec 0.4716, kl 0.1313) | val 0.5458 (rec 0.4842, kl 0.1232) | beta 0.500
Epoch 096 | train 0.5395 (rec 0.4738, kl 0.1314) | val 0.5456 (rec 0.4815, kl 0.1281) | beta 0.500
Epoch 097 | train 0.5366 (rec 0.4710, kl 0.1312) | val 0.5424 (rec 0.4765, kl 0.1317) | beta 0.500
Epoch 098 | train 0.5376 (rec 0.4720, kl 0.1311) | val 0.5410 (rec 0.4772, kl 0.1276) | beta 0.500
Epoch 099 | train 0.5366 (rec 0.4708, kl 0.1316) | val 0.5433 (rec 0.4773, kl 0.1319) | beta 0.500
Epoch 100 | train 0.5370 (rec 0.4711, kl 0.1316) | val 0.5412 (rec 0.4768, kl 0.1288) | beta 0.500
Epoch 101 | train 0.5361 (rec 0.4705, kl 0.1313) | val 0.5429 (rec 0.4793, kl 0.1271) | beta 0.500
Epoch 102 | train 0.5364 (rec 0.4707, kl 0.1314) | val 0.5460 (rec 0.4809, kl 0.1302) | beta 0.500
Epoch 103 | train 0.5366 (rec 0.4708, kl 0.1316) | val 0.5448 (rec 0.4797, kl 0.1302) | beta 0.500
Epoch 104 | train 0.5360 (rec 0.4701, kl 0.1319) | val 0.5421 (rec 0.4770, kl 0.1302) | beta 0.500
Epoch 105 | train 0.5356 (rec 0.4698, kl 0.1317) | val 0.5418 (rec 0.4776, kl 0.1284) | beta 0.500
Epoch 106 | train 0.5361 (rec 0.4703, kl 0.1315) | val 0.5429 (rec 0.4786, kl 0.1285) | beta 0.500
Epoch 107 | train 0.5356 (rec 0.4701, kl 0.1308) | val 0.5397 (rec 0.4753, kl 0.1289) | beta 0.500
Epoch 108 | train 0.5343 (rec 0.4686, kl 0.1314) | val 0.5411 (rec 0.4774, kl 0.1274) | beta 0.500
Epoch 109 | train 0.5361 (rec 0.4703, kl 0.1316) | val 0.5429 (rec 0.4778, kl 0.1301) | beta 0.500
Epoch 110 | train 0.5342 (rec 0.4688, kl 0.1310) | val 0.5419 (rec 0.4775, kl 0.1289) | beta 0.500
Epoch 111 | train 0.5370 (rec 0.4714, kl 0.1311) | val 0.5489 (rec 0.4840, kl 0.1298) | beta 0.500
Epoch 112 | train 0.5340 (rec 0.4686, kl 0.1310) | val 0.5406 (rec 0.4765, kl 0.1283) | beta 0.500
Epoch 113 | train 0.5342 (rec 0.4684, kl 0.1316) | val 0.5404 (rec 0.4755, kl 0.1298) | beta 0.500
Epoch 114 | train 0.5339 (rec 0.4682, kl 0.1314) | val 0.5427 (rec 0.4787, kl 0.1280) | beta 0.500
Epoch 115 | train 0.5348 (rec 0.4690, kl 0.1317) | val 0.5405 (rec 0.4765, kl 0.1279) | beta 0.500
Epoch 116 | train 0.5345 (rec 0.4686, kl 0.1316) | val 0.5399 (rec 0.4762, kl 0.1275) | beta 0.500
Epoch 117 | train 0.5333 (rec 0.4675, kl 0.1315) | val 0.5416 (rec 0.4780, kl 0.1273) | beta 0.500
Epoch 118 | train 0.5328 (rec 0.4673, kl 0.1310) | val 0.5399 (rec 0.4762, kl 0.1274) | beta 0.500
Epoch 119 | train 0.5330 (rec 0.4675, kl 0.1312) | val 0.5408 (rec 0.4763, kl 0.1291) | beta 0.500
Epoch 120 | train 0.5337 (rec 0.4678, kl 0.1317) | val 0.5397 (rec 0.4762, kl 0.1271) | beta 0.500
Epoch 121 | train 0.5319 (rec 0.4662, kl 0.1316) | val 0.5394 (rec 0.4756, kl 0.1275) | beta 0.500
Epoch 122 | train 0.5326 (rec 0.4668, kl 0.1317) | val 0.5397 (rec 0.4753, kl 0.1289) | beta 0.500
Epoch 123 | train 0.5332 (rec 0.4674, kl 0.1316) | val 0.5412 (rec 0.4756, kl 0.1312) | beta 0.500
Epoch 124 | train 0.5326 (rec 0.4667, kl 0.1317) | val 0.5403 (rec 0.4747, kl 0.1313) | beta 0.500
Epoch 125 | train 0.5308 (rec 0.4654, kl 0.1307) | val 0.5434 (rec 0.4786, kl 0.1296) | beta 0.500
Epoch 126 | train 0.5329 (rec 0.4670, kl 0.1317) | val 0.5366 (rec 0.4731, kl 0.1270) | beta 0.500
Epoch 127 | train 0.5333 (rec 0.4675, kl 0.1316) | val 0.5427 (rec 0.4785, kl 0.1283) | beta 0.500
Epoch 128 | train 0.5360 (rec 0.4700, kl 0.1320) | val 0.5410 (rec 0.4765, kl 0.1291) | beta 0.500
Epoch 129 | train 0.5319 (rec 0.4661, kl 0.1317) | val 0.5367 (rec 0.4735, kl 0.1264) | beta 0.500
Epoch 130 | train 0.5306 (rec 0.4650, kl 0.1313) | val 0.5409 (rec 0.4765, kl 0.1288) | beta 0.500
Epoch 131 | train 0.5302 (rec 0.4648, kl 0.1308) | val 0.5417 (rec 0.4768, kl 0.1297) | beta 0.500
Epoch 132 | train 0.5309 (rec 0.4649, kl 0.1321) | val 0.5368 (rec 0.4727, kl 0.1280) | beta 0.500
Epoch 133 | train 0.5317 (rec 0.4656, kl 0.1321) | val 0.5373 (rec 0.4715, kl 0.1316) | beta 0.500
Epoch 134 | train 0.5303 (rec 0.4642, kl 0.1322) | val 0.5378 (rec 0.4728, kl 0.1299) | beta 0.500
Epoch 135 | train 0.5315 (rec 0.4656, kl 0.1319) | val 0.5391 (rec 0.4744, kl 0.1295) | beta 0.500
Epoch 136 | train 0.5298 (rec 0.4639, kl 0.1318) | val 0.5398 (rec 0.4754, kl 0.1287) | beta 0.500
Epoch 137 | train 0.5309 (rec 0.4649, kl 0.1320) | val 0.5397 (rec 0.4741, kl 0.1312) | beta 0.500
Epoch 138 | train 0.5308 (rec 0.4648, kl 0.1320) | val 0.5410 (rec 0.4777, kl 0.1268) | beta 0.500
Epoch 139 | train 0.5311 (rec 0.4651, kl 0.1320) | val 0.5397 (rec 0.4753, kl 0.1288) | beta 0.500
Epoch 140 | train 0.5311 (rec 0.4652, kl 0.1318) | val 0.5399 (rec 0.4749, kl 0.1300) | beta 0.500
Epoch 141 | train 0.5317 (rec 0.4657, kl 0.1319) | val 0.5393 (rec 0.4731, kl 0.1325) | beta 0.500
Epoch 142 | train 0.5306 (rec 0.4645, kl 0.1321) | val 0.5411 (rec 0.4760, kl 0.1302) | beta 0.500
Epoch 143 | train 0.5298 (rec 0.4639, kl 0.1317) | val 0.5357 (rec 0.4714, kl 0.1286) | beta 0.500
Epoch 144 | train 0.5278 (rec 0.4621, kl 0.1315) | val 0.5386 (rec 0.4741, kl 0.1292) | beta 0.500
Epoch 145 | train 0.5305 (rec 0.4646, kl 0.1318) | val 0.5381 (rec 0.4744, kl 0.1274) | beta 0.500
Epoch 146 | train 0.5304 (rec 0.4647, kl 0.1314) | val 0.5379 (rec 0.4733, kl 0.1292) | beta 0.500
Epoch 147 | train 0.5306 (rec 0.4647, kl 0.1318) | val 0.5405 (rec 0.4765, kl 0.1279) | beta 0.500
Epoch 148 | train 0.5316 (rec 0.4656, kl 0.1320) | val 0.5400 (rec 0.4754, kl 0.1292) | beta 0.500
Epoch 149 | train 0.5312 (rec 0.4653, kl 0.1318) | val 0.5386 (rec 0.4746, kl 0.1281) | beta 0.500
Epoch 150 | train 0.5280 (rec 0.4624, kl 0.1312) | val 0.5341 (rec 0.4696, kl 0.1289) | beta 0.500
Epoch 151 | train 0.5294 (rec 0.4635, kl 0.1316) | val 0.5371 (rec 0.4715, kl 0.1313) | beta 0.500
Epoch 152 | train 0.5285 (rec 0.4627, kl 0.1315) | val 0.5375 (rec 0.4731, kl 0.1290) | beta 0.500
Epoch 153 | train 0.5295 (rec 0.4634, kl 0.1320) | val 0.5363 (rec 0.4725, kl 0.1277) | beta 0.500
Epoch 154 | train 0.5291 (rec 0.4632, kl 0.1318) | val 0.5353 (rec 0.4703, kl 0.1300) | beta 0.500
Epoch 155 | train 0.5295 (rec 0.4635, kl 0.1319) | val 0.5381 (rec 0.4737, kl 0.1289) | beta 0.500
Epoch 156 | train 0.5278 (rec 0.4621, kl 0.1314) | val 0.5364 (rec 0.4718, kl 0.1292) | beta 0.500
Epoch 157 | train 0.5282 (rec 0.4624, kl 0.1317) | val 0.5357 (rec 0.4711, kl 0.1294) | beta 0.500
Epoch 158 | train 0.5280 (rec 0.4621, kl 0.1317) | val 0.5361 (rec 0.4716, kl 0.1289) | beta 0.500
Epoch 159 | train 0.5271 (rec 0.4615, kl 0.1312) | val 0.5362 (rec 0.4711, kl 0.1302) | beta 0.500
Epoch 160 | train 0.5292 (rec 0.4631, kl 0.1322) | val 0.5365 (rec 0.4724, kl 0.1281) | beta 0.500
Epoch 161 | train 0.5294 (rec 0.4629, kl 0.1331) | val 0.5352 (rec 0.4704, kl 0.1295) | beta 0.500
Epoch 162 | train 0.5270 (rec 0.4613, kl 0.1314) | val 0.5359 (rec 0.4709, kl 0.1300) | beta 0.500
Epoch 163 | train 0.5276 (rec 0.4616, kl 0.1321) | val 0.5354 (rec 0.4699, kl 0.1310) | beta 0.500
Epoch 164 | train 0.5270 (rec 0.4611, kl 0.1318) | val 0.5366 (rec 0.4723, kl 0.1286) | beta 0.500
Epoch 165 | train 0.5293 (rec 0.4630, kl 0.1325) | val 0.5379 (rec 0.4739, kl 0.1281) | beta 0.500
Epoch 166 | train 0.5305 (rec 0.4646, kl 0.1318) | val 0.5345 (rec 0.4706, kl 0.1278) | beta 0.500
Epoch 167 | train 0.5275 (rec 0.4615, kl 0.1320) | val 0.5371 (rec 0.4718, kl 0.1308) | beta 0.500
Epoch 168 | train 0.5269 (rec 0.4609, kl 0.1319) | val 0.5374 (rec 0.4724, kl 0.1299) | beta 0.500
Epoch 169 | train 0.5286 (rec 0.4624, kl 0.1323) | val 0.5341 (rec 0.4686, kl 0.1310) | beta 0.500
Epoch 170 | train 0.5257 (rec 0.4600, kl 0.1312) | val 0.5350 (rec 0.4703, kl 0.1294) | beta 0.500
Epoch 171 | train 0.5286 (rec 0.4626, kl 0.1320) | val 0.5373 (rec 0.4719, kl 0.1308) | beta 0.500
Epoch 172 | train 0.5276 (rec 0.4619, kl 0.1315) | val 0.5444 (rec 0.4795, kl 0.1297) | beta 0.500
Epoch 173 | train 0.5267 (rec 0.4610, kl 0.1314) | val 0.5364 (rec 0.4715, kl 0.1300) | beta 0.500
Epoch 174 | train 0.5258 (rec 0.4599, kl 0.1318) | val 0.5363 (rec 0.4720, kl 0.1287) | beta 0.500
Epoch 175 | train 0.5258 (rec 0.4600, kl 0.1318) | val 0.5339 (rec 0.4700, kl 0.1278) | beta 0.500
Epoch 176 | train 0.5261 (rec 0.4603, kl 0.1317) | val 0.5358 (rec 0.4714, kl 0.1288) | beta 0.500
Epoch 177 | train 0.5262 (rec 0.4603, kl 0.1318) | val 0.5371 (rec 0.4720, kl 0.1301) | beta 0.500
Epoch 178 | train 0.5265 (rec 0.4604, kl 0.1322) | val 0.5357 (rec 0.4713, kl 0.1288) | beta 0.500
Epoch 179 | train 0.5270 (rec 0.4611, kl 0.1319) | val 0.5370 (rec 0.4722, kl 0.1296) | beta 0.500
Epoch 180 | train 0.5288 (rec 0.4626, kl 0.1325) | val 0.5356 (rec 0.4720, kl 0.1272) | beta 0.500
Epoch 181 | train 0.5256 (rec 0.4599, kl 0.1313) | val 0.5393 (rec 0.4746, kl 0.1294) | beta 0.500
Epoch 182 | train 0.5255 (rec 0.4597, kl 0.1316) | val 0.5334 (rec 0.4692, kl 0.1283) | beta 0.500
Epoch 183 | train 0.5262 (rec 0.4605, kl 0.1315) | val 0.5338 (rec 0.4688, kl 0.1299) | beta 0.500
Epoch 184 | train 0.5276 (rec 0.4617, kl 0.1319) | val 0.5362 (rec 0.4711, kl 0.1301) | beta 0.500
Epoch 185 | train 0.5272 (rec 0.4612, kl 0.1320) | val 0.5368 (rec 0.4726, kl 0.1283) | beta 0.500
Epoch 186 | train 0.5255 (rec 0.4596, kl 0.1318) | val 0.5358 (rec 0.4711, kl 0.1293) | beta 0.500
Epoch 187 | train 0.5246 (rec 0.4586, kl 0.1321) | val 0.5335 (rec 0.4681, kl 0.1308) | beta 0.500
Epoch 188 | train 0.5252 (rec 0.4593, kl 0.1317) | val 0.5329 (rec 0.4681, kl 0.1296) | beta 0.500
Epoch 189 | train 0.5257 (rec 0.4596, kl 0.1321) | val 0.5340 (rec 0.4689, kl 0.1303) | beta 0.500
Epoch 190 | train 0.5252 (rec 0.4594, kl 0.1316) | val 0.5348 (rec 0.4711, kl 0.1273) | beta 0.500
Epoch 191 | train 0.5254 (rec 0.4595, kl 0.1319) | val 0.5346 (rec 0.4701, kl 0.1290) | beta 0.500
Epoch 192 | train 0.5244 (rec 0.4583, kl 0.1322) | val 0.5344 (rec 0.4695, kl 0.1298) | beta 0.500
Epoch 193 | train 0.5259 (rec 0.4598, kl 0.1322) | val 0.5340 (rec 0.4685, kl 0.1310) | beta 0.500
Epoch 194 | train 0.5233 (rec 0.4574, kl 0.1320) | val 0.5339 (rec 0.4709, kl 0.1261) | beta 0.500
Epoch 195 | train 0.5246 (rec 0.4588, kl 0.1316) | val 0.5329 (rec 0.4689, kl 0.1281) | beta 0.500
Epoch 196 | train 0.5248 (rec 0.4589, kl 0.1318) | val 0.5351 (rec 0.4708, kl 0.1287) | beta 0.500
Epoch 197 | train 0.5249 (rec 0.4590, kl 0.1319) | val 0.5345 (rec 0.4692, kl 0.1305) | beta 0.500
Epoch 198 | train 0.5253 (rec 0.4591, kl 0.1324) | val 0.5342 (rec 0.4686, kl 0.1312) | beta 0.500
Epoch 199 | train 0.5282 (rec 0.4621, kl 0.1323) | val 0.5346 (rec 0.4706, kl 0.1279) | beta 0.500
Epoch 200 | train 0.5242 (rec 0.4582, kl 0.1320) | val 0.5330 (rec 0.4673, kl 0.1314) | beta 0.500
Test loss 0.5292 (rec 0.4649, kl 0.1287)
[Predictor] Epoch 001 | train 4.4910 | val 1.5386 | lr 5.00e-04
[Predictor] Epoch 002 | train 1.2301 | val 1.1377 | lr 5.00e-04
[Predictor] Epoch 003 | train 1.1004 | val 1.0074 | lr 5.00e-04
[Predictor] Epoch 004 | train 1.0396 | val 0.9985 | lr 5.00e-04
[Predictor] Epoch 005 | train 1.0057 | val 0.9506 | lr 5.00e-04
[Predictor] Epoch 006 | train 0.9821 | val 0.9456 | lr 5.00e-04
[Predictor] Epoch 007 | train 0.9631 | val 0.9109 | lr 5.00e-04
[Predictor] Epoch 008 | train 0.9524 | val 0.8924 | lr 5.00e-04
[Predictor] Epoch 009 | train 0.9387 | val 0.8990 | lr 5.00e-04
[Predictor] Epoch 010 | train 0.9315 | val 0.8824 | lr 5.00e-04
[Predictor] Epoch 011 | train 0.9241 | val 0.8772 | lr 5.00e-04
[Predictor] Epoch 012 | train 0.9169 | val 0.8682 | lr 5.00e-04
[Predictor] Epoch 013 | train 0.9129 | val 0.8672 | lr 5.00e-04
[Predictor] Epoch 014 | train 0.9068 | val 0.8555 | lr 5.00e-04
[Predictor] Epoch 015 | train 0.9001 | val 0.8632 | lr 5.00e-04
[Predictor] Epoch 016 | train 0.8960 | val 0.8682 | lr 5.00e-04
[Predictor] Epoch 017 | train 0.8934 | val 0.8496 | lr 5.00e-04
[Predictor] Epoch 018 | train 0.8873 | val 0.8543 | lr 5.00e-04
[Predictor] Epoch 019 | train 0.8850 | val 0.8445 | lr 5.00e-04
[Predictor] Epoch 020 | train 0.8790 | val 0.8386 | lr 5.00e-04
[Predictor] Epoch 021 | train 0.8758 | val 0.8295 | lr 5.00e-04
[Predictor] Epoch 022 | train 0.8737 | val 0.8347 | lr 5.00e-04
[Predictor] Epoch 023 | train 0.8702 | val 0.8304 | lr 5.00e-04
[Predictor] Epoch 024 | train 0.8669 | val 0.8269 | lr 5.00e-04
[Predictor] Epoch 025 | train 0.8653 | val 0.8226 | lr 5.00e-04
[Predictor] Epoch 026 | train 0.8651 | val 0.8236 | lr 5.00e-04
[Predictor] Epoch 027 | train 0.8603 | val 0.8218 | lr 5.00e-04
[Predictor] Epoch 028 | train 0.8589 | val 0.8176 | lr 5.00e-04
[Predictor] Epoch 029 | train 0.8577 | val 0.8179 | lr 5.00e-04
[Predictor] Epoch 030 | train 0.8543 | val 0.8078 | lr 5.00e-04
[Predictor] Epoch 031 | train 0.8510 | val 0.8066 | lr 5.00e-04
[Predictor] Epoch 032 | train 0.8526 | val 0.8159 | lr 5.00e-04
[Predictor] Epoch 033 | train 0.8498 | val 0.8108 | lr 5.00e-04
[Predictor] Epoch 034 | train 0.8446 | val 0.8044 | lr 5.00e-04
[Predictor] Epoch 035 | train 0.8453 | val 0.8010 | lr 5.00e-04
[Predictor] Epoch 036 | train 0.8418 | val 0.7987 | lr 5.00e-04
[Predictor] Epoch 037 | train 0.8456 | val 0.7970 | lr 5.00e-04
[Predictor] Epoch 038 | train 0.8410 | val 0.8110 | lr 5.00e-04
[Predictor] Epoch 039 | train 0.8418 | val 0.8001 | lr 5.00e-04
[Predictor] Epoch 040 | train 0.8341 | val 0.8033 | lr 5.00e-04
[Predictor] Epoch 041 | train 0.8348 | val 0.7936 | lr 5.00e-04
[Predictor] Epoch 042 | train 0.8334 | val 0.7994 | lr 5.00e-04
[Predictor] Epoch 043 | train 0.8334 | val 0.8065 | lr 5.00e-04
[Predictor] Epoch 044 | train 0.8319 | val 0.7914 | lr 5.00e-04
[Predictor] Epoch 045 | train 0.8302 | val 0.7908 | lr 5.00e-04
[Predictor] Epoch 046 | train 0.8306 | val 0.7836 | lr 5.00e-04
[Predictor] Epoch 047 | train 0.8272 | val 0.7847 | lr 5.00e-04
[Predictor] Epoch 048 | train 0.8298 | val 0.7993 | lr 5.00e-04
[Predictor] Epoch 049 | train 0.8262 | val 0.7837 | lr 5.00e-04
[Predictor] Epoch 050 | train 0.8265 | val 0.7791 | lr 5.00e-04
[Predictor] Epoch 051 | train 0.8244 | val 0.7824 | lr 5.00e-04
[Predictor] Epoch 052 | train 0.8231 | val 0.7741 | lr 5.00e-04
[Predictor] Epoch 053 | train 0.8225 | val 0.7761 | lr 5.00e-04
[Predictor] Epoch 054 | train 0.8209 | val 0.7810 | lr 5.00e-04
[Predictor] Epoch 055 | train 0.8205 | val 0.7771 | lr 5.00e-04
[Predictor] Epoch 056 | train 0.8177 | val 0.7768 | lr 5.00e-04
[Predictor] Epoch 057 | train 0.8163 | val 0.7887 | lr 5.00e-04
[Predictor] Epoch 058 | train 0.8177 | val 0.7755 | lr 5.00e-04
[Predictor] Epoch 059 | train 0.8163 | val 0.7672 | lr 5.00e-04
[Predictor] Epoch 060 | train 0.8183 | val 0.7784 | lr 5.00e-04
[Predictor] Epoch 061 | train 0.8159 | val 0.7756 | lr 5.00e-04
[Predictor] Epoch 062 | train 0.8143 | val 0.7714 | lr 5.00e-04
[Predictor] Epoch 063 | train 0.8155 | val 0.7738 | lr 5.00e-04
[Predictor] Epoch 064 | train 0.8113 | val 0.7681 | lr 5.00e-04
[Predictor] Epoch 065 | train 0.8128 | val 0.7658 | lr 5.00e-04
[Predictor] Epoch 066 | train 0.8119 | val 0.7634 | lr 5.00e-04
[Predictor] Epoch 067 | train 0.8109 | val 0.7619 | lr 5.00e-04
[Predictor] Epoch 068 | train 0.8114 | val 0.7673 | lr 5.00e-04
[Predictor] Epoch 069 | train 0.8105 | val 0.7721 | lr 5.00e-04
[Predictor] Epoch 070 | train 0.8128 | val 0.7762 | lr 5.00e-04
[Predictor] Epoch 071 | train 0.8101 | val 0.7638 | lr 5.00e-04
[Predictor] Epoch 072 | train 0.8068 | val 0.7639 | lr 5.00e-04
[Predictor] Epoch 073 | train 0.8066 | val 0.7748 | lr 5.00e-04
[Predictor] Epoch 074 | train 0.8050 | val 0.7630 | lr 5.00e-04
[Predictor] Epoch 075 | train 0.8077 | val 0.7611 | lr 5.00e-04
[Predictor] Epoch 076 | train 0.8088 | val 0.7679 | lr 5.00e-04
[Predictor] Epoch 077 | train 0.8097 | val 0.7620 | lr 5.00e-04
[Predictor] Epoch 078 | train 0.8060 | val 0.7715 | lr 5.00e-04
[Predictor] Epoch 079 | train 0.8048 | val 0.7662 | lr 5.00e-04
[Predictor] Epoch 080 | train 0.8026 | val 0.7577 | lr 5.00e-04
[Predictor] Epoch 081 | train 0.8044 | val 0.7622 | lr 5.00e-04
[Predictor] Epoch 082 | train 0.8061 | val 0.7562 | lr 5.00e-04
[Predictor] Epoch 083 | train 0.8016 | val 0.7659 | lr 5.00e-04
[Predictor] Epoch 084 | train 0.8025 | val 0.7582 | lr 5.00e-04
[Predictor] Epoch 085 | train 0.8061 | val 0.7564 | lr 5.00e-04
[Predictor] Epoch 086 | train 0.8030 | val 0.7556 | lr 5.00e-04
[Predictor] Epoch 087 | train 0.8012 | val 0.7646 | lr 5.00e-04
[Predictor] Epoch 088 | train 0.8000 | val 0.7570 | lr 5.00e-04
[Predictor] Epoch 089 | train 0.8009 | val 0.7538 | lr 5.00e-04
[Predictor] Epoch 090 | train 0.8021 | val 0.7575 | lr 5.00e-04
[Predictor] Epoch 091 | train 0.7997 | val 0.7602 | lr 5.00e-04
[Predictor] Epoch 092 | train 0.8018 | val 0.7567 | lr 5.00e-04
[Predictor] Epoch 093 | train 0.7978 | val 0.7668 | lr 5.00e-04
[Predictor] Epoch 094 | train 0.8000 | val 0.7628 | lr 5.00e-04
[Predictor] Epoch 095 | train 0.8001 | val 0.7561 | lr 5.00e-04
[Predictor] Epoch 096 | train 0.7992 | val 0.7554 | lr 5.00e-04
[Predictor] Epoch 097 | train 0.7996 | val 0.7644 | lr 5.00e-04
[Predictor] Epoch 098 | train 0.7982 | val 0.7509 | lr 5.00e-04
[Predictor] Epoch 099 | train 0.7995 | val 0.7507 | lr 5.00e-04
[Predictor] Epoch 100 | train 0.7986 | val 0.7530 | lr 5.00e-04
[Predictor] Epoch 101 | train 0.7979 | val 0.7570 | lr 5.00e-04
[Predictor] Epoch 102 | train 0.7966 | val 0.7609 | lr 5.00e-04
[Predictor] Epoch 103 | train 0.7989 | val 0.7577 | lr 5.00e-04
[Predictor] Epoch 104 | train 0.8014 | val 0.7572 | lr 5.00e-04
[Predictor] Epoch 105 | train 0.7987 | val 0.7567 | lr 5.00e-04
[Predictor] Epoch 106 | train 0.7966 | val 0.7537 | lr 5.00e-04
[Predictor] Epoch 107 | train 0.7967 | val 0.7510 | lr 5.00e-04
[Predictor] Epoch 108 | train 0.7965 | val 0.7528 | lr 5.00e-04
[Predictor] Epoch 109 | train 0.7948 | val 0.7547 | lr 5.00e-04
[Predictor] Epoch 110 | train 0.7956 | val 0.7584 | lr 2.50e-04
[Predictor] Epoch 111 | train 0.7798 | val 0.7388 | lr 2.50e-04
[Predictor] Epoch 112 | train 0.7744 | val 0.7361 | lr 2.50e-04
[Predictor] Epoch 113 | train 0.7761 | val 0.7344 | lr 2.50e-04
[Predictor] Epoch 114 | train 0.7760 | val 0.7370 | lr 2.50e-04
[Predictor] Epoch 115 | train 0.7749 | val 0.7343 | lr 2.50e-04
[Predictor] Epoch 116 | train 0.7757 | val 0.7369 | lr 2.50e-04
[Predictor] Epoch 117 | train 0.7742 | val 0.7319 | lr 2.50e-04
[Predictor] Epoch 118 | train 0.7734 | val 0.7328 | lr 2.50e-04
[Predictor] Epoch 119 | train 0.7721 | val 0.7336 | lr 2.50e-04
[Predictor] Epoch 120 | train 0.7734 | val 0.7325 | lr 2.50e-04
[Predictor] Epoch 121 | train 0.7733 | val 0.7395 | lr 2.50e-04
[Predictor] Epoch 122 | train 0.7744 | val 0.7328 | lr 2.50e-04
[Predictor] Epoch 123 | train 0.7721 | val 0.7336 | lr 2.50e-04
[Predictor] Epoch 124 | train 0.7709 | val 0.7337 | lr 2.50e-04
[Predictor] Epoch 125 | train 0.7719 | val 0.7317 | lr 2.50e-04
[Predictor] Epoch 126 | train 0.7716 | val 0.7355 | lr 2.50e-04
[Predictor] Epoch 127 | train 0.7724 | val 0.7351 | lr 2.50e-04
[Predictor] Epoch 128 | train 0.7702 | val 0.7321 | lr 2.50e-04
[Predictor] Epoch 129 | train 0.7728 | val 0.7332 | lr 2.50e-04
[Predictor] Epoch 130 | train 0.7738 | val 0.7357 | lr 2.50e-04
[Predictor] Epoch 131 | train 0.7712 | val 0.7373 | lr 2.50e-04
[Predictor] Epoch 132 | train 0.7711 | val 0.7351 | lr 2.50e-04
[Predictor] Epoch 133 | train 0.7712 | val 0.7333 | lr 2.50e-04
[Predictor] Epoch 134 | train 0.7712 | val 0.7330 | lr 2.50e-04
[Predictor] Epoch 135 | train 0.7704 | val 0.7380 | lr 2.50e-04
[Predictor] Epoch 136 | train 0.7710 | val 0.7294 | lr 2.50e-04
[Predictor] Epoch 137 | train 0.7712 | val 0.7321 | lr 2.50e-04
[Predictor] Epoch 138 | train 0.7724 | val 0.7481 | lr 2.50e-04
[Predictor] Epoch 139 | train 0.7703 | val 0.7298 | lr 2.50e-04
[Predictor] Epoch 140 | train 0.7730 | val 0.7363 | lr 2.50e-04
[Predictor] Epoch 141 | train 0.7721 | val 0.7321 | lr 2.50e-04
[Predictor] Epoch 142 | train 0.7702 | val 0.7318 | lr 2.50e-04
[Predictor] Epoch 143 | train 0.7694 | val 0.7280 | lr 2.50e-04
[Predictor] Epoch 144 | train 0.7704 | val 0.7379 | lr 2.50e-04
[Predictor] Epoch 145 | train 0.7687 | val 0.7286 | lr 2.50e-04
[Predictor] Epoch 146 | train 0.7708 | val 0.7411 | lr 2.50e-04
[Predictor] Epoch 147 | train 0.7698 | val 0.7313 | lr 2.50e-04
[Predictor] Epoch 148 | train 0.7701 | val 0.7378 | lr 2.50e-04
[Predictor] Epoch 149 | train 0.7689 | val 0.7287 | lr 2.50e-04
[Predictor] Epoch 150 | train 0.7692 | val 0.7302 | lr 2.50e-04
[Predictor] Epoch 151 | train 0.7694 | val 0.7296 | lr 2.50e-04
[Predictor] Epoch 152 | train 0.7707 | val 0.7325 | lr 2.50e-04
[Predictor] Epoch 153 | train 0.7682 | val 0.7295 | lr 2.50e-04
[Predictor] Epoch 154 | train 0.7683 | val 0.7312 | lr 1.25e-04
[Predictor] Epoch 155 | train 0.7604 | val 0.7240 | lr 1.25e-04
[Predictor] Epoch 156 | train 0.7565 | val 0.7216 | lr 1.25e-04
[Predictor] Epoch 157 | train 0.7561 | val 0.7204 | lr 1.25e-04
[Predictor] Epoch 158 | train 0.7552 | val 0.7208 | lr 1.25e-04
[Predictor] Epoch 159 | train 0.7546 | val 0.7187 | lr 1.25e-04
[Predictor] Epoch 160 | train 0.7562 | val 0.7237 | lr 1.25e-04
[Predictor] Epoch 161 | train 0.7548 | val 0.7218 | lr 1.25e-04
[Predictor] Epoch 162 | train 0.7548 | val 0.7210 | lr 1.25e-04
[Predictor] Epoch 163 | train 0.7552 | val 0.7187 | lr 1.25e-04
[Predictor] Epoch 164 | train 0.7571 | val 0.7238 | lr 1.25e-04
[Predictor] Epoch 165 | train 0.7550 | val 0.7257 | lr 1.25e-04
[Predictor] Epoch 166 | train 0.7541 | val 0.7210 | lr 1.25e-04
[Predictor] Epoch 167 | train 0.7530 | val 0.7178 | lr 1.25e-04
[Predictor] Epoch 168 | train 0.7537 | val 0.7189 | lr 1.25e-04
[Predictor] Epoch 169 | train 0.7530 | val 0.7189 | lr 1.25e-04
[Predictor] Epoch 170 | train 0.7543 | val 0.7186 | lr 1.25e-04
[Predictor] Epoch 171 | train 0.7536 | val 0.7191 | lr 1.25e-04
[Predictor] Epoch 172 | train 0.7543 | val 0.7191 | lr 1.25e-04
[Predictor] Epoch 173 | train 0.7514 | val 0.7194 | lr 1.25e-04
[Predictor] Epoch 174 | train 0.7536 | val 0.7196 | lr 1.25e-04
[Predictor] Epoch 175 | train 0.7523 | val 0.7245 | lr 1.25e-04
[Predictor] Epoch 176 | train 0.7525 | val 0.7170 | lr 1.25e-04
[Predictor] Epoch 177 | train 0.7536 | val 0.7167 | lr 1.25e-04
[Predictor] Epoch 178 | train 0.7543 | val 0.7179 | lr 1.25e-04
[Predictor] Epoch 179 | train 0.7527 | val 0.7215 | lr 1.25e-04
[Predictor] Epoch 180 | train 0.7530 | val 0.7213 | lr 1.25e-04
[Predictor] Epoch 181 | train 0.7522 | val 0.7194 | lr 1.25e-04
[Predictor] Epoch 182 | train 0.7519 | val 0.7180 | lr 1.25e-04
[Predictor] Epoch 183 | train 0.7525 | val 0.7196 | lr 1.25e-04
[Predictor] Epoch 184 | train 0.7521 | val 0.7208 | lr 1.25e-04
[Predictor] Epoch 185 | train 0.7531 | val 0.7186 | lr 1.25e-04
[Predictor] Epoch 186 | train 0.7518 | val 0.7150 | lr 1.25e-04
[Predictor] Epoch 187 | train 0.7510 | val 0.7168 | lr 1.25e-04
[Predictor] Epoch 188 | train 0.7526 | val 0.7159 | lr 1.25e-04
[Predictor] Epoch 189 | train 0.7517 | val 0.7196 | lr 1.25e-04
[Predictor] Epoch 190 | train 0.7512 | val 0.7184 | lr 1.25e-04
[Predictor] Epoch 191 | train 0.7525 | val 0.7195 | lr 1.25e-04
[Predictor] Epoch 192 | train 0.7528 | val 0.7237 | lr 1.25e-04
[Predictor] Epoch 193 | train 0.7507 | val 0.7196 | lr 1.25e-04
[Predictor] Epoch 194 | train 0.7512 | val 0.7185 | lr 1.25e-04
[Predictor] Epoch 195 | train 0.7516 | val 0.7177 | lr 1.25e-04
[Predictor] Epoch 196 | train 0.7509 | val 0.7188 | lr 1.25e-04
[Predictor] Epoch 197 | train 0.7519 | val 0.7188 | lr 6.25e-05
[Predictor] Epoch 198 | train 0.7466 | val 0.7160 | lr 6.25e-05
[Predictor] Epoch 199 | train 0.7446 | val 0.7151 | lr 6.25e-05
[Predictor] Epoch 200 | train 0.7449 | val 0.7141 | lr 6.25e-05
Predictor MSE | train 0.6853 | val 0.7141 | test 0.7121
Isoform correlation: mean Pearson 0.5430, median 0.5598 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_vae/isoform_correlation_boxplot_vae.png
Saved log-scale scatter to isoform_model_vae/pred_vs_true_scatter_log.png
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7493 | median of means 0.7688 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_vae/gene_rank_corr_mean_boxplot_vae.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.7945 | median of medians 0.8119 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_vae/gene_rank_corr_median_boxplot_vae.png


## VAE + res-MLP

In [ ]:
Loading AnnData...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Data stats after scaling: mean -0.000, std 1.000
Using whole dataset: train 9941, val 6958, test 2983
Epoch 001 | train 0.7154 (rec 0.7050, kl 0.4154) | val 0.6501 (rec 0.6380, kl 0.4852) | beta 0.025
Epoch 002 | train 0.6255 (rec 0.6077, kl 0.3559) | val 0.6069 (rec 0.5899, kl 0.3383) | beta 0.050
Epoch 003 | train 0.6027 (rec 0.5802, kl 0.3001) | val 0.5934 (rec 0.5687, kl 0.3293) | beta 0.075
Epoch 004 | train 0.5923 (rec 0.5659, kl 0.2637) | val 0.5844 (rec 0.5591, kl 0.2529) | beta 0.100
Epoch 005 | train 0.5788 (rec 0.5502, kl 0.2286) | val 0.5730 (rec 0.5451, kl 0.2225) | beta 0.125
Epoch 006 | train 0.5683 (rec 0.5380, kl 0.2019) | val 0.5612 (rec 0.5328, kl 0.1890) | beta 0.150
Epoch 007 | train 0.5642 (rec 0.5318, kl 0.1849) | val 0.5561 (rec 0.5253, kl 0.1760) | beta 0.175
Epoch 008 | train 0.5573 (rec 0.5240, kl 0.1668) | val 0.5506 (rec 0.5192, kl 0.1567) | beta 0.200
Epoch 009 | train 0.5548 (rec 0.5196, kl 0.1563) | val 0.5523 (rec 0.5176, kl 0.1542) | beta 0.225
Epoch 010 | train 0.5461 (rec 0.5103, kl 0.1435) | val 0.5470 (rec 0.5133, kl 0.1346) | beta 0.250
Epoch 011 | train 0.5456 (rec 0.5083, kl 0.1353) | val 0.5422 (rec 0.5064, kl 0.1304) | beta 0.275
Epoch 012 | train 0.5432 (rec 0.5053, kl 0.1263) | val 0.5505 (rec 0.5132, kl 0.1245) | beta 0.300
Epoch 013 | train 0.5396 (rec 0.5008, kl 0.1195) | val 0.5406 (rec 0.5037, kl 0.1136) | beta 0.325
Epoch 014 | train 0.5392 (rec 0.4992, kl 0.1145) | val 0.5404 (rec 0.5021, kl 0.1094) | beta 0.350
Epoch 015 | train 0.5351 (rec 0.4941, kl 0.1094) | val 0.5366 (rec 0.4975, kl 0.1042) | beta 0.375
Epoch 016 | train 0.5363 (rec 0.4939, kl 0.1058) | val 0.5365 (rec 0.4967, kl 0.0996) | beta 0.400
Epoch 017 | train 0.5347 (rec 0.4912, kl 0.1024) | val 0.5504 (rec 0.5073, kl 0.1013) | beta 0.425
Epoch 018 | train 0.5390 (rec 0.4941, kl 0.0998) | val 0.5497 (rec 0.5076, kl 0.0933) | beta 0.450
Epoch 019 | train 0.5371 (rec 0.4908, kl 0.0975) | val 0.5364 (rec 0.4926, kl 0.0922) | beta 0.475
Epoch 020 | train 0.5373 (rec 0.4902, kl 0.0941) | val 0.5368 (rec 0.4915, kl 0.0906) | beta 0.500
Epoch 021 | train 0.5346 (rec 0.4877, kl 0.0939) | val 0.5348 (rec 0.4899, kl 0.0899) | beta 0.500
Epoch 022 | train 0.5354 (rec 0.4885, kl 0.0938) | val 0.5305 (rec 0.4851, kl 0.0907) | beta 0.500
Epoch 023 | train 0.5299 (rec 0.4836, kl 0.0925) | val 0.5321 (rec 0.4872, kl 0.0900) | beta 0.500
Epoch 024 | train 0.5295 (rec 0.4832, kl 0.0925) | val 0.5396 (rec 0.4947, kl 0.0898) | beta 0.500
Epoch 025 | train 0.5290 (rec 0.4826, kl 0.0927) | val 0.5495 (rec 0.5038, kl 0.0914) | beta 0.500
Epoch 026 | train 0.5271 (rec 0.4805, kl 0.0931) | val 0.5263 (rec 0.4819, kl 0.0887) | beta 0.500
Epoch 027 | train 0.5251 (rec 0.4788, kl 0.0926) | val 0.5303 (rec 0.4856, kl 0.0895) | beta 0.500
Epoch 028 | train 0.5241 (rec 0.4779, kl 0.0924) | val 0.5274 (rec 0.4833, kl 0.0884) | beta 0.500
Epoch 029 | train 0.5245 (rec 0.4784, kl 0.0922) | val 0.5264 (rec 0.4805, kl 0.0918) | beta 0.500
Epoch 030 | train 0.5228 (rec 0.4764, kl 0.0928) | val 0.5236 (rec 0.4793, kl 0.0884) | beta 0.500
Epoch 031 | train 0.5235 (rec 0.4772, kl 0.0926) | val 0.5266 (rec 0.4814, kl 0.0904) | beta 0.500
Epoch 032 | train 0.5214 (rec 0.4751, kl 0.0926) | val 0.5199 (rec 0.4755, kl 0.0887) | beta 0.500
Epoch 033 | train 0.5201 (rec 0.4738, kl 0.0925) | val 0.5230 (rec 0.4785, kl 0.0889) | beta 0.500
Epoch 034 | train 0.5198 (rec 0.4734, kl 0.0928) | val 0.5278 (rec 0.4827, kl 0.0902) | beta 0.500
Epoch 035 | train 0.5207 (rec 0.4743, kl 0.0928) | val 0.5389 (rec 0.4903, kl 0.0971) | beta 0.500
Epoch 036 | train 0.5194 (rec 0.4732, kl 0.0926) | val 0.5187 (rec 0.4741, kl 0.0892) | beta 0.500
Epoch 037 | train 0.5178 (rec 0.4716, kl 0.0924) | val 0.5230 (rec 0.4771, kl 0.0919) | beta 0.500
Epoch 038 | train 0.5172 (rec 0.4709, kl 0.0925) | val 0.5229 (rec 0.4769, kl 0.0921) | beta 0.500
Epoch 039 | train 0.5155 (rec 0.4692, kl 0.0926) | val 0.5161 (rec 0.4707, kl 0.0908) | beta 0.500
Epoch 040 | train 0.5147 (rec 0.4685, kl 0.0923) | val 0.5208 (rec 0.4763, kl 0.0890) | beta 0.500
Epoch 041 | train 0.5152 (rec 0.4690, kl 0.0924) | val 0.5166 (rec 0.4704, kl 0.0923) | beta 0.500
Epoch 042 | train 0.5116 (rec 0.4655, kl 0.0922) | val 0.5175 (rec 0.4736, kl 0.0877) | beta 0.500
Epoch 043 | train 0.5104 (rec 0.4642, kl 0.0924) | val 0.5161 (rec 0.4700, kl 0.0922) | beta 0.500
Epoch 044 | train 0.5108 (rec 0.4645, kl 0.0926) | val 0.5156 (rec 0.4707, kl 0.0899) | beta 0.500
Epoch 045 | train 0.5123 (rec 0.4660, kl 0.0926) | val 0.5156 (rec 0.4705, kl 0.0903) | beta 0.500
Epoch 046 | train 0.5149 (rec 0.4682, kl 0.0933) | val 0.5170 (rec 0.4716, kl 0.0907) | beta 0.500
Epoch 047 | train 0.5112 (rec 0.4650, kl 0.0924) | val 0.5204 (rec 0.4747, kl 0.0913) | beta 0.500
Epoch 048 | train 0.5114 (rec 0.4650, kl 0.0927) | val 0.5158 (rec 0.4711, kl 0.0893) | beta 0.500
Epoch 049 | train 0.5084 (rec 0.4622, kl 0.0924) | val 0.5140 (rec 0.4697, kl 0.0885) | beta 0.500
Epoch 050 | train 0.5094 (rec 0.4632, kl 0.0925) | val 0.5166 (rec 0.4717, kl 0.0897) | beta 0.500
Epoch 051 | train 0.5091 (rec 0.4628, kl 0.0926) | val 0.5117 (rec 0.4677, kl 0.0879) | beta 0.500
Epoch 052 | train 0.5080 (rec 0.4616, kl 0.0927) | val 0.5112 (rec 0.4658, kl 0.0907) | beta 0.500
Epoch 053 | train 0.5060 (rec 0.4598, kl 0.0924) | val 0.5118 (rec 0.4664, kl 0.0909) | beta 0.500
Epoch 054 | train 0.5076 (rec 0.4611, kl 0.0929) | val 0.5158 (rec 0.4707, kl 0.0903) | beta 0.500
Epoch 055 | train 0.5067 (rec 0.4603, kl 0.0928) | val 0.5124 (rec 0.4676, kl 0.0896) | beta 0.500
Epoch 056 | train 0.5065 (rec 0.4601, kl 0.0928) | val 0.5115 (rec 0.4664, kl 0.0904) | beta 0.500
Epoch 057 | train 0.5041 (rec 0.4577, kl 0.0929) | val 0.5086 (rec 0.4632, kl 0.0908) | beta 0.500
Epoch 058 | train 0.5042 (rec 0.4579, kl 0.0925) | val 0.5129 (rec 0.4682, kl 0.0894) | beta 0.500
Epoch 059 | train 0.5033 (rec 0.4569, kl 0.0927) | val 0.5096 (rec 0.4649, kl 0.0894) | beta 0.500
Epoch 060 | train 0.5045 (rec 0.4580, kl 0.0931) | val 0.5151 (rec 0.4700, kl 0.0902) | beta 0.500
Epoch 061 | train 0.5054 (rec 0.4590, kl 0.0929) | val 0.5126 (rec 0.4664, kl 0.0924) | beta 0.500
Epoch 062 | train 0.5052 (rec 0.4586, kl 0.0932) | val 0.5093 (rec 0.4644, kl 0.0899) | beta 0.500
Epoch 063 | train 0.5030 (rec 0.4566, kl 0.0928) | val 0.5131 (rec 0.4692, kl 0.0878) | beta 0.500
Epoch 064 | train 0.5058 (rec 0.4592, kl 0.0932) | val 0.5120 (rec 0.4661, kl 0.0918) | beta 0.500
Epoch 065 | train 0.5021 (rec 0.4557, kl 0.0928) | val 0.5093 (rec 0.4640, kl 0.0906) | beta 0.500
Epoch 066 | train 0.5013 (rec 0.4546, kl 0.0934) | val 0.5067 (rec 0.4606, kl 0.0920) | beta 0.500
Epoch 067 | train 0.5020 (rec 0.4556, kl 0.0928) | val 0.5046 (rec 0.4597, kl 0.0897) | beta 0.500
Epoch 068 | train 0.5032 (rec 0.4567, kl 0.0930) | val 0.5068 (rec 0.4611, kl 0.0914) | beta 0.500
Epoch 069 | train 0.4984 (rec 0.4519, kl 0.0929) | val 0.5068 (rec 0.4620, kl 0.0895) | beta 0.500
Epoch 070 | train 0.5004 (rec 0.4540, kl 0.0928) | val 0.5057 (rec 0.4608, kl 0.0900) | beta 0.500
Epoch 071 | train 0.5013 (rec 0.4547, kl 0.0931) | val 0.5061 (rec 0.4610, kl 0.0902) | beta 0.500
Epoch 072 | train 0.4992 (rec 0.4526, kl 0.0933) | val 0.5088 (rec 0.4636, kl 0.0904) | beta 0.500
Epoch 073 | train 0.5000 (rec 0.4534, kl 0.0932) | val 0.5071 (rec 0.4616, kl 0.0910) | beta 0.500
Epoch 074 | train 0.5021 (rec 0.4554, kl 0.0933) | val 0.5194 (rec 0.4730, kl 0.0928) | beta 0.500
Epoch 075 | train 0.5004 (rec 0.4536, kl 0.0935) | val 0.5074 (rec 0.4629, kl 0.0890) | beta 0.500
Epoch 076 | train 0.4986 (rec 0.4522, kl 0.0929) | val 0.5051 (rec 0.4598, kl 0.0906) | beta 0.500
Epoch 077 | train 0.4983 (rec 0.4519, kl 0.0927) | val 0.5070 (rec 0.4618, kl 0.0904) | beta 0.500
Epoch 078 | train 0.4997 (rec 0.4531, kl 0.0932) | val 0.5043 (rec 0.4593, kl 0.0902) | beta 0.500
Epoch 079 | train 0.4991 (rec 0.4525, kl 0.0931) | val 0.5061 (rec 0.4609, kl 0.0903) | beta 0.500
Epoch 080 | train 0.4976 (rec 0.4510, kl 0.0933) | val 0.5021 (rec 0.4574, kl 0.0894) | beta 0.500
Epoch 081 | train 0.4975 (rec 0.4512, kl 0.0927) | val 0.5070 (rec 0.4614, kl 0.0912) | beta 0.500
Epoch 082 | train 0.4968 (rec 0.4501, kl 0.0933) | val 0.5030 (rec 0.4574, kl 0.0913) | beta 0.500
Epoch 083 | train 0.4971 (rec 0.4507, kl 0.0928) | val 0.5009 (rec 0.4562, kl 0.0895) | beta 0.500
Epoch 084 | train 0.4965 (rec 0.4500, kl 0.0930) | val 0.5016 (rec 0.4560, kl 0.0912) | beta 0.500
Epoch 085 | train 0.4954 (rec 0.4488, kl 0.0933) | val 0.5029 (rec 0.4574, kl 0.0910) | beta 0.500
Epoch 086 | train 0.4965 (rec 0.4499, kl 0.0933) | val 0.5118 (rec 0.4660, kl 0.0916) | beta 0.500
Epoch 087 | train 0.4962 (rec 0.4496, kl 0.0931) | val 0.5021 (rec 0.4564, kl 0.0914) | beta 0.500
Epoch 088 | train 0.4957 (rec 0.4492, kl 0.0931) | val 0.5053 (rec 0.4603, kl 0.0899) | beta 0.500
Epoch 089 | train 0.4950 (rec 0.4484, kl 0.0933) | val 0.5027 (rec 0.4565, kl 0.0922) | beta 0.500
Epoch 090 | train 0.4955 (rec 0.4488, kl 0.0934) | val 0.5040 (rec 0.4582, kl 0.0916) | beta 0.500
Epoch 091 | train 0.4942 (rec 0.4476, kl 0.0933) | val 0.5019 (rec 0.4564, kl 0.0910) | beta 0.500
Epoch 092 | train 0.4962 (rec 0.4494, kl 0.0935) | val 0.5015 (rec 0.4555, kl 0.0920) | beta 0.500
Epoch 093 | train 0.4934 (rec 0.4467, kl 0.0934) | val 0.5030 (rec 0.4573, kl 0.0913) | beta 0.500
Epoch 094 | train 0.4931 (rec 0.4464, kl 0.0934) | val 0.5040 (rec 0.4587, kl 0.0906) | beta 0.500
Epoch 095 | train 0.4944 (rec 0.4478, kl 0.0932) | val 0.5048 (rec 0.4588, kl 0.0921) | beta 0.500
Epoch 096 | train 0.4950 (rec 0.4483, kl 0.0934) | val 0.5036 (rec 0.4586, kl 0.0900) | beta 0.500
Epoch 097 | train 0.4949 (rec 0.4482, kl 0.0935) | val 0.5110 (rec 0.4649, kl 0.0923) | beta 0.500
Epoch 098 | train 0.4934 (rec 0.4467, kl 0.0933) | val 0.5019 (rec 0.4561, kl 0.0917) | beta 0.500
Epoch 099 | train 0.4913 (rec 0.4447, kl 0.0932) | val 0.5012 (rec 0.4556, kl 0.0912) | beta 0.500
Epoch 100 | train 0.4924 (rec 0.4459, kl 0.0929) | val 0.5029 (rec 0.4563, kl 0.0934) | beta 0.500
Epoch 101 | train 0.4944 (rec 0.4476, kl 0.0936) | val 0.5016 (rec 0.4561, kl 0.0909) | beta 0.500
Epoch 102 | train 0.4922 (rec 0.4457, kl 0.0931) | val 0.5015 (rec 0.4545, kl 0.0941) | beta 0.500
Epoch 103 | train 0.4914 (rec 0.4445, kl 0.0939) | val 0.4992 (rec 0.4531, kl 0.0923) | beta 0.500
Epoch 104 | train 0.4920 (rec 0.4454, kl 0.0933) | val 0.5011 (rec 0.4556, kl 0.0910) | beta 0.500
Epoch 105 | train 0.4929 (rec 0.4463, kl 0.0933) | val 0.5035 (rec 0.4574, kl 0.0920) | beta 0.500
Epoch 106 | train 0.4925 (rec 0.4456, kl 0.0938) | val 0.4987 (rec 0.4523, kl 0.0926) | beta 0.500
Epoch 107 | train 0.4919 (rec 0.4450, kl 0.0936) | val 0.5029 (rec 0.4570, kl 0.0918) | beta 0.500
Epoch 108 | train 0.4916 (rec 0.4448, kl 0.0936) | val 0.4994 (rec 0.4530, kl 0.0929) | beta 0.500
Epoch 109 | train 0.4919 (rec 0.4450, kl 0.0938) | val 0.4986 (rec 0.4527, kl 0.0918) | beta 0.500
Epoch 110 | train 0.4917 (rec 0.4451, kl 0.0932) | val 0.5031 (rec 0.4577, kl 0.0908) | beta 0.500
Epoch 111 | train 0.4923 (rec 0.4455, kl 0.0936) | val 0.5015 (rec 0.4556, kl 0.0918) | beta 0.500
Epoch 112 | train 0.4931 (rec 0.4463, kl 0.0936) | val 0.5019 (rec 0.4564, kl 0.0911) | beta 0.500
Epoch 113 | train 0.4911 (rec 0.4445, kl 0.0934) | val 0.5009 (rec 0.4557, kl 0.0905) | beta 0.500
Epoch 114 | train 0.4900 (rec 0.4433, kl 0.0934) | val 0.5018 (rec 0.4561, kl 0.0914) | beta 0.500
Epoch 115 | train 0.4892 (rec 0.4425, kl 0.0935) | val 0.4994 (rec 0.4535, kl 0.0918) | beta 0.500
Epoch 116 | train 0.4913 (rec 0.4445, kl 0.0936) | val 0.5022 (rec 0.4566, kl 0.0912) | beta 0.500
Epoch 117 | train 0.4912 (rec 0.4443, kl 0.0938) | val 0.5003 (rec 0.4542, kl 0.0922) | beta 0.500
Epoch 118 | train 0.4892 (rec 0.4426, kl 0.0934) | val 0.4967 (rec 0.4513, kl 0.0907) | beta 0.500
Epoch 119 | train 0.4915 (rec 0.4447, kl 0.0936) | val 0.4974 (rec 0.4514, kl 0.0921) | beta 0.500
Epoch 120 | train 0.4884 (rec 0.4416, kl 0.0935) | val 0.4989 (rec 0.4529, kl 0.0920) | beta 0.500
Epoch 121 | train 0.4900 (rec 0.4433, kl 0.0933) | val 0.4969 (rec 0.4511, kl 0.0916) | beta 0.500
Epoch 122 | train 0.4923 (rec 0.4452, kl 0.0941) | val 0.5023 (rec 0.4569, kl 0.0908) | beta 0.500
Epoch 123 | train 0.4903 (rec 0.4435, kl 0.0935) | val 0.4979 (rec 0.4525, kl 0.0909) | beta 0.500
Epoch 124 | train 0.4890 (rec 0.4424, kl 0.0934) | val 0.5026 (rec 0.4563, kl 0.0926) | beta 0.500
Epoch 125 | train 0.4882 (rec 0.4415, kl 0.0933) | val 0.4970 (rec 0.4514, kl 0.0911) | beta 0.500
Epoch 126 | train 0.4893 (rec 0.4425, kl 0.0937) | val 0.5008 (rec 0.4546, kl 0.0924) | beta 0.500
Epoch 127 | train 0.4877 (rec 0.4409, kl 0.0937) | val 0.4967 (rec 0.4508, kl 0.0918) | beta 0.500
Epoch 128 | train 0.4883 (rec 0.4415, kl 0.0936) | val 0.4968 (rec 0.4509, kl 0.0918) | beta 0.500
Epoch 129 | train 0.4881 (rec 0.4413, kl 0.0936) | val 0.4991 (rec 0.4534, kl 0.0913) | beta 0.500
Epoch 130 | train 0.4883 (rec 0.4414, kl 0.0939) | val 0.5001 (rec 0.4544, kl 0.0914) | beta 0.500
Epoch 131 | train 0.4900 (rec 0.4432, kl 0.0937) | val 0.4986 (rec 0.4532, kl 0.0907) | beta 0.500
Epoch 132 | train 0.4881 (rec 0.4413, kl 0.0936) | val 0.4979 (rec 0.4518, kl 0.0922) | beta 0.500
Epoch 133 | train 0.4908 (rec 0.4438, kl 0.0940) | val 0.4989 (rec 0.4533, kl 0.0912) | beta 0.500
Epoch 134 | train 0.4890 (rec 0.4421, kl 0.0937) | val 0.4979 (rec 0.4523, kl 0.0913) | beta 0.500
Epoch 135 | train 0.4877 (rec 0.4409, kl 0.0937) | val 0.4977 (rec 0.4516, kl 0.0921) | beta 0.500
Epoch 136 | train 0.4862 (rec 0.4394, kl 0.0935) | val 0.4992 (rec 0.4536, kl 0.0912) | beta 0.500
Epoch 137 | train 0.4866 (rec 0.4398, kl 0.0936) | val 0.4959 (rec 0.4502, kl 0.0914) | beta 0.500
Epoch 138 | train 0.4864 (rec 0.4397, kl 0.0933) | val 0.5007 (rec 0.4548, kl 0.0919) | beta 0.500
Epoch 139 | train 0.4868 (rec 0.4400, kl 0.0936) | val 0.4973 (rec 0.4512, kl 0.0921) | beta 0.500
Epoch 140 | train 0.4868 (rec 0.4399, kl 0.0939) | val 0.4948 (rec 0.4482, kl 0.0931) | beta 0.500
Epoch 141 | train 0.4863 (rec 0.4396, kl 0.0934) | val 0.4957 (rec 0.4498, kl 0.0918) | beta 0.500
Epoch 142 | train 0.4868 (rec 0.4398, kl 0.0940) | val 0.5015 (rec 0.4558, kl 0.0915) | beta 0.500
Epoch 143 | train 0.4883 (rec 0.4415, kl 0.0937) | val 0.4946 (rec 0.4489, kl 0.0915) | beta 0.500
Epoch 144 | train 0.4883 (rec 0.4415, kl 0.0938) | val 0.4978 (rec 0.4514, kl 0.0928) | beta 0.500
Epoch 145 | train 0.4864 (rec 0.4394, kl 0.0939) | val 0.4969 (rec 0.4516, kl 0.0906) | beta 0.500
Epoch 146 | train 0.4858 (rec 0.4390, kl 0.0936) | val 0.4970 (rec 0.4520, kl 0.0900) | beta 0.500
Epoch 147 | train 0.4856 (rec 0.4389, kl 0.0934) | val 0.4969 (rec 0.4516, kl 0.0905) | beta 0.500
Epoch 148 | train 0.4862 (rec 0.4394, kl 0.0937) | val 0.4967 (rec 0.4515, kl 0.0904) | beta 0.500
Epoch 149 | train 0.4848 (rec 0.4378, kl 0.0939) | val 0.4946 (rec 0.4488, kl 0.0916) | beta 0.500
Epoch 150 | train 0.4851 (rec 0.4381, kl 0.0939) | val 0.4983 (rec 0.4522, kl 0.0921) | beta 0.500
Epoch 151 | train 0.4870 (rec 0.4401, kl 0.0938) | val 0.4963 (rec 0.4501, kl 0.0924) | beta 0.500
Epoch 152 | train 0.4858 (rec 0.4389, kl 0.0938) | val 0.5032 (rec 0.4579, kl 0.0907) | beta 0.500
Epoch 153 | train 0.4872 (rec 0.4404, kl 0.0936) | val 0.4948 (rec 0.4502, kl 0.0892) | beta 0.500
Epoch 154 | train 0.4843 (rec 0.4375, kl 0.0935) | val 0.4945 (rec 0.4491, kl 0.0909) | beta 0.500
Epoch 155 | train 0.4856 (rec 0.4388, kl 0.0936) | val 0.4938 (rec 0.4475, kl 0.0926) | beta 0.500
Epoch 156 | train 0.4862 (rec 0.4392, kl 0.0938) | val 0.4942 (rec 0.4484, kl 0.0916) | beta 0.500
Epoch 157 | train 0.4851 (rec 0.4381, kl 0.0940) | val 0.4991 (rec 0.4525, kl 0.0931) | beta 0.500
Epoch 158 | train 0.4855 (rec 0.4386, kl 0.0938) | val 0.4983 (rec 0.4527, kl 0.0912) | beta 0.500
Epoch 159 | train 0.4850 (rec 0.4379, kl 0.0942) | val 0.4963 (rec 0.4507, kl 0.0912) | beta 0.500
Epoch 160 | train 0.4846 (rec 0.4377, kl 0.0938) | val 0.4944 (rec 0.4493, kl 0.0903) | beta 0.500
Epoch 161 | train 0.4844 (rec 0.4373, kl 0.0943) | val 0.4936 (rec 0.4477, kl 0.0919) | beta 0.500
Epoch 162 | train 0.4823 (rec 0.4354, kl 0.0939) | val 0.4939 (rec 0.4480, kl 0.0918) | beta 0.500
Epoch 163 | train 0.4855 (rec 0.4386, kl 0.0938) | val 0.4955 (rec 0.4497, kl 0.0917) | beta 0.500
Epoch 164 | train 0.4864 (rec 0.4394, kl 0.0940) | val 0.4956 (rec 0.4493, kl 0.0927) | beta 0.500
Epoch 165 | train 0.4849 (rec 0.4380, kl 0.0937) | val 0.4981 (rec 0.4524, kl 0.0915) | beta 0.500
Epoch 166 | train 0.4844 (rec 0.4372, kl 0.0943) | val 0.4947 (rec 0.4486, kl 0.0922) | beta 0.500
Epoch 167 | train 0.4831 (rec 0.4361, kl 0.0940) | val 0.4940 (rec 0.4483, kl 0.0915) | beta 0.500
Epoch 168 | train 0.4830 (rec 0.4360, kl 0.0939) | val 0.4946 (rec 0.4484, kl 0.0924) | beta 0.500
Epoch 169 | train 0.4848 (rec 0.4377, kl 0.0943) | val 0.4937 (rec 0.4478, kl 0.0919) | beta 0.500
Epoch 170 | train 0.4844 (rec 0.4372, kl 0.0944) | val 0.4928 (rec 0.4469, kl 0.0917) | beta 0.500
Epoch 171 | train 0.4842 (rec 0.4373, kl 0.0937) | val 0.4948 (rec 0.4492, kl 0.0911) | beta 0.500
Epoch 172 | train 0.4819 (rec 0.4348, kl 0.0943) | val 0.4954 (rec 0.4500, kl 0.0909) | beta 0.500
Epoch 173 | train 0.4837 (rec 0.4368, kl 0.0937) | val 0.4946 (rec 0.4491, kl 0.0909) | beta 0.500
Epoch 174 | train 0.4847 (rec 0.4376, kl 0.0942) | val 0.4935 (rec 0.4477, kl 0.0916) | beta 0.500
Epoch 175 | train 0.4868 (rec 0.4398, kl 0.0941) | val 0.4959 (rec 0.4495, kl 0.0927) | beta 0.500
Epoch 176 | train 0.4829 (rec 0.4359, kl 0.0939) | val 0.4942 (rec 0.4484, kl 0.0915) | beta 0.500
Epoch 177 | train 0.4815 (rec 0.4346, kl 0.0940) | val 0.4935 (rec 0.4477, kl 0.0915) | beta 0.500
Epoch 178 | train 0.4814 (rec 0.4346, kl 0.0936) | val 0.4917 (rec 0.4461, kl 0.0911) | beta 0.500
Epoch 179 | train 0.4826 (rec 0.4357, kl 0.0938) | val 0.4930 (rec 0.4473, kl 0.0915) | beta 0.500
Epoch 180 | train 0.4827 (rec 0.4355, kl 0.0943) | val 0.4961 (rec 0.4498, kl 0.0926) | beta 0.500
Epoch 181 | train 0.4845 (rec 0.4373, kl 0.0943) | val 0.4950 (rec 0.4485, kl 0.0931) | beta 0.500
Epoch 182 | train 0.4845 (rec 0.4373, kl 0.0945) | val 0.4957 (rec 0.4504, kl 0.0905) | beta 0.500
Epoch 183 | train 0.4832 (rec 0.4363, kl 0.0938) | val 0.4937 (rec 0.4486, kl 0.0902) | beta 0.500
Epoch 184 | train 0.4806 (rec 0.4336, kl 0.0939) | val 0.4926 (rec 0.4468, kl 0.0917) | beta 0.500
Epoch 185 | train 0.4814 (rec 0.4344, kl 0.0940) | val 0.4927 (rec 0.4473, kl 0.0908) | beta 0.500
Epoch 186 | train 0.4833 (rec 0.4361, kl 0.0944) | val 0.4976 (rec 0.4514, kl 0.0923) | beta 0.500
Epoch 187 | train 0.4823 (rec 0.4352, kl 0.0942) | val 0.4935 (rec 0.4478, kl 0.0913) | beta 0.500
Epoch 188 | train 0.4828 (rec 0.4358, kl 0.0942) | val 0.4929 (rec 0.4468, kl 0.0921) | beta 0.500
Epoch 189 | train 0.4821 (rec 0.4351, kl 0.0940) | val 0.4942 (rec 0.4480, kl 0.0925) | beta 0.500
Epoch 190 | train 0.4849 (rec 0.4379, kl 0.0941) | val 0.4952 (rec 0.4485, kl 0.0933) | beta 0.500
Epoch 191 | train 0.4827 (rec 0.4355, kl 0.0945) | val 0.4927 (rec 0.4470, kl 0.0913) | beta 0.500
Epoch 192 | train 0.4822 (rec 0.4350, kl 0.0944) | val 0.4931 (rec 0.4477, kl 0.0908) | beta 0.500
Epoch 193 | train 0.4815 (rec 0.4344, kl 0.0942) | val 0.4933 (rec 0.4474, kl 0.0916) | beta 0.500
Epoch 194 | train 0.4795 (rec 0.4325, kl 0.0940) | val 0.4981 (rec 0.4527, kl 0.0908) | beta 0.500
Epoch 195 | train 0.4863 (rec 0.4388, kl 0.0951) | val 0.4959 (rec 0.4496, kl 0.0926) | beta 0.500
Epoch 196 | train 0.4806 (rec 0.4335, kl 0.0943) | val 0.4933 (rec 0.4474, kl 0.0918) | beta 0.500
Epoch 197 | train 0.4830 (rec 0.4359, kl 0.0941) | val 0.4955 (rec 0.4492, kl 0.0927) | beta 0.500
Epoch 198 | train 0.4825 (rec 0.4353, kl 0.0942) | val 0.4934 (rec 0.4466, kl 0.0936) | beta 0.500
Epoch 199 | train 0.4819 (rec 0.4349, kl 0.0941) | val 0.4912 (rec 0.4452, kl 0.0921) | beta 0.500
Epoch 200 | train 0.4807 (rec 0.4335, kl 0.0943) | val 0.4919 (rec 0.4464, kl 0.0910) | beta 0.500
Test loss 0.4881 (rec 0.4424, kl 0.0914)
[Predictor] Epoch 001 | train 2.5481 | val 1.3278 | lr 5.00e-04
[Predictor] Epoch 002 | train 1.2489 | val 1.1944 | lr 5.00e-04
[Predictor] Epoch 003 | train 1.1677 | val 1.1158 | lr 5.00e-04
[Predictor] Epoch 004 | train 1.0401 | val 0.9757 | lr 5.00e-04
[Predictor] Epoch 005 | train 0.9357 | val 0.8907 | lr 5.00e-04
[Predictor] Epoch 006 | train 0.8799 | val 0.8572 | lr 5.00e-04
[Predictor] Epoch 007 | train 0.8449 | val 0.8281 | lr 5.00e-04
[Predictor] Epoch 008 | train 0.8165 | val 0.8065 | lr 5.00e-04
[Predictor] Epoch 009 | train 0.7950 | val 0.7824 | lr 5.00e-04
[Predictor] Epoch 010 | train 0.7733 | val 0.7622 | lr 5.00e-04
[Predictor] Epoch 011 | train 0.7548 | val 0.7493 | lr 5.00e-04
[Predictor] Epoch 012 | train 0.7430 | val 0.7366 | lr 5.00e-04
[Predictor] Epoch 013 | train 0.7311 | val 0.7284 | lr 5.00e-04
[Predictor] Epoch 014 | train 0.7217 | val 0.7209 | lr 5.00e-04
[Predictor] Epoch 015 | train 0.7152 | val 0.7168 | lr 5.00e-04
[Predictor] Epoch 016 | train 0.7080 | val 0.7083 | lr 5.00e-04
[Predictor] Epoch 017 | train 0.7022 | val 0.7018 | lr 5.00e-04
[Predictor] Epoch 018 | train 0.6984 | val 0.7030 | lr 5.00e-04
[Predictor] Epoch 019 | train 0.6932 | val 0.6985 | lr 5.00e-04
[Predictor] Epoch 020 | train 0.6908 | val 0.6951 | lr 5.00e-04
[Predictor] Epoch 021 | train 0.6865 | val 0.6961 | lr 5.00e-04
[Predictor] Epoch 022 | train 0.6842 | val 0.6873 | lr 5.00e-04
[Predictor] Epoch 023 | train 0.6825 | val 0.6914 | lr 5.00e-04
[Predictor] Epoch 024 | train 0.6773 | val 0.6830 | lr 5.00e-04
[Predictor] Epoch 025 | train 0.6760 | val 0.6913 | lr 5.00e-04
[Predictor] Epoch 026 | train 0.6747 | val 0.6820 | lr 5.00e-04
[Predictor] Epoch 027 | train 0.6727 | val 0.6793 | lr 5.00e-04
[Predictor] Epoch 028 | train 0.6724 | val 0.6758 | lr 5.00e-04
[Predictor] Epoch 029 | train 0.6681 | val 0.6752 | lr 5.00e-04
[Predictor] Epoch 030 | train 0.6677 | val 0.6724 | lr 5.00e-04
[Predictor] Epoch 031 | train 0.6659 | val 0.6767 | lr 5.00e-04
[Predictor] Epoch 032 | train 0.6665 | val 0.6722 | lr 5.00e-04
[Predictor] Epoch 033 | train 0.6656 | val 0.6730 | lr 5.00e-04
[Predictor] Epoch 034 | train 0.6638 | val 0.6754 | lr 5.00e-04
[Predictor] Epoch 035 | train 0.6636 | val 0.6656 | lr 5.00e-04
[Predictor] Epoch 036 | train 0.6620 | val 0.6702 | lr 5.00e-04
[Predictor] Epoch 037 | train 0.6601 | val 0.6682 | lr 5.00e-04
[Predictor] Epoch 038 | train 0.6598 | val 0.6657 | lr 5.00e-04
[Predictor] Epoch 039 | train 0.6588 | val 0.6659 | lr 5.00e-04
[Predictor] Epoch 040 | train 0.6569 | val 0.6672 | lr 5.00e-04
[Predictor] Epoch 041 | train 0.6588 | val 0.6674 | lr 5.00e-04
[Predictor] Epoch 042 | train 0.6565 | val 0.6640 | lr 5.00e-04
[Predictor] Epoch 043 | train 0.6583 | val 0.6728 | lr 5.00e-04
[Predictor] Epoch 044 | train 0.6559 | val 0.6625 | lr 5.00e-04
[Predictor] Epoch 045 | train 0.6548 | val 0.6647 | lr 5.00e-04
[Predictor] Epoch 046 | train 0.6539 | val 0.6623 | lr 5.00e-04
[Predictor] Epoch 047 | train 0.6544 | val 0.6644 | lr 5.00e-04
[Predictor] Epoch 048 | train 0.6540 | val 0.6626 | lr 5.00e-04
[Predictor] Epoch 049 | train 0.6538 | val 0.6672 | lr 5.00e-04
[Predictor] Epoch 050 | train 0.6532 | val 0.6619 | lr 5.00e-04
[Predictor] Epoch 051 | train 0.6525 | val 0.6606 | lr 5.00e-04
[Predictor] Epoch 052 | train 0.6519 | val 0.6615 | lr 5.00e-04
[Predictor] Epoch 053 | train 0.6519 | val 0.6626 | lr 5.00e-04
[Predictor] Epoch 054 | train 0.6510 | val 0.6637 | lr 5.00e-04
[Predictor] Epoch 055 | train 0.6511 | val 0.6623 | lr 5.00e-04
[Predictor] Epoch 056 | train 0.6515 | val 0.6596 | lr 5.00e-04
[Predictor] Epoch 057 | train 0.6511 | val 0.6564 | lr 5.00e-04
[Predictor] Epoch 058 | train 0.6503 | val 0.6619 | lr 5.00e-04
[Predictor] Epoch 059 | train 0.6516 | val 0.6611 | lr 5.00e-04
[Predictor] Epoch 060 | train 0.6496 | val 0.6554 | lr 5.00e-04
[Predictor] Epoch 061 | train 0.6482 | val 0.6541 | lr 5.00e-04
[Predictor] Epoch 062 | train 0.6480 | val 0.6579 | lr 5.00e-04
[Predictor] Epoch 063 | train 0.6482 | val 0.6585 | lr 5.00e-04
[Predictor] Epoch 064 | train 0.6499 | val 0.6586 | lr 5.00e-04
[Predictor] Epoch 065 | train 0.6473 | val 0.6589 | lr 5.00e-04
[Predictor] Epoch 066 | train 0.6475 | val 0.6572 | lr 5.00e-04
[Predictor] Epoch 067 | train 0.6480 | val 0.6555 | lr 5.00e-04
[Predictor] Epoch 068 | train 0.6480 | val 0.6588 | lr 5.00e-04
[Predictor] Epoch 069 | train 0.6485 | val 0.6572 | lr 5.00e-04
[Predictor] Epoch 070 | train 0.6465 | val 0.6577 | lr 5.00e-04
[Predictor] Epoch 071 | train 0.6462 | val 0.6561 | lr 5.00e-04
[Predictor] Epoch 072 | train 0.6465 | val 0.6616 | lr 2.50e-04
[Predictor] Epoch 073 | train 0.6278 | val 0.6395 | lr 2.50e-04
[Predictor] Epoch 074 | train 0.6242 | val 0.6396 | lr 2.50e-04
[Predictor] Epoch 075 | train 0.6241 | val 0.6396 | lr 2.50e-04
[Predictor] Epoch 076 | train 0.6247 | val 0.6393 | lr 2.50e-04
[Predictor] Epoch 077 | train 0.6247 | val 0.6427 | lr 2.50e-04
[Predictor] Epoch 078 | train 0.6241 | val 0.6398 | lr 2.50e-04
[Predictor] Epoch 079 | train 0.6245 | val 0.6410 | lr 2.50e-04
[Predictor] Epoch 080 | train 0.6239 | val 0.6420 | lr 2.50e-04
[Predictor] Epoch 081 | train 0.6236 | val 0.6400 | lr 2.50e-04
[Predictor] Epoch 082 | train 0.6240 | val 0.6395 | lr 2.50e-04
[Predictor] Epoch 083 | train 0.6231 | val 0.6396 | lr 2.50e-04
[Predictor] Epoch 084 | train 0.6229 | val 0.6392 | lr 2.50e-04
[Predictor] Epoch 085 | train 0.6234 | val 0.6384 | lr 2.50e-04
[Predictor] Epoch 086 | train 0.6225 | val 0.6388 | lr 2.50e-04
[Predictor] Epoch 087 | train 0.6226 | val 0.6388 | lr 2.50e-04
[Predictor] Epoch 088 | train 0.6219 | val 0.6384 | lr 2.50e-04
[Predictor] Epoch 089 | train 0.6217 | val 0.6388 | lr 2.50e-04
[Predictor] Epoch 090 | train 0.6214 | val 0.6378 | lr 2.50e-04
[Predictor] Epoch 091 | train 0.6213 | val 0.6392 | lr 2.50e-04
[Predictor] Epoch 092 | train 0.6204 | val 0.6387 | lr 2.50e-04
[Predictor] Epoch 093 | train 0.6207 | val 0.6379 | lr 2.50e-04
[Predictor] Epoch 094 | train 0.6203 | val 0.6382 | lr 2.50e-04
[Predictor] Epoch 095 | train 0.6203 | val 0.6383 | lr 2.50e-04
[Predictor] Epoch 096 | train 0.6201 | val 0.6382 | lr 2.50e-04
[Predictor] Epoch 097 | train 0.6199 | val 0.6379 | lr 2.50e-04
[Predictor] Epoch 098 | train 0.6193 | val 0.6363 | lr 2.50e-04
[Predictor] Epoch 099 | train 0.6191 | val 0.6364 | lr 2.50e-04
[Predictor] Epoch 100 | train 0.6184 | val 0.6373 | lr 2.50e-04
[Predictor] Epoch 101 | train 0.6183 | val 0.6378 | lr 2.50e-04
[Predictor] Epoch 102 | train 0.6184 | val 0.6363 | lr 2.50e-04
[Predictor] Epoch 103 | train 0.6185 | val 0.6372 | lr 2.50e-04
[Predictor] Epoch 104 | train 0.6177 | val 0.6368 | lr 2.50e-04
[Predictor] Epoch 105 | train 0.6183 | val 0.6363 | lr 2.50e-04
[Predictor] Epoch 106 | train 0.6180 | val 0.6378 | lr 2.50e-04
[Predictor] Epoch 107 | train 0.6181 | val 0.6363 | lr 2.50e-04
[Predictor] Epoch 108 | train 0.6172 | val 0.6362 | lr 2.50e-04
[Predictor] Epoch 109 | train 0.6172 | val 0.6367 | lr 2.50e-04
[Predictor] Epoch 110 | train 0.6169 | val 0.6364 | lr 2.50e-04
[Predictor] Epoch 111 | train 0.6169 | val 0.6352 | lr 2.50e-04
[Predictor] Epoch 112 | train 0.6161 | val 0.6352 | lr 2.50e-04
[Predictor] Epoch 113 | train 0.6167 | val 0.6352 | lr 2.50e-04
[Predictor] Epoch 114 | train 0.6164 | val 0.6352 | lr 2.50e-04
[Predictor] Epoch 115 | train 0.6160 | val 0.6363 | lr 2.50e-04
[Predictor] Epoch 116 | train 0.6156 | val 0.6344 | lr 2.50e-04
[Predictor] Epoch 117 | train 0.6158 | val 0.6359 | lr 2.50e-04
[Predictor] Epoch 118 | train 0.6158 | val 0.6353 | lr 2.50e-04
[Predictor] Epoch 119 | train 0.6146 | val 0.6340 | lr 2.50e-04
[Predictor] Epoch 120 | train 0.6148 | val 0.6337 | lr 2.50e-04
[Predictor] Epoch 121 | train 0.6148 | val 0.6356 | lr 2.50e-04
[Predictor] Epoch 122 | train 0.6148 | val 0.6346 | lr 2.50e-04
[Predictor] Epoch 123 | train 0.6145 | val 0.6344 | lr 2.50e-04
[Predictor] Epoch 124 | train 0.6142 | val 0.6344 | lr 2.50e-04
[Predictor] Epoch 125 | train 0.6136 | val 0.6347 | lr 2.50e-04
[Predictor] Epoch 126 | train 0.6137 | val 0.6341 | lr 2.50e-04
[Predictor] Epoch 127 | train 0.6140 | val 0.6336 | lr 2.50e-04
[Predictor] Epoch 128 | train 0.6145 | val 0.6336 | lr 2.50e-04
[Predictor] Epoch 129 | train 0.6133 | val 0.6341 | lr 2.50e-04
[Predictor] Epoch 130 | train 0.6132 | val 0.6333 | lr 2.50e-04
[Predictor] Epoch 131 | train 0.6130 | val 0.6339 | lr 2.50e-04
[Predictor] Epoch 132 | train 0.6128 | val 0.6344 | lr 2.50e-04
[Predictor] Epoch 133 | train 0.6128 | val 0.6339 | lr 2.50e-04
[Predictor] Epoch 134 | train 0.6132 | val 0.6345 | lr 2.50e-04
[Predictor] Epoch 135 | train 0.6132 | val 0.6341 | lr 2.50e-04
[Predictor] Epoch 136 | train 0.6128 | val 0.6327 | lr 2.50e-04
[Predictor] Epoch 137 | train 0.6131 | val 0.6335 | lr 2.50e-04
[Predictor] Epoch 138 | train 0.6120 | val 0.6328 | lr 2.50e-04
[Predictor] Epoch 139 | train 0.6117 | val 0.6333 | lr 2.50e-04
[Predictor] Epoch 140 | train 0.6111 | val 0.6345 | lr 2.50e-04
[Predictor] Epoch 141 | train 0.6116 | val 0.6341 | lr 2.50e-04
[Predictor] Epoch 142 | train 0.6113 | val 0.6326 | lr 2.50e-04
[Predictor] Epoch 143 | train 0.6114 | val 0.6335 | lr 2.50e-04
[Predictor] Epoch 144 | train 0.6110 | val 0.6342 | lr 2.50e-04
[Predictor] Epoch 145 | train 0.6116 | val 0.6325 | lr 2.50e-04
[Predictor] Epoch 146 | train 0.6106 | val 0.6319 | lr 2.50e-04
[Predictor] Epoch 147 | train 0.6107 | val 0.6309 | lr 2.50e-04
[Predictor] Epoch 148 | train 0.6108 | val 0.6340 | lr 2.50e-04
[Predictor] Epoch 149 | train 0.6104 | val 0.6320 | lr 2.50e-04
[Predictor] Epoch 150 | train 0.6110 | val 0.6321 | lr 2.50e-04
[Predictor] Epoch 151 | train 0.6104 | val 0.6326 | lr 2.50e-04
[Predictor] Epoch 152 | train 0.6102 | val 0.6328 | lr 2.50e-04
[Predictor] Epoch 153 | train 0.6099 | val 0.6328 | lr 2.50e-04
[Predictor] Epoch 154 | train 0.6102 | val 0.6328 | lr 2.50e-04
[Predictor] Epoch 155 | train 0.6095 | val 0.6346 | lr 2.50e-04
[Predictor] Epoch 156 | train 0.6094 | val 0.6328 | lr 2.50e-04
[Predictor] Epoch 157 | train 0.6092 | val 0.6316 | lr 2.50e-04
[Predictor] Epoch 158 | train 0.6089 | val 0.6327 | lr 1.25e-04
[Predictor] Epoch 159 | train 0.5960 | val 0.6236 | lr 1.25e-04
[Predictor] Epoch 160 | train 0.5944 | val 0.6222 | lr 1.25e-04
[Predictor] Epoch 161 | train 0.5933 | val 0.6213 | lr 1.25e-04
[Predictor] Epoch 162 | train 0.5935 | val 0.6217 | lr 1.25e-04
[Predictor] Epoch 163 | train 0.5933 | val 0.6213 | lr 1.25e-04
[Predictor] Epoch 164 | train 0.5933 | val 0.6220 | lr 1.25e-04
[Predictor] Epoch 165 | train 0.5930 | val 0.6213 | lr 1.25e-04
[Predictor] Epoch 166 | train 0.5931 | val 0.6217 | lr 1.25e-04
[Predictor] Epoch 167 | train 0.5926 | val 0.6211 | lr 1.25e-04
[Predictor] Epoch 168 | train 0.5921 | val 0.6221 | lr 1.25e-04
[Predictor] Epoch 169 | train 0.5926 | val 0.6228 | lr 1.25e-04
[Predictor] Epoch 170 | train 0.5926 | val 0.6217 | lr 1.25e-04
[Predictor] Epoch 171 | train 0.5918 | val 0.6215 | lr 1.25e-04
[Predictor] Epoch 172 | train 0.5921 | val 0.6209 | lr 1.25e-04
[Predictor] Epoch 173 | train 0.5924 | val 0.6214 | lr 1.25e-04
[Predictor] Epoch 174 | train 0.5915 | val 0.6216 | lr 1.25e-04
[Predictor] Epoch 175 | train 0.5918 | val 0.6220 | lr 1.25e-04
[Predictor] Epoch 176 | train 0.5915 | val 0.6210 | lr 1.25e-04
[Predictor] Epoch 177 | train 0.5909 | val 0.6216 | lr 1.25e-04
[Predictor] Epoch 178 | train 0.5908 | val 0.6214 | lr 1.25e-04
[Predictor] Epoch 179 | train 0.5912 | val 0.6225 | lr 1.25e-04
[Predictor] Epoch 180 | train 0.5905 | val 0.6207 | lr 1.25e-04
[Predictor] Epoch 181 | train 0.5906 | val 0.6213 | lr 1.25e-04
[Predictor] Epoch 182 | train 0.5906 | val 0.6216 | lr 1.25e-04
[Predictor] Epoch 183 | train 0.5907 | val 0.6213 | lr 1.25e-04
[Predictor] Epoch 184 | train 0.5902 | val 0.6212 | lr 1.25e-04
[Predictor] Epoch 185 | train 0.5904 | val 0.6217 | lr 1.25e-04
[Predictor] Epoch 186 | train 0.5904 | val 0.6230 | lr 1.25e-04
[Predictor] Epoch 187 | train 0.5903 | val 0.6200 | lr 1.25e-04
[Predictor] Epoch 188 | train 0.5896 | val 0.6210 | lr 1.25e-04
[Predictor] Epoch 189 | train 0.5895 | val 0.6202 | lr 1.25e-04
[Predictor] Epoch 190 | train 0.5896 | val 0.6221 | lr 1.25e-04
[Predictor] Epoch 191 | train 0.5894 | val 0.6205 | lr 1.25e-04
[Predictor] Epoch 192 | train 0.5894 | val 0.6201 | lr 1.25e-04
[Predictor] Epoch 193 | train 0.5891 | val 0.6207 | lr 1.25e-04
[Predictor] Epoch 194 | train 0.5890 | val 0.6210 | lr 1.25e-04
[Predictor] Epoch 195 | train 0.5890 | val 0.6214 | lr 1.25e-04
[Predictor] Epoch 196 | train 0.5889 | val 0.6207 | lr 1.25e-04
[Predictor] Epoch 197 | train 0.5885 | val 0.6224 | lr 1.25e-04
[Predictor] Epoch 198 | train 0.5884 | val 0.6203 | lr 6.25e-05
[Predictor] Epoch 199 | train 0.5799 | val 0.6149 | lr 6.25e-05
[Predictor] Epoch 200 | train 0.5785 | val 0.6146 | lr 6.25e-05
Predictor MSE | train 0.5565 | val 0.6146 | test 0.6156
Isoform correlation: mean Pearson 0.5924, median 0.6188 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_vae_res/isoform_correlation_boxplot_vae.png
Saved log-scale scatter to isoform_model_vae_res/pred_vs_true_scatter_log.png
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7683 | median of means 0.7870 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_vae_res/gene_rank_corr_mean_boxplot_vae.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.8128 | median of medians 0.8286 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_vae_res/gene_rank_corr_median_boxplot_vae.png


## PCA + MLP

In [ ]:
Loading AnnData files...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Using whole dataset: train 9941, val 6958, test 2983
Applying PCA with 512 components...
PCA complete: (9941, 512)
Explained variance: 0.7216 (72.16%)
Using PCA-transformed inputs: 512 components (explained variance 0.7216, 72.16%)
Epoch 001 | train 8.0318 | val 3.3459 | lr 2.00e-04
Epoch 002 | train 2.1073 | val 1.4289 | lr 2.00e-04
Epoch 003 | train 1.3798 | val 1.2141 | lr 2.00e-04
Epoch 004 | train 1.2678 | val 1.1508 | lr 2.00e-04
Epoch 005 | train 1.1988 | val 1.1044 | lr 2.00e-04
Epoch 006 | train 1.1471 | val 1.0680 | lr 2.00e-04
Epoch 007 | train 1.1007 | val 1.0064 | lr 2.00e-04
Epoch 008 | train 1.0662 | val 0.9706 | lr 2.00e-04
Epoch 009 | train 1.0322 | val 0.9405 | lr 2.00e-04
Epoch 010 | train 1.0064 | val 0.9192 | lr 2.00e-04
Epoch 011 | train 0.9837 | val 0.9058 | lr 2.00e-04
Epoch 012 | train 0.9616 | val 0.8763 | lr 2.00e-04
Epoch 013 | train 0.9421 | val 0.8521 | lr 2.00e-04
Epoch 014 | train 0.9279 | val 0.8442 | lr 2.00e-04
Epoch 015 | train 0.9143 | val 0.8319 | lr 2.00e-04
Epoch 016 | train 0.9018 | val 0.8229 | lr 2.00e-04
Epoch 017 | train 0.8915 | val 0.8100 | lr 2.00e-04
Epoch 018 | train 0.8819 | val 0.8028 | lr 2.00e-04
Epoch 019 | train 0.8730 | val 0.7973 | lr 2.00e-04
Epoch 020 | train 0.8681 | val 0.7810 | lr 2.00e-04
Epoch 021 | train 0.8604 | val 0.7830 | lr 2.00e-04
Epoch 022 | train 0.8549 | val 0.7809 | lr 2.00e-04
Epoch 023 | train 0.8466 | val 0.7697 | lr 2.00e-04
Epoch 024 | train 0.8434 | val 0.7677 | lr 2.00e-04
Epoch 025 | train 0.8357 | val 0.7630 | lr 2.00e-04
Epoch 026 | train 0.8321 | val 0.7540 | lr 2.00e-04
Epoch 027 | train 0.8267 | val 0.7476 | lr 2.00e-04
Epoch 028 | train 0.8236 | val 0.7479 | lr 2.00e-04
Epoch 029 | train 0.8215 | val 0.7477 | lr 2.00e-04
Epoch 030 | train 0.8177 | val 0.7400 | lr 2.00e-04
Epoch 031 | train 0.8144 | val 0.7365 | lr 2.00e-04
Epoch 032 | train 0.8124 | val 0.7403 | lr 2.00e-04
Epoch 033 | train 0.8062 | val 0.7340 | lr 2.00e-04
Epoch 034 | train 0.8038 | val 0.7256 | lr 2.00e-04
Epoch 035 | train 0.8016 | val 0.7295 | lr 2.00e-04
Epoch 036 | train 0.7998 | val 0.7227 | lr 2.00e-04
Epoch 037 | train 0.7969 | val 0.7236 | lr 2.00e-04
Epoch 038 | train 0.7943 | val 0.7210 | lr 2.00e-04
Epoch 039 | train 0.7900 | val 0.7169 | lr 2.00e-04
Epoch 040 | train 0.7900 | val 0.7093 | lr 2.00e-04
Epoch 041 | train 0.7894 | val 0.7099 | lr 2.00e-04
Epoch 042 | train 0.7858 | val 0.7097 | lr 2.00e-04
Epoch 043 | train 0.7862 | val 0.7106 | lr 2.00e-04
Epoch 044 | train 0.7849 | val 0.7077 | lr 2.00e-04
Epoch 045 | train 0.7847 | val 0.6988 | lr 2.00e-04
Epoch 046 | train 0.7807 | val 0.7014 | lr 2.00e-04
Epoch 047 | train 0.7792 | val 0.7015 | lr 2.00e-04
Epoch 048 | train 0.7782 | val 0.7012 | lr 2.00e-04
Epoch 049 | train 0.7760 | val 0.6985 | lr 2.00e-04
Epoch 050 | train 0.7743 | val 0.6959 | lr 2.00e-04
Epoch 051 | train 0.7736 | val 0.6915 | lr 2.00e-04
Epoch 052 | train 0.7728 | val 0.6904 | lr 2.00e-04
Epoch 053 | train 0.7699 | val 0.6874 | lr 2.00e-04
Epoch 054 | train 0.7714 | val 0.6909 | lr 2.00e-04
Epoch 055 | train 0.7684 | val 0.6877 | lr 2.00e-04
Epoch 056 | train 0.7669 | val 0.6856 | lr 2.00e-04
Epoch 057 | train 0.7660 | val 0.6845 | lr 2.00e-04
Epoch 058 | train 0.7670 | val 0.6841 | lr 2.00e-04
Epoch 059 | train 0.7655 | val 0.6846 | lr 2.00e-04
Epoch 060 | train 0.7627 | val 0.6868 | lr 2.00e-04
Epoch 061 | train 0.7596 | val 0.6810 | lr 2.00e-04
Epoch 062 | train 0.7622 | val 0.6840 | lr 2.00e-04
Epoch 063 | train 0.7623 | val 0.6833 | lr 2.00e-04
Epoch 064 | train 0.7620 | val 0.6816 | lr 2.00e-04
Epoch 065 | train 0.7602 | val 0.6821 | lr 2.00e-04
Epoch 066 | train 0.7588 | val 0.6783 | lr 2.00e-04
Epoch 067 | train 0.7553 | val 0.6752 | lr 2.00e-04
Epoch 068 | train 0.7570 | val 0.6826 | lr 2.00e-04
Epoch 069 | train 0.7566 | val 0.6736 | lr 2.00e-04
Epoch 070 | train 0.7542 | val 0.6742 | lr 2.00e-04
Epoch 071 | train 0.7549 | val 0.6705 | lr 2.00e-04
Epoch 072 | train 0.7557 | val 0.6690 | lr 2.00e-04
Epoch 073 | train 0.7528 | val 0.6761 | lr 2.00e-04
Epoch 074 | train 0.7534 | val 0.6658 | lr 2.00e-04
Epoch 075 | train 0.7532 | val 0.6702 | lr 2.00e-04
Epoch 076 | train 0.7499 | val 0.6734 | lr 2.00e-04
Epoch 077 | train 0.7514 | val 0.6753 | lr 2.00e-04
Epoch 078 | train 0.7504 | val 0.6729 | lr 2.00e-04
Epoch 079 | train 0.7497 | val 0.6737 | lr 2.00e-04
Epoch 080 | train 0.7513 | val 0.6671 | lr 2.00e-04
Epoch 081 | train 0.7472 | val 0.6680 | lr 2.00e-04
Epoch 082 | train 0.7484 | val 0.6731 | lr 2.00e-04
Epoch 083 | train 0.7467 | val 0.6701 | lr 2.00e-04
Epoch 084 | train 0.7480 | val 0.6725 | lr 2.00e-04
Epoch 085 | train 0.7472 | val 0.6654 | lr 2.00e-04
Epoch 086 | train 0.7475 | val 0.6681 | lr 2.00e-04
Epoch 087 | train 0.7455 | val 0.6657 | lr 2.00e-04
Epoch 088 | train 0.7439 | val 0.6656 | lr 2.00e-04
Epoch 089 | train 0.7448 | val 0.6605 | lr 2.00e-04
Epoch 090 | train 0.7445 | val 0.6638 | lr 2.00e-04
Epoch 091 | train 0.7434 | val 0.6615 | lr 2.00e-04
Epoch 092 | train 0.7420 | val 0.6619 | lr 2.00e-04
Epoch 093 | train 0.7419 | val 0.6679 | lr 2.00e-04
Epoch 094 | train 0.7427 | val 0.6613 | lr 2.00e-04
Epoch 095 | train 0.7435 | val 0.6595 | lr 2.00e-04
Epoch 096 | train 0.7425 | val 0.6649 | lr 2.00e-04
Epoch 097 | train 0.7426 | val 0.6555 | lr 2.00e-04
Epoch 098 | train 0.7432 | val 0.6535 | lr 2.00e-04
Epoch 099 | train 0.7398 | val 0.6554 | lr 2.00e-04
Epoch 100 | train 0.7392 | val 0.6607 | lr 2.00e-04
Epoch 101 | train 0.7413 | val 0.6590 | lr 2.00e-04
Epoch 102 | train 0.7398 | val 0.6603 | lr 2.00e-04
Epoch 103 | train 0.7405 | val 0.6580 | lr 2.00e-04
Epoch 104 | train 0.7399 | val 0.6578 | lr 2.00e-04
Epoch 105 | train 0.7370 | val 0.6579 | lr 2.00e-04
Epoch 106 | train 0.7393 | val 0.6617 | lr 2.00e-04
Epoch 107 | train 0.7370 | val 0.6524 | lr 2.00e-04
Epoch 108 | train 0.7377 | val 0.6558 | lr 2.00e-04
Epoch 109 | train 0.7350 | val 0.6527 | lr 2.00e-04
Epoch 110 | train 0.7382 | val 0.6533 | lr 2.00e-04
Epoch 111 | train 0.7369 | val 0.6588 | lr 2.00e-04
Epoch 112 | train 0.7370 | val 0.6567 | lr 2.00e-04
Epoch 113 | train 0.7343 | val 0.6511 | lr 2.00e-04
Epoch 114 | train 0.7380 | val 0.6584 | lr 2.00e-04
Epoch 115 | train 0.7351 | val 0.6504 | lr 2.00e-04
Epoch 116 | train 0.7342 | val 0.6503 | lr 2.00e-04
Epoch 117 | train 0.7332 | val 0.6536 | lr 2.00e-04
Epoch 118 | train 0.7353 | val 0.6469 | lr 2.00e-04
Epoch 119 | train 0.7342 | val 0.6480 | lr 2.00e-04
Epoch 120 | train 0.7326 | val 0.6546 | lr 2.00e-04
Epoch 121 | train 0.7343 | val 0.6478 | lr 2.00e-04
Epoch 122 | train 0.7353 | val 0.6590 | lr 2.00e-04
Epoch 123 | train 0.7348 | val 0.6485 | lr 2.00e-04
Epoch 124 | train 0.7344 | val 0.6514 | lr 2.00e-04
Epoch 125 | train 0.7345 | val 0.6503 | lr 2.00e-04
Epoch 126 | train 0.7334 | val 0.6439 | lr 2.00e-04
Epoch 127 | train 0.7354 | val 0.6488 | lr 2.00e-04
Epoch 128 | train 0.7313 | val 0.6459 | lr 2.00e-04
Epoch 129 | train 0.7328 | val 0.6453 | lr 2.00e-04
Epoch 130 | train 0.7319 | val 0.6467 | lr 2.00e-04
Epoch 131 | train 0.7305 | val 0.6449 | lr 2.00e-04
Epoch 132 | train 0.7331 | val 0.6464 | lr 2.00e-04
Epoch 133 | train 0.7304 | val 0.6465 | lr 2.00e-04
Epoch 134 | train 0.7304 | val 0.6498 | lr 2.00e-04
Epoch 135 | train 0.7303 | val 0.6477 | lr 2.00e-04
Epoch 136 | train 0.7283 | val 0.6443 | lr 2.00e-04
Epoch 137 | train 0.7298 | val 0.6414 | lr 2.00e-04
Epoch 138 | train 0.7283 | val 0.6463 | lr 2.00e-04
Epoch 139 | train 0.7324 | val 0.6477 | lr 2.00e-04
Epoch 140 | train 0.7298 | val 0.6461 | lr 2.00e-04
Epoch 141 | train 0.7307 | val 0.6459 | lr 2.00e-04
Epoch 142 | train 0.7309 | val 0.6455 | lr 2.00e-04
Epoch 143 | train 0.7271 | val 0.6409 | lr 2.00e-04
Epoch 144 | train 0.7287 | val 0.6448 | lr 2.00e-04
Epoch 145 | train 0.7291 | val 0.6486 | lr 2.00e-04
Epoch 146 | train 0.7290 | val 0.6401 | lr 2.00e-04
Epoch 147 | train 0.7283 | val 0.6530 | lr 2.00e-04
Epoch 148 | train 0.7308 | val 0.6508 | lr 2.00e-04
Epoch 149 | train 0.7289 | val 0.6401 | lr 2.00e-04
Epoch 150 | train 0.7281 | val 0.6458 | lr 2.00e-04
Epoch 151 | train 0.7284 | val 0.6393 | lr 2.00e-04
Epoch 152 | train 0.7261 | val 0.6393 | lr 2.00e-04
Epoch 153 | train 0.7265 | val 0.6417 | lr 2.00e-04
Epoch 154 | train 0.7265 | val 0.6483 | lr 2.00e-04
Epoch 155 | train 0.7254 | val 0.6365 | lr 2.00e-04
Epoch 156 | train 0.7276 | val 0.6419 | lr 2.00e-04
Epoch 157 | train 0.7275 | val 0.6429 | lr 2.00e-04
Epoch 158 | train 0.7276 | val 0.6394 | lr 2.00e-04
Epoch 159 | train 0.7263 | val 0.6423 | lr 2.00e-04
Epoch 160 | train 0.7265 | val 0.6355 | lr 2.00e-04
Epoch 161 | train 0.7263 | val 0.6390 | lr 2.00e-04
Epoch 162 | train 0.7249 | val 0.6360 | lr 2.00e-04
Epoch 163 | train 0.7255 | val 0.6394 | lr 2.00e-04
Epoch 164 | train 0.7266 | val 0.6373 | lr 2.00e-04
Epoch 165 | train 0.7257 | val 0.6349 | lr 2.00e-04
Epoch 166 | train 0.7250 | val 0.6389 | lr 2.00e-04
Epoch 167 | train 0.7256 | val 0.6408 | lr 2.00e-04
Epoch 168 | train 0.7252 | val 0.6424 | lr 2.00e-04
Epoch 169 | train 0.7264 | val 0.6386 | lr 2.00e-04
Epoch 170 | train 0.7257 | val 0.6340 | lr 2.00e-04
Epoch 171 | train 0.7256 | val 0.6397 | lr 2.00e-04
Epoch 172 | train 0.7264 | val 0.6416 | lr 2.00e-04
Epoch 173 | train 0.7243 | val 0.6368 | lr 2.00e-04
Epoch 174 | train 0.7234 | val 0.6386 | lr 2.00e-04
Epoch 175 | train 0.7244 | val 0.6368 | lr 2.00e-04
Epoch 176 | train 0.7249 | val 0.6403 | lr 2.00e-04
Epoch 177 | train 0.7228 | val 0.6433 | lr 2.00e-04
Epoch 178 | train 0.7233 | val 0.6334 | lr 2.00e-04
Epoch 179 | train 0.7237 | val 0.6417 | lr 2.00e-04
Epoch 180 | train 0.7241 | val 0.6417 | lr 2.00e-04
Epoch 181 | train 0.7264 | val 0.6385 | lr 2.00e-04
Epoch 182 | train 0.7241 | val 0.6362 | lr 2.00e-04
Epoch 183 | train 0.7228 | val 0.6352 | lr 2.00e-04
Epoch 184 | train 0.7217 | val 0.6395 | lr 2.00e-04
Epoch 185 | train 0.7231 | val 0.6373 | lr 2.00e-04
Epoch 186 | train 0.7241 | val 0.6321 | lr 2.00e-04
Epoch 187 | train 0.7222 | val 0.6396 | lr 2.00e-04
Epoch 188 | train 0.7225 | val 0.6336 | lr 2.00e-04
Epoch 189 | train 0.7208 | val 0.6375 | lr 2.00e-04
Epoch 190 | train 0.7232 | val 0.6321 | lr 2.00e-04
Epoch 191 | train 0.7219 | val 0.6345 | lr 2.00e-04
Epoch 192 | train 0.7223 | val 0.6336 | lr 2.00e-04
Epoch 193 | train 0.7205 | val 0.6322 | lr 2.00e-04
Epoch 194 | train 0.7225 | val 0.6334 | lr 2.00e-04
Epoch 195 | train 0.7213 | val 0.6327 | lr 2.00e-04
Epoch 196 | train 0.7221 | val 0.6325 | lr 2.00e-04
Epoch 197 | train 0.7233 | val 0.6305 | lr 2.00e-04
Epoch 198 | train 0.7232 | val 0.6328 | lr 2.00e-04
Epoch 199 | train 0.7200 | val 0.6312 | lr 2.00e-04
Epoch 200 | train 0.7204 | val 0.6330 | lr 2.00e-04
Epoch 201 | train 0.7205 | val 0.6316 | lr 2.00e-04
Epoch 202 | train 0.7227 | val 0.6313 | lr 2.00e-04
Epoch 203 | train 0.7219 | val 0.6330 | lr 2.00e-04
Epoch 204 | train 0.7214 | val 0.6357 | lr 2.00e-04
Epoch 205 | train 0.7220 | val 0.6370 | lr 2.00e-04
Epoch 206 | train 0.7206 | val 0.6299 | lr 2.00e-04
Epoch 207 | train 0.7226 | val 0.6281 | lr 2.00e-04
Epoch 208 | train 0.7202 | val 0.6300 | lr 2.00e-04
Epoch 209 | train 0.7190 | val 0.6315 | lr 2.00e-04
Epoch 210 | train 0.7198 | val 0.6349 | lr 2.00e-04
Epoch 211 | train 0.7193 | val 0.6308 | lr 2.00e-04
Epoch 212 | train 0.7206 | val 0.6342 | lr 2.00e-04
Epoch 213 | train 0.7212 | val 0.6326 | lr 2.00e-04
Epoch 214 | train 0.7202 | val 0.6238 | lr 2.00e-04
Epoch 215 | train 0.7207 | val 0.6324 | lr 2.00e-04
Epoch 216 | train 0.7202 | val 0.6302 | lr 2.00e-04
Epoch 217 | train 0.7213 | val 0.6305 | lr 2.00e-04
Epoch 218 | train 0.7208 | val 0.6270 | lr 2.00e-04
Epoch 219 | train 0.7208 | val 0.6340 | lr 2.00e-04
Epoch 220 | train 0.7190 | val 0.6280 | lr 2.00e-04
Epoch 221 | train 0.7179 | val 0.6264 | lr 2.00e-04
Epoch 222 | train 0.7177 | val 0.6347 | lr 2.00e-04
Epoch 223 | train 0.7187 | val 0.6284 | lr 2.00e-04
Epoch 224 | train 0.7196 | val 0.6298 | lr 2.00e-04
Epoch 225 | train 0.7193 | val 0.6292 | lr 2.00e-04
Epoch 226 | train 0.7197 | val 0.6309 | lr 2.00e-04
Epoch 227 | train 0.7164 | val 0.6286 | lr 2.00e-04
Epoch 228 | train 0.7204 | val 0.6331 | lr 2.00e-04
Epoch 229 | train 0.7190 | val 0.6309 | lr 2.00e-04
Epoch 230 | train 0.7177 | val 0.6251 | lr 2.00e-04
Epoch 231 | train 0.7194 | val 0.6361 | lr 2.00e-04
Epoch 232 | train 0.7194 | val 0.6284 | lr 2.00e-04
Epoch 233 | train 0.7169 | val 0.6279 | lr 2.00e-04
Epoch 234 | train 0.7185 | val 0.6280 | lr 2.00e-04
No val improvement for 20 epochs, stopping early at epoch 234.
Train | log-space MSE: 0.5977
Val | log-space MSE: 0.6238
Test | log-space MSE: 0.6227
Isoform correlation: mean Pearson 0.5784 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_mlp_pca/isoform_correlation_boxplot_fcnn.png
Scatter correlation log1p (n=50000): 0.8301
Saved log-scale scatter to isoform_model_mlp_pca/pred_vs_true_scatter_log.png
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7699 | median of means 0.7904 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_mlp_pca/gene_rank_corr_mean_boxplot.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.8148 | median of medians 0.8287 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_mlp_pca/gene_rank_corr_median_boxplot.png

Skipping artifact saving to avoid large files.


## PCA + res-MLP

In [ ]:
Loading AnnData files...
Data loaded.
Filtered zero-expression features: genes kept 45244/45263, isoforms kept 161990/162009
Using whole dataset: train 9941, val 6958, test 2983
Applying PCA with 512 components...
PCA complete: (9941, 512)
Explained variance: 0.7216 (72.16%)
Using PCA-transformed inputs: 512 components (explained variance 0.7216, 72.16%)
Epoch 001 | train 1.8699 | val 1.1939
Epoch 002 | train 1.1209 | val 1.0109
Epoch 003 | train 0.9614 | val 0.9170
Epoch 004 | train 0.8746 | val 0.8474
Epoch 005 | train 0.8271 | val 0.8062
Epoch 006 | train 0.7886 | val 0.7715
Epoch 007 | train 0.7567 | val 0.7450
Epoch 008 | train 0.7292 | val 0.7198
Epoch 009 | train 0.7099 | val 0.7096
Epoch 010 | train 0.6921 | val 0.6938
Epoch 011 | train 0.6742 | val 0.6748
Epoch 012 | train 0.6600 | val 0.6598
Epoch 013 | train 0.6480 | val 0.6545
Epoch 014 | train 0.6392 | val 0.6409
Epoch 015 | train 0.6305 | val 0.6339
Epoch 016 | train 0.6212 | val 0.6288
Epoch 017 | train 0.6149 | val 0.6227
Epoch 018 | train 0.6080 | val 0.6157
Epoch 019 | train 0.5994 | val 0.6077
Epoch 020 | train 0.5950 | val 0.6235
Epoch 021 | train 0.5909 | val 0.6027
Epoch 022 | train 0.5844 | val 0.6000
Epoch 023 | train 0.5792 | val 0.6034
Epoch 024 | train 0.5769 | val 0.5933
Epoch 025 | train 0.5709 | val 0.5883
Epoch 026 | train 0.5682 | val 0.5884
Epoch 027 | train 0.5651 | val 0.5877
Epoch 028 | train 0.5640 | val 0.5789
Epoch 029 | train 0.5587 | val 0.5785
Epoch 030 | train 0.5555 | val 0.5763
Epoch 031 | train 0.5547 | val 0.5795
Epoch 032 | train 0.5526 | val 0.5757
Epoch 033 | train 0.5506 | val 0.5719
Epoch 034 | train 0.5479 | val 0.5821
Epoch 035 | train 0.5456 | val 0.5698
Epoch 036 | train 0.5434 | val 0.5782
Epoch 037 | train 0.5419 | val 0.5657
Epoch 038 | train 0.5409 | val 0.5677
Epoch 039 | train 0.5393 | val 0.5656
Epoch 040 | train 0.5362 | val 0.5621
Epoch 041 | train 0.5355 | val 0.5666
Epoch 042 | train 0.5342 | val 0.5617
Epoch 043 | train 0.5329 | val 0.5609
Epoch 044 | train 0.5317 | val 0.5608
Epoch 045 | train 0.5314 | val 0.5594
Epoch 046 | train 0.5296 | val 0.5579
Epoch 047 | train 0.5286 | val 0.5552
Epoch 048 | train 0.5288 | val 0.5578
Epoch 049 | train 0.5268 | val 0.5602
Epoch 050 | train 0.5273 | val 0.5561
Epoch 051 | train 0.5256 | val 0.5559
Epoch 052 | train 0.5252 | val 0.5548
Epoch 053 | train 0.5239 | val 0.5559
Epoch 054 | train 0.5236 | val 0.5536
Epoch 055 | train 0.5222 | val 0.5536
Epoch 056 | train 0.5212 | val 0.5489
Epoch 057 | train 0.5211 | val 0.5513
Epoch 058 | train 0.5200 | val 0.5506
Epoch 059 | train 0.5187 | val 0.5481
Epoch 060 | train 0.5183 | val 0.5511
Epoch 061 | train 0.5182 | val 0.5498
Epoch 062 | train 0.5177 | val 0.5490
Epoch 063 | train 0.5164 | val 0.5508
Epoch 064 | train 0.5161 | val 0.5472
Epoch 065 | train 0.5149 | val 0.5491
Epoch 066 | train 0.5144 | val 0.5489
Epoch 067 | train 0.5140 | val 0.5470
Epoch 068 | train 0.5137 | val 0.5456
Epoch 069 | train 0.5134 | val 0.5457
Epoch 070 | train 0.5125 | val 0.5451
Epoch 071 | train 0.5117 | val 0.5440
Epoch 072 | train 0.5115 | val 0.5437
Epoch 073 | train 0.5106 | val 0.5434
Epoch 074 | train 0.5119 | val 0.5496
Epoch 075 | train 0.5109 | val 0.5443
Epoch 076 | train 0.5092 | val 0.5441
Epoch 077 | train 0.5096 | val 0.5480
Epoch 078 | train 0.5094 | val 0.5417
Epoch 079 | train 0.5084 | val 0.5434
Epoch 080 | train 0.5092 | val 0.5407
Epoch 081 | train 0.5080 | val 0.5419
Epoch 082 | train 0.5080 | val 0.5415
Epoch 083 | train 0.5074 | val 0.5417
Epoch 084 | train 0.5068 | val 0.5415
Epoch 085 | train 0.5064 | val 0.5417
Epoch 086 | train 0.5066 | val 0.5404
Epoch 087 | train 0.5052 | val 0.5407
Epoch 088 | train 0.5053 | val 0.5385
Epoch 089 | train 0.5049 | val 0.5393
Epoch 090 | train 0.5045 | val 0.5394
Epoch 091 | train 0.5041 | val 0.5385
Epoch 092 | train 0.5041 | val 0.5455
Epoch 093 | train 0.5040 | val 0.5384
Epoch 094 | train 0.5038 | val 0.5394
Epoch 095 | train 0.5037 | val 0.5381
Epoch 096 | train 0.5028 | val 0.5381
Epoch 097 | train 0.5028 | val 0.5379
Epoch 098 | train 0.5022 | val 0.5360
Epoch 099 | train 0.5018 | val 0.5374
Epoch 100 | train 0.5016 | val 0.5364
Epoch 101 | train 0.5010 | val 0.5368
Epoch 102 | train 0.5014 | val 0.5341
Epoch 103 | train 0.5005 | val 0.5376
Epoch 104 | train 0.5004 | val 0.5369
Epoch 105 | train 0.5007 | val 0.5357
Epoch 106 | train 0.4995 | val 0.5372
Epoch 107 | train 0.5007 | val 0.5327
Epoch 108 | train 0.4992 | val 0.5364
Epoch 109 | train 0.5002 | val 0.5366
Epoch 110 | train 0.4997 | val 0.5369
Epoch 111 | train 0.4991 | val 0.5343
Epoch 112 | train 0.4990 | val 0.5356
Epoch 113 | train 0.4989 | val 0.5340
Epoch 114 | train 0.4981 | val 0.5338
Epoch 115 | train 0.4982 | val 0.5327
Epoch 116 | train 0.4977 | val 0.5327
Epoch 117 | train 0.4977 | val 0.5337
Epoch 118 | train 0.4977 | val 0.5344
Epoch 119 | train 0.4971 | val 0.5346
Epoch 120 | train 0.4972 | val 0.5317
Epoch 121 | train 0.4972 | val 0.5356
Epoch 122 | train 0.4984 | val 0.5300
Epoch 123 | train 0.4959 | val 0.5333
Epoch 124 | train 0.4967 | val 0.5337
Epoch 125 | train 0.4969 | val 0.5306
Epoch 126 | train 0.4966 | val 0.5334
Epoch 127 | train 0.4959 | val 0.5337
Epoch 128 | train 0.4961 | val 0.5327
Epoch 129 | train 0.4962 | val 0.5331
Epoch 130 | train 0.4959 | val 0.5285
Epoch 131 | train 0.4955 | val 0.5297
Epoch 132 | train 0.4950 | val 0.5315
Epoch 133 | train 0.4951 | val 0.5350
Epoch 134 | train 0.4960 | val 0.5319
Epoch 135 | train 0.4950 | val 0.5319
Epoch 136 | train 0.4948 | val 0.5312
Epoch 137 | train 0.4952 | val 0.5310
Epoch 138 | train 0.4945 | val 0.5281
Epoch 139 | train 0.4941 | val 0.5315
Epoch 140 | train 0.4940 | val 0.5304
Epoch 141 | train 0.4948 | val 0.5301
Epoch 142 | train 0.4954 | val 0.5318
Epoch 143 | train 0.4940 | val 0.5320
Epoch 144 | train 0.4941 | val 0.5297
Epoch 145 | train 0.4936 | val 0.5319
Epoch 146 | train 0.4941 | val 0.5295
Epoch 147 | train 0.4937 | val 0.5318
Epoch 148 | train 0.4953 | val 0.5326
Epoch 149 | train 0.4945 | val 0.5304
Epoch 150 | train 0.4933 | val 0.5296
Epoch 151 | train 0.4933 | val 0.5302
Epoch 152 | train 0.4935 | val 0.5305
Epoch 153 | train 0.4931 | val 0.5289
Epoch 154 | train 0.4930 | val 0.5314
Epoch 155 | train 0.4932 | val 0.5289
Epoch 156 | train 0.4932 | val 0.5310
Epoch 157 | train 0.4929 | val 0.5300
Epoch 158 | train 0.4936 | val 0.5310
No val improvement for 20 epochs, stopping early at epoch 158.
Train | log-space MSE: 0.4630
Val | log-space MSE: 0.5281
Test | log-space MSE: 0.5283
Isoform correlation: mean Pearson 0.6237 over 161990/161990 isoforms
Saved isoform correlation boxplot to isoform_model_residual_pca/isoform_correlation_boxplot_pca_residual.png
Scatter correlation log1p (n=50000): 0.8478
Saved log-scale scatter to isoform_model_residual_pca/pred_vs_true_scatter_residual_pca_log.png
Per-gene isoform rank correlation (Spearman over observations): mean of means 0.7869 | median of means 0.8062 over 899 genes
Saved per-gene mean rank correlation boxplot to isoform_model_residual_pca/gene_rank_corr_mean_boxplot_residual_pca.png
Per-gene isoform rank correlation (Spearman over observations): mean of medians 0.8300 | median of medians 0.8476 over 899 genes
Saved per-gene median rank correlation boxplot to isoform_model_residual_pca/gene_rank_corr_median_boxplot_residual_pca.png

Skipping artifact saving to avoid large files.
